# Ventilation Patient Environment
- Builds a state transition dictionary (experiment: bin strategies, observation hours, and variables)
- Implements approximation strategies for unseen state-action pairs
- Designs rewards based on ventilation weaning (extubation) guidelines to build a Python Gym environment
- Generates trajectories by interacting with different offline policy doctor agents
- Visualizes trajectories with transition action options
- Visualizes trajectories with the same initial state (digital twin concept) to demonstrate evaluation

## Import library, Settings and configuration, Read in data

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.manifold import TSNE

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

In [ ]:
# ================================
# Settings and configuration
# ================================
bins = 10                          # number of bins for discretization
bins_strategy = "s2"               # "s1": linearly cut, "s2": NEWS score
obs_hrs = 3                        # number of observation hours (e.g., current + previous)
subset_size = 10000                # number of distinct stay_id to use (sample a subset)
train_percentage = 0.7             # percentage of data to use for training : testing -> 7 : 3
test_cases_num = 1000              # number of test cases to generate
id_cols = ['stay_id', 'before_weaning_hr']
feature_cols = ['heart_rate', 'resp_rate', 'spo2', 'fio2', 'respiratory_rate_set', 'gender_M', 'age'] # TODO: add gender, age, peep, (tidal_volume_set, tidal_volume_observed, sbp, dbp, mbp)
selected_cols = id_cols + feature_cols
state_cols = ['heart_rate', 'resp_rate', 'spo2']  # State variables
action_cols = ['fio2', 'respiratory_rate_set']  # Action variables
state_units = ['bpm', 'bpm', '%']  # Units for state variables
action_units = ['%', 'bpm']  # Units for action variables
baseline_cols = ['gender_M', 'age']  # Baseline variables

GROUND_TRUTH_WITH_NEIGHBORS = True  # Use ground truth neighbors for evaluation, False for run different K for finding best neighbors

In [ ]:
# ================================
# Data reading
# ================================
# Define data paths (update if needed)
eICU_prefix_path = "../data/eICU"
mimic_iii_prefix_path = "../data/mimic_iii"
mimic_iv_prefix_path = "../data/mimic_iv"

eICU_file = os.path.join(eICU_prefix_path, "baseline_charttime_ground_truth.csv")
mimic_iv_file = os.path.join(mimic_iv_prefix_path, "baseline_charttime_ground_truth.csv")
mimic_iii_file = os.path.join(mimic_iii_prefix_path, "baseline_charttime_ground_truth.csv")

# Read dataframes
eICU_df = pd.read_csv(eICU_file)
mimic_iv_df = pd.read_csv(mimic_iv_file)
mimic_iii_df = pd.read_csv(mimic_iii_file)

# ================================
# Add "hours_in" column (if missing)
# ================================
def add_hours_in(baseline_df):
    # if hours_in not present, create it based on before_weaning_hr
    if 'hours_in' not in list(baseline_df.columns):
        vital_sign_df = baseline_df[selected_cols]
        vital_sign_df_with_hours = vital_sign_df.copy()
        # For each stay_id, use the maximum before_weaning_hr as baseline
        max_before_weaning = vital_sign_df.groupby('stay_id')['before_weaning_hr'].transform('max')
        vital_sign_df_with_hours['hours_in'] = max_before_weaning - vital_sign_df['before_weaning_hr']
        vital_sign_df_with_hours = vital_sign_df_with_hours.sort_values(['stay_id', 'hours_in'])
        vital_sign_df_with_hours.drop(columns=['before_weaning_hr'], inplace=True)
        cols = list(vital_sign_df_with_hours.columns)
        # Move hours_in to position 1 (after stay_id)
        if 'hours_in' in cols:
            cols.remove('hours_in')
        cols.insert(1, 'hours_in')
        baseline_df = vital_sign_df_with_hours[cols]
    baseline_df['gender_M'] = baseline_df['gender_M'].astype(int)
    return baseline_df

# Apply to all datasets
eICU_df = add_hours_in(eICU_df)
mimic_iv_df = add_hours_in(mimic_iv_df)
mimic_iii_df = add_hours_in(mimic_iii_df)

In [ ]:
mimic_iv_df

In [ ]:
mimic_iv_df.describe()

In [ ]:
# check the hours_in distribution for each stay_id (only pick the max hours_in)
def check_hours_in_distribution(baseline_df):
    # Check the distribution of hours_in for each stay_id
    stay_id_distribution = baseline_df.groupby('stay_id')['hours_in'].max().reset_index()
    stay_id_distribution['hours_in'] = stay_id_distribution['hours_in'].astype(int)
    return stay_id_distribution
eICU_hours_in_distribution = check_hours_in_distribution(eICU_df)
mimic_iv_hours_in_distribution = check_hours_in_distribution(mimic_iv_df)
mimic_iii_hours_in_distribution = check_hours_in_distribution(mimic_iii_df)
# Plot the distribution of hours_in for each stay_id
def plot_hours_in_distribution(hours_in_distribution, title):
    plt.figure(figsize=(10, 6))
    plt.hist(hours_in_distribution['hours_in'], bins=30, color='blue', alpha=0.7)
    plt.title(title)
    plt.xlabel('Hours In')
    plt.ylabel('Frequency')
    plt.grid()
    plt.show()
plot_hours_in_distribution(eICU_hours_in_distribution, "eICU Hours In Distribution")
plot_hours_in_distribution(mimic_iv_hours_in_distribution, "MIMIC-IV Hours In Distribution")
plot_hours_in_distribution(mimic_iii_hours_in_distribution, "MIMIC-III Hours In Distribution")

## State transition dictionary
- (experiment: bin strategies, observation hours, and variables)

### Bin strategies
- Baseline:
    - age: 5 bin (age_bins = np.linspace(20, 100, 6))
    - gender: binary
- State:
    - strategy 1: linear cut
    - strategy 2: NEWS for reference boundary
    - (advanced: Chi-Merge / similar transition behavior for finding the varibles interval)
- Action:
    - (strategy 1: qcut with 10 bin, can be less than 10 bins)
    - strategy 2:
        - fio2, (peep or others) use 10 bins
        - resp rate set: same as resp_rate (due to visualization)

In [ ]:
# ================================
# Data visualization
# ================================
def visualize_data(df, feature_cols):
    # Plot histograms for each feature
    n_features = len(feature_cols)
    fig, axes = plt.subplots(nrows=n_features, ncols=1, figsize=(12, 5 * n_features), sharex=False)

    if n_features == 1:
        axes = [axes]  # Ensure axes is iterable

    for idx, col in enumerate(feature_cols):
        ax = axes[idx]
        df[col].hist(bins=30, ax=ax, color='blue', alpha=0.7)
        ax.set_title(f'Distribution of {col}', fontsize=14)
        ax.set_xlabel(col, fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.show()
# Visualize data for mimic_iv_df
visualize_data(mimic_iv_df, feature_cols)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define min_max_values as provided (from MIMIC-IV)
min_max_values = {
    "fio2": {"min": 21.0, "max": 100.0},
    "respiratory_rate_set": {"min": 5.0, "max": 40.0},
    "heart_rate": {"min": 37.0, "max": 180.0},
    "resp_rate": {"min": 4.5, "max": 36.5},
    "spo2": {"min": 80.0, "max": 100.0},
    "age": {"min": 20.0, "max": 100.0}
}

# 1. Baseline: Age discretization
def discretize_age(df):
    age_bins = np.linspace(20, 100, 6)  # 5 bins
    df['age'] = pd.cut(df['age'], bins=age_bins, labels=range(len(age_bins) - 1), include_lowest=True).astype('Int64')
    return df, age_bins

# 2. Action Discretization Function
def discretize_actions(df, action_cols, bins):
    bin_edges_dict = {
        'fio2': [21, 30, 35, 40, 50, 60, 70, 80, 90, 100],
        # 'fio2': [21, 30, 35, 40, 50, 60, 70, 80, 90, 100, float('inf')],
        'respiratory_rate_set': [0, 4, 8, 11, 14, 17, 20, 24, 27, 30, 33, 40]
        # 'respiratory_rate_set': [0, 4, 8, 11, 14, 17, 20, 24, 27, 30, 33, 37, float('inf')]
    }
    df_disc = df.copy()
    for col in action_cols:
        if col in bin_edges_dict:
            bin_edges = bin_edges_dict[col]
            try:
                # Use pd.cut with fixed bin edges
                df_disc[col] = pd.cut(
                    df_disc[col].clip(lower=bin_edges[0], upper=bin_edges[-1]),  # Clip values to the range
                    bins=bin_edges,
                    labels=range(len(bin_edges) - 1),
                    include_lowest=True
                ).astype('Int64')
            except ValueError:
                # Handle cases where pd.cut fails (e.g., not enough unique values)
                df_disc[col] = pd.NA
        else:
            # If no bin edges are provided for the column, set it to NaN
            df_disc[col] = pd.NA
    return df_disc, bin_edges_dict

# 3.a. State and Action: Strategy 1 - Equal-width bins
def get_bin_edges_strategy1(df, feature_cols, min_max_values, bins): # TODO: make sure bin for eICU won't cause Nan
    """
    Generate bin edges for state and action variables using equal-width bins.

    Args:
        df (pd.DataFrame): Input DataFrame.
        feature_cols (list): List of feature columns (state + action variables).
        min_max_values (dict): Dictionary containing min and max values for each feature.
        bins (int): Number of bins.

    Returns:
        dict: Dictionary containing bin edges for each feature.
    """
    bin_edges_dict = {}
    for col in feature_cols:
        if col in min_max_values:
            col_min = min(df[col].min(), min_max_values[col]["min"])
            col_max = max(df[col].max(), min_max_values[col]["max"])
        else:
            col_min = df[col].min()
            col_max = df[col].max()
        bin_edges = np.linspace(col_min, col_max, bins + 1)
        bin_edges_dict[col] = bin_edges
    return bin_edges_dict

# 3.b. State and Action: Strategy 2 - Fixed NEWS boundaries for state, equal-width for action
def get_bin_edges_strategy2(action_cols, min_max_values, bins):
    """
    Generate bin edges for state variables using fixed NEWS boundaries and action variables using equal-width bins.

    Args:
        action_cols (list): List of action variable columns.
        min_max_values (dict): Dictionary containing min and max values for each feature.
        bins (int): Number of bins for action variables.

    Returns:
        dict: Dictionary containing bin edges for each feature.
    """

    # Fixed NEWS boundaries for state variables v2
    bin_edges_dict = {
        'heart_rate': [0, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 180],
        'resp_rate': [0, 4, 8, 11, 14, 17, 20, 24, 27, 30, 33, 40],
        'spo2': [0, 80, 90, 91, 93, 95, 98, 100],
        'fio2': [21, 30, 35, 40, 50, 60, 70, 80, 90, 100],
        'respiratory_rate_set': [0, 4, 8, 11, 14, 17, 20, 24, 27, 30, 33, 40],
        'gender_M': [0, 1, 2], # TODO: not sure
        'age':[20, 36, 52, 68, 84, 100],
    }

    return bin_edges_dict

# Function to discretize using fixed bin edges
def discretize_with_fixed_bins(df, feature_cols, bin_edges_dict):
    df_disc = df.copy()
    for col in feature_cols:
        bin_edges = bin_edges_dict[col]
        labels = list(range(len(bin_edges) - 1))  # 0 to bins-1
        df_disc[col] = pd.cut(
            df_disc[col].clip(lower=bin_edges[0], upper=bin_edges[-1]),  # Clip values to the range
            bins=bin_edges,
            labels=list(range(len(bin_edges) - 1)),  # 0 to bins-1
            include_lowest=True
        ).astype('Int64')
    return df_disc

# 4. Calculate bin edges for Strategy 1 using mimic_iv_df
bin_edges_dict_s1 = get_bin_edges_strategy1(mimic_iv_df, feature_cols, min_max_values, bins)

# 4. Calculate bin edges for Strategy 2
bin_edges_dict_s2 = get_bin_edges_strategy2(action_cols, min_max_values, bins)

# 4. Discretize all datasets using Strategy 1
eICU_disc_s1 = discretize_with_fixed_bins(eICU_df, state_cols, bin_edges_dict_s1)
mimic_iv_disc_s1 = discretize_with_fixed_bins(mimic_iv_df, state_cols, bin_edges_dict_s1)
mimic_iii_disc_s1 = discretize_with_fixed_bins(mimic_iii_df, state_cols, bin_edges_dict_s1)

# 4. Discretize all datasets using Strategy 2
eICU_disc_s2 = discretize_with_fixed_bins(eICU_df, state_cols, bin_edges_dict_s2)
mimic_iv_disc_s2 = discretize_with_fixed_bins(mimic_iv_df, state_cols, bin_edges_dict_s2)
mimic_iii_disc_s2 = discretize_with_fixed_bins(mimic_iii_df, state_cols, bin_edges_dict_s2)

# 5. Visualization: Compare bin edges for different strategies
def compare_bin_edges(bin_edges_dict_s1, bin_edges_dict_s2, state_cols):
    for col in state_cols:
        print(f"\nFeature: {col}")
        print(f"Strategy 1 (Equal-Width): {bin_edges_dict_s1[col]}")
        print(f"Strategy 2 (Fixed NEWS): {bin_edges_dict_s2[col]}")

compare_bin_edges(bin_edges_dict_s1, bin_edges_dict_s2, state_cols)

# 6. Visualization: Compare distributions for different strategies
def plot_discretization_comparison(df_s1, df_s2, state_cols):
    n_features = len(state_cols)
    fig, axes = plt.subplots(nrows=n_features, ncols=1, figsize=(12, 5 * n_features), sharex=False)

    if n_features == 1:
        axes = [axes]  # Ensure axes is iterable

    for idx, col in enumerate(state_cols):
        ax = axes[idx]
        s1_data = df_s1[col].dropna()
        s2_data = df_s2[col].dropna()

        bins_range = np.arange(max(s1_data.max(), s2_data.max()) + 2)  # +2 to include the last bin edge
        width = 0.4

        s1_counts, _ = np.histogram(s1_data, bins=bins_range)
        s2_counts, _ = np.histogram(s2_data, bins=bins_range)

        x = np.arange(len(bins_range) - 1)
        ax.bar(x - width / 2, s1_counts, width=width, label='Strategy 1 (Equal-Width)', color='blue', alpha=0.7)
        ax.bar(x + width / 2, s2_counts, width=width, label='Strategy 2 (Fixed NEWS)', color='green', alpha=0.7)

        ax.set_title(f'Discretization Comparison for {col}', fontsize=14)
        ax.set_xlabel('Bin Number', fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()
    plt.show()

# Plot comparison for mimic_iv_df
plot_discretization_comparison(mimic_iv_disc_s1, mimic_iv_disc_s2, state_cols)

# 7. Discretize actions
mimic_iv_disc_s1, action_bins_dict_s1 = discretize_actions(mimic_iv_disc_s1, action_cols, bins)
mimic_iv_disc_s2, action_bins_dict_s2 = discretize_actions(mimic_iv_disc_s2, action_cols, bins)
mimic_iii_disc_s1, action_bins_dict_s1 = discretize_actions(mimic_iii_disc_s1, action_cols, bins)
mimic_iii_disc_s2, action_bins_dict_s2 = discretize_actions(mimic_iii_disc_s2, action_cols, bins)
eICU_disc_s1, action_bins_dict_s1 = discretize_actions(eICU_disc_s1, action_cols, bins)
eICU_disc_s2, action_bins_dict_s2 = discretize_actions(eICU_disc_s2, action_cols, bins)

# 8. Discretize age
mimic_iv_disc_s1, age_bins = discretize_age(mimic_iv_disc_s1)
mimic_iv_disc_s2, age_bins = discretize_age(mimic_iv_disc_s2)
mimic_iii_disc_s1, age_bins = discretize_age(mimic_iii_disc_s1)
mimic_iii_disc_s2, age_bins = discretize_age(mimic_iii_disc_s2)
eICU_disc_s1, age_bins = discretize_age(eICU_disc_s1)
eICU_disc_s2, age_bins = discretize_age(eICU_disc_s2)

# 9. Plot action and age discretization
def plot_action_age_discretization(df, action_cols):
    n_actions = len(action_cols)
    fig, axes = plt.subplots(nrows=n_actions + 1, ncols=1, figsize=(12, 5 * (n_actions + 1)), sharex=False)
    if n_actions == 1:
        axes = [axes]
    for idx, col in enumerate(action_cols): 
        ax = axes[idx]
        df[col].hist(bins=30, ax=ax, color='blue', alpha=0.7)
        ax.set_title(f'Distribution of {col}', fontsize=14)
        ax.set_xlabel(col, fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.grid(True, alpha=0.3, linestyle='--')
    ax = axes[-1]
    df['age'].value_counts().sort_index().plot(kind='bar', ax=ax, color='blue', alpha=0.7)
    ax.set_title('Age Discretization', fontsize=14)
    ax.set_xlabel('Age Bin', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
# Plot action and age discretization for mimic_iv_df
plot_action_age_discretization(mimic_iv_disc_s1, action_cols)

- Select the strategy for using

In [ ]:
# TODO: set s1 or s2 or others
if bins_strategy == "s1":
    eICU_disc = eICU_disc_s1.copy()
    mimic_iv_disc = mimic_iv_disc_s1.copy()
    mimic_iii_disc = mimic_iii_disc_s1.copy()
    num_bins_dict = {key: len(edges) - 1 for key, edges in bin_edges_dict_s1.items()}
elif bins_strategy == "s2":
    eICU_disc = eICU_disc_s2.copy()
    mimic_iv_disc = mimic_iv_disc_s2.copy()
    mimic_iii_disc = mimic_iii_disc_s2.copy()
    num_bins_dict = {key: len(edges) - 1 for key, edges in bin_edges_dict_s2.items()}

In [ ]:
num_bins_dict

In [ ]:
eICU_disc.describe()

In [ ]:
num_bins_dict

In [ ]:
bin_edges_dict_s2

In [ ]:
mimic_iv_disc.describe()

In [ ]:
mimic_iv_disc

### Select subset / train / test for expericement

In [ ]:
# ================================
# Sample a subset of distinct stay_id (applied to one dataset; you can repeat for others)
# ================================
def sample_subset(df, subset_size):
    unique_ids = df['stay_id'].unique()
    sampled_ids = random.sample(list(unique_ids), min(subset_size, len(unique_ids)))
    return df[df['stay_id'].isin(sampled_ids)]

# Use subset for further analysis (example uses mimic_iv_disc) -> use for debugging
# mimic_iv_disc = sample_subset(mimic_iv_disc, subset_size)
# print(f"Subset shape: {mimic_iv_disc.shape}")

In [ ]:
len(list(mimic_iv_disc['stay_id'].unique()))

In [ ]:
# Split MIMIC-IV for internal evaluation
train_ids = random.sample(list(mimic_iv_disc['stay_id'].unique()), int(train_percentage * len(list(mimic_iv_disc['stay_id'].unique()))))
test_ids = [sid for sid in mimic_iv_disc['stay_id'].unique() if sid not in train_ids]
train_df = mimic_iv_disc[mimic_iv_disc['stay_id'].isin(train_ids)]
test_df = mimic_iv_disc[mimic_iv_disc['stay_id'].isin(test_ids)]

# [cont_update]
train_cont_df = mimic_iv_df[mimic_iv_df['stay_id'].isin(train_ids)]
test_cont_df = mimic_iv_df[mimic_iv_df['stay_id'].isin(test_ids)]

##### Train : Test (on MIMIC IV) -> 7 : 3

In [ ]:
train_df['stay_id'].nunique(), test_df['stay_id'].nunique()

In [ ]:
# save the train_ids and test_ids
train_ids_df = pd.DataFrame(train_ids, columns=['train_ids'])
test_ids_df = pd.DataFrame(test_ids, columns=['test_ids'])
train_ids_df.to_csv(os.path.join(mimic_iv_prefix_path, 'train_ids.csv'), index=False)
test_ids_df.to_csv(os.path.join(mimic_iv_prefix_path, 'test_ids.csv'), index=False)

### In / Out of Distribution Analysis

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp

# Assume train_df, test_df, eICU_disc are already loaded as pandas DataFrames

# 1. Summary statistics
def summary_stats(df, name):
    print(f"Summary statistics for {name}:")
    print(df.describe())
    print("\n")

summary_stats(train_df, "train_df")
summary_stats(test_df, "test_df")
summary_stats(eICU_disc, "eICU_disc")

# 2. Kolmogorov-Smirnov test for each numerical column
def ks_test(df1, df2, columns, name1, name2):
    print(f"KS test between {name1} and {name2}:")
    for col in columns:
        stat, p = ks_2samp(df1[col], df2[col])
        print(f"{col}: KS statistic={stat:.3f}, p-value={p:.3e}")
    print("\n")

num_cols = ['heart_rate', 'resp_rate', 'spo2', 'fio2', 'respiratory_rate_set', 'age']

# Train vs Test (should be similar, high p-values)
ks_test(train_df, test_df, num_cols, "train_df", "test_df")

# Train vs eICU (should be different, low p-values)
ks_test(train_df, eICU_disc, num_cols, "train_df", "eICU_disc")

# Test vs eICU (should be different, low p-values)
ks_test(test_df, eICU_disc, num_cols, "test_df", "eICU_disc")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Assume mimic_iv_df and eICU_df are already loaded

# 1. Visualize distributions
cols = ['heart_rate', 'resp_rate', 'spo2', 'fio2', 'respiratory_rate_set', 'age']

for col in cols:
    plt.figure(figsize=(6,3))
    sns.kdeplot(mimic_iv_df[col], label='MIMIC-IV', fill=True)
    sns.kdeplot(eICU_df[col], label='eICU', fill=True)
    plt.title(f'Distribution of {col}')
    plt.legend()
    plt.show()

# 2. Kolmogorov-Smirnov test for each variable
print("Kolmogorov-Smirnov test results:")
for col in cols:
    stat, p = ks_2samp(mimic_iv_df[col].dropna(), eICU_df[col].dropna())
    print(f"{col}: KS statistic={stat:.3f}, p-value={p:.3e}")

### Construct state & transition structures with KDTree

In [ ]:
# ================================
# Construct state & transition structures
# ================================
# Here we define a simple approach to create state with pre-state-action for obs_hrs.
# For each stay_id, sort by hours_in and then generate tuples.
from collections import defaultdict
from sklearn.neighbors import KDTree

def create_state_transitions(df, obs_hrs, state_cols, action_cols, baseline_cols=None):
    """
    Create state transitions with flexibility to add state, action, and baseline variables.

    Args:
        df (pd.DataFrame): Input dataframe containing the data.
        obs_hrs (int): Number of observation hours to include in the state history.
        state_cols (list): List of columns to include in the state.
        action_cols (list): List of columns to include in the action.
        baseline_cols (list, optional): List of baseline columns (e.g., age, gender) to include. Defaults to None.

    Returns:
        dict: A dictionary where keys are tuples of (state, action) history and values are lists of next states.
    """
    transitions = defaultdict(list)
    
    # Group by stay_id
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in')
        rows = group.to_dict('records')
        
        # Sliding window with length obs_hrs+1 to get (state/action history, next_state)
        for i in range(len(rows) - obs_hrs):
            window = rows[i:i+obs_hrs+1]
            
            # Build state_with_pre_state_action: for each of the first obs_hrs rows
            key = []
            for j in range(obs_hrs):
                state = tuple(window[j][col] for col in state_cols)  # Extract state variables
                action = tuple(window[j][col] for col in action_cols)  # Extract action variables
                key.append((state, action))
            
            # Add baseline variables (if provided) to the key
            if baseline_cols:
                baseline = tuple(window[0][col] for col in baseline_cols)  # Use the first row for baseline variables
                key.append(baseline)
            
            # The current state for the last row
            curr_state = tuple(window[obs_hrs][col] for col in state_cols)
            
            # Store transition: key -> next_state
            transitions[tuple(key)].append(curr_state)
    
    return transitions

def flatten_key(key):
    # Flatten the key for KDTree
    flat = []
    for element in key[:-1]:  # Iterate over all elements except the last (state-action history)
        state, action = element
        flat.extend([int(x) for x in state])  # Convert state values to int
        flat.extend([int(x) for x in action])  # Convert action values to int
    flat.extend([int(x) for x in key[-1]])  # Convert baseline values to int
    return np.array(flat, dtype=np.int32)

def create_state_transitions_with_stay_id_mapping(df, obs_hrs, state_cols, action_cols, baseline_cols=None):
    """
    Create state transitions and a mapping of transition keys to stay_id.

    Args:
        df (pd.DataFrame): Input dataframe containing the data.
        obs_hrs (int): Number of observation hours to include in the state history.
        state_cols (list): List of columns to include in the state.
        action_cols (list): List of columns to include in the action.
        baseline_cols (list, optional): List of baseline columns (e.g., age, gender) to include. Defaults to None.

    Returns:
        dict: A dictionary where keys are tuples of (state, action) history and values are lists of next states.
        dict: A dictionary mapping each transition key to a list of associated stay_id.
    """
    transitions = defaultdict(list)
    stay_id_mapping = defaultdict(list)

    # Group by stay_id
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in')
        rows = group.to_dict('records')

        # Sliding window with length obs_hrs+1 to get (state/action history, next_state)
        for i in range(len(rows) - obs_hrs):
            window = rows[i:i+obs_hrs+1]

            # Build state_with_pre_state_action: for each of the first obs_hrs rows
            key = []
            for j in range(obs_hrs):
                state = tuple(window[j][col] for col in state_cols)  # Extract state variables
                action = tuple(window[j][col] for col in action_cols)  # Extract action variables
                key.append((state, action))

            # Add baseline variables (if provided) to the key
            if baseline_cols:
                baseline = tuple(window[0][col] for col in baseline_cols)  # Use the first row for baseline variables
                key.append(baseline)

            # The current state for the last row
            curr_state = tuple(window[obs_hrs][col] for col in state_cols)

            # Store transition: key -> next_state
            transitions[tuple(key)].append(curr_state)

            # Map transition key to stay_id
            stay_id_mapping[tuple(key)].append(stay_id)

    return transitions, stay_id_mapping

def filter_transitions_by_stay_id(transitions, stay_id_mapping, min_stay_ids=2):
    """
    Filter transitions based on the number of distinct stay_id associated with each key.

    Args:
        transitions (dict): Dictionary of transitions where keys are transition keys and values are lists of next states.
        stay_id_mapping (dict): Dictionary mapping each transition key to a list of associated stay_id.
        min_stay_ids (int): Minimum number of distinct stay_id required for a key to be included.

    Returns:
        dict: Filtered transitions containing only keys with at least min_stay_ids distinct stay_id.
    """
    filtered_transitions = {}
    for key, next_states in transitions.items():
        if len(set(stay_id_mapping.get(key, []))) >= min_stay_ids:
            filtered_transitions[key] = next_states
    return filtered_transitions

In [ ]:
# Example usage: transitions for whole mimic_iv_disc
transitions = create_state_transitions(mimic_iv_disc, obs_hrs, state_cols, action_cols, baseline_cols)
print(f"Number of unique state_with_pre_state_action keys: {len(transitions)}")

# [cont_update]
transitions_cont = create_state_transitions(mimic_iv_df, obs_hrs, state_cols, action_cols, baseline_cols)
print(f"Number of unique state_with_pre_state_action keys: {len(transitions_cont)}")


In [ ]:
# Example of how to access the transitions
# [ ((state_1, action_1), (state_2, action_2), ..., (state_obs_hrs, action_obs_hrs), (age, gender)) ]
for key in list(transitions.keys())[:5]:  # Display first 5 keys
    print(f"Key: {key}, Next States: {transitions[key]}")

# [cont_update]
for key in list(transitions_cont.keys())[:5]:  # Display first 5 keys
    print(f"Key: {key}, Next States: {transitions_cont[key]}")

##### generate transitions for different dataset

In [ ]:
train_transitions, train_stay_id_mapping = create_state_transitions_with_stay_id_mapping(train_df, obs_hrs, state_cols, action_cols, baseline_cols)
test_transitions, test_stay_id_mapping = create_state_transitions_with_stay_id_mapping(test_df, obs_hrs, state_cols, action_cols, baseline_cols)
eicu_trans, eicu_stay_id_mapping = create_state_transitions_with_stay_id_mapping(eICU_disc, obs_hrs, state_cols, action_cols, baseline_cols) 
mimic_iii_trans, mimic_iii_stay_id_mapping = create_state_transitions_with_stay_id_mapping(mimic_iii_disc, obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iii_trans, eicu_mimic_iii_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_disc, mimic_iii_disc]), obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iii_mimic_iv_train_trans, eicu_mimic_iii_mimic_iv_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_disc, mimic_iii_disc, train_df]), obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iv_train_trans, eicu_mimic_iv_train_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_disc, train_df]), obs_hrs, state_cols, action_cols, baseline_cols)

# [cont_update]
train_cont_transitions, train_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(train_cont_df, obs_hrs, state_cols, action_cols, baseline_cols)
test_cont_transitions, test_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(test_cont_df, obs_hrs, state_cols, action_cols, baseline_cols)
eicu_cont_trans, eicu_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(eICU_df, obs_hrs, state_cols, action_cols, baseline_cols) 
mimic_iii_cont_trans, mimic_iii_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(mimic_iii_df, obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iii_cont_trans, eicu_mimic_iii_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_df, mimic_iii_df]), obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iii_mimic_iv_train_cont_trans, eicu_mimic_iii_mimic_iv_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_df, mimic_iii_df, train_cont_df]), obs_hrs, state_cols, action_cols, baseline_cols)
eicu_mimic_iv_train_cont_trans, eicu_mimic_iv_train_cont_stay_id_mapping = create_state_transitions_with_stay_id_mapping(pd.concat([eICU_df, train_cont_df]), obs_hrs, state_cols, action_cols, baseline_cols)

In [ ]:
train_keys_list = list(train_transitions.keys())
train_X = np.array([flatten_key(key) for key in train_transitions.keys()])
train_tree = KDTree(train_X, metric='manhattan')

# [gt_with_neighbors_update] build KDTree for calculate the ground truth
test_keys_list = list(test_transitions.keys())
test_X = np.array([flatten_key(key) for key in test_transitions.keys()])
test_tree = KDTree(test_X, metric='manhattan')

eicu_keys_list = list(eicu_trans.keys())
eicu_X = np.array([flatten_key(key) for key in eicu_trans.keys()])
eicu_tree = KDTree(eicu_X, metric='manhattan')

eicu_mimic_iii_keys_list = list(eicu_mimic_iii_trans.keys())
eicu_mimic_iii_X = np.array([flatten_key(key) for key in eicu_mimic_iii_trans.keys()])
eicu_mimic_iii_tree = KDTree(eicu_mimic_iii_X, metric='manhattan')

eicu_mimic_iii_mimic_iv_train_keys_list = list(eicu_mimic_iii_mimic_iv_train_trans.keys())
eicu_mimic_iii_mimic_iv_train_X = np.array([flatten_key(key) for key in eicu_mimic_iii_mimic_iv_train_trans.keys()])
eicu_mimic_iii_mimic_iv_train_tree = KDTree(eicu_mimic_iii_mimic_iv_train_X, metric='manhattan')

eicu_mimic_iv_train_keys_list = list(eicu_mimic_iv_train_trans.keys())
eicu_mimic_iv_train_X = np.array([flatten_key(key) for key in eicu_mimic_iv_train_trans.keys()])
eicu_mimic_iv_train_tree = KDTree(eicu_mimic_iv_train_X, metric='manhattan')

In [ ]:
# [cont_update]
train_cont_keys_list = list(train_cont_transitions.keys())
train_cont_X = np.array([flatten_key(key) for key in train_cont_transitions.keys()])
train_cont_tree = KDTree(train_cont_X, metric='manhattan')

# [gt_with_neighbors_update] build KDTree for calculate the ground truth
test_cont_keys_list = list(test_cont_transitions.keys())
test_cont_X = np.array([flatten_key(key) for key in test_cont_transitions.keys()])
test_cont_tree = KDTree(test_cont_X, metric='manhattan')

eicu_cont_keys_list = list(eicu_cont_trans.keys())
eicu_cont_X = np.array([flatten_key(key) for key in eicu_cont_trans.keys()])
eicu_cont_tree = KDTree(eicu_cont_X, metric='manhattan')

eicu_mimic_iii_cont_keys_list = list(eicu_mimic_iii_cont_trans.keys())
eicu_mimic_iii_cont_X = np.array([flatten_key(key) for key in eicu_mimic_iii_cont_trans.keys()])
eicu_mimic_iii_cont_tree = KDTree(eicu_mimic_iii_cont_X, metric='manhattan')

eicu_mimic_iii_mimic_iv_train_cont_keys_list = list(eicu_mimic_iii_mimic_iv_train_cont_trans.keys())
eicu_mimic_iii_mimic_iv_train_cont_X = np.array([flatten_key(key) for key in eicu_mimic_iii_mimic_iv_train_cont_trans.keys()])
eicu_mimic_iii_mimic_iv_train_cont_tree = KDTree(eicu_mimic_iii_mimic_iv_train_cont_X, metric='manhattan')

eicu_mimic_iv_train_cont_keys_list = list(eicu_mimic_iv_train_cont_trans.keys())
eicu_mimic_iv_train_cont_X = np.array([flatten_key(key) for key in eicu_mimic_iv_train_cont_trans.keys()])
eicu_mimic_iv_train_cont_tree = KDTree(eicu_mimic_iv_train_cont_X, metric='manhattan')

##### filter transitions based on distinct stay_id number appear in this key

In [ ]:
if GROUND_TRUTH_WITH_NEIGHBORS:
    min_stay_ids = 0
else:
    min_stay_ids = 3

train_filtered_transitions = filter_transitions_by_stay_id(train_transitions, train_stay_id_mapping, min_stay_ids=min_stay_ids)
test_filtered_transitions = filter_transitions_by_stay_id(test_transitions, test_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_filtered_transitions = filter_transitions_by_stay_id(eicu_trans, eicu_stay_id_mapping, min_stay_ids=min_stay_ids)
mimic_iii_filtered_transitions = filter_transitions_by_stay_id(mimic_iii_trans, mimic_iii_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_mimic_iii_filtered_transitions = filter_transitions_by_stay_id(eicu_mimic_iii_trans, eicu_mimic_iii_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_mimic_iii_mimic_iv_train_filtered_transitions = filter_transitions_by_stay_id(eicu_mimic_iii_mimic_iv_train_trans, eicu_mimic_iii_mimic_iv_stay_id_mapping, min_stay_ids=min_stay_ids)

# common_keys = set(train_transitions.keys()) & set(eicu_filtered_transitions.keys()) & set(mimic_iii_filtered_transitions.keys()) 
common_keys = set(train_transitions.keys()) & set(eicu_filtered_transitions.keys()) # [cont_update] remove mimic-iii
not_in_mimic_iv = set(eicu_filtered_transitions.keys()) - set(train_transitions.keys()) # [cont_update] remove mimic-iii
# not_in_mimic_iv = (set(eicu_filtered_transitions.keys()) | set(mimic_iii_filtered_transitions.keys())) - set(train_transitions.keys()) # TODO: since use & only have 4 keys left, so use |
# not_in_mimic_iv = set(eicu_filtered_transitions.keys()) & set(mimic_iii_filtered_transitions.keys()) - set(train_transitions.keys()) 
external_common = list(common_keys)[:test_cases_num] 
external_not_mimic_iv = list(not_in_mimic_iv)[:test_cases_num]


# [cont_update]
train_cont_filtered_transitions = filter_transitions_by_stay_id(train_cont_transitions, train_cont_stay_id_mapping, min_stay_ids=min_stay_ids)
test_cont_filtered_transitions = filter_transitions_by_stay_id(test_cont_transitions, test_cont_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_cont_filtered_transitions = filter_transitions_by_stay_id(eicu_cont_trans, eicu_cont_stay_id_mapping, min_stay_ids=min_stay_ids)
mimic_iii_cont_filtered_transitions = filter_transitions_by_stay_id(mimic_iii_cont_trans, mimic_iii_cont_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_mimic_iii_cont_filtered_transitions = filter_transitions_by_stay_id(eicu_mimic_iii_cont_trans, eicu_mimic_iii_cont_stay_id_mapping, min_stay_ids=min_stay_ids)
eicu_mimic_iii_mimic_iv_train_cont_filtered_transitions = filter_transitions_by_stay_id(eicu_mimic_iii_mimic_iv_train_cont_trans, eicu_mimic_iii_mimic_iv_cont_stay_id_mapping, min_stay_ids=min_stay_ids)

common_cont_keys = set(train_cont_transitions.keys()) & set(eicu_cont_filtered_transitions.keys()) # [cont_update] remove mimic-iii
not_in_mimic_iv_cont = set(eicu_cont_filtered_transitions.keys()) - set(train_cont_transitions.keys()) # [cont_update] remove mimic-iii
external_common_cont = list(common_cont_keys)[:test_cases_num] 
external_not_mimic_iv_cont = list(not_in_mimic_iv_cont)[:test_cases_num]

In [ ]:
train_transitions

## Transition approximation strategies
- unseen state-action pairs
- seen state-action pairs (for avoid overfitting)

- Keep current state: next state always keep current state
- Random work: random (+1, 0, -1) on each of the state variables
- Action-Based KNN:
    - sample next_state if state_action_with_pre_state_action_baseline appear in train_transitions
    - approximate next_state by KNN with similar state_with_pre_state_action_baseline with the same action, get the delta distribution
- State-Based KNN
    - sample next_state if state_action_with_pre_state_action_baseline appear in train_transitions
    - approximate next_state by KNN with the same state_with_pre_state_action_baseline with different action, get the delta distribution
- Bayesian
- (Random Forest Regressor: since the next state should not be deterministic)
- (XGBoost Regressor)
- XGBoost Classifier
- (Random Forest Classifier)
- (SVC Classifier)
- Transformer
- RNN (GRU)

### Transition approximation strategies with n samples and save models

In [ ]:
import os
import pickle
from collections import Counter
import random
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Helper function to extract the current state from the key
def get_current_state(key):
    return key[-2][0]  # The last element's state

# Helper function to extract the baseline variables from the key
def get_baseline(key):
    return key[-1]  # The last element (age, gender)

# Helper function to flatten the key for KDTree
def flatten_key(key):
    # Flatten the key for KDTree
    flat = []
    for element in key[:-1]:  # Iterate over all elements except the last (state-action history)
        state, action = element
        flat.extend([int(x) for x in state])  # Convert state values to int
        flat.extend([int(x) for x in action])  # Convert action values to int
    flat.extend([int(x) for x in key[-1]])  # Convert baseline values to int
    return np.array(flat, dtype=np.int32)

In [ ]:
# ================================
# Keep Current State - Batch Version
# ================================
def keep_current_state_approximation(key, transitions, tree, k=10, bins=10, n_samples=1):
    """Approximate next_state using current state (batch version)."""
    current_state = get_current_state(key)
    # Return n_samples copies of the current state
    return [current_state] * n_samples

# ================================
# Random Walk - Batch Version
# ================================
def random_walk_approximation(key, transitions, tree, k=10, bins=10, n_samples=1):
    """Approximate next_state using small random changes from current state (batch version)."""
    s_t = get_current_state(key)  # Get current state
    results = []
    
    for _ in range(n_samples):
        # Generate random noise for each sample
        clipped_state = []
        for i, s in enumerate(s_t):
            variable_name = list(num_bins_dict.keys())[i]  # Get the variable name
            max_value = num_bins_dict[variable_name] - 1   # Max value is bins - 1
            clipped_value = np.clip(s + random.randint(-1, 1), 0, max_value)   # Clip to [0, max_value]
            clipped_state.append(int(clipped_value))
        results.append(tuple(clipped_state))
    
    return results

# ================================
# Action-Based KNN - Batch Version
# ================================
# TODO: analysis percentage of action-based neighbors usage
def action_based_approximation(key, transitions, tree, k=100, bins=10, n_samples=1, keys_list=train_keys_list, used_for_ground_truth_with_neighbors=False):  # TODO: shold k be larger? -> k larger means higher diversity
    """Approximate next_state using neighbors with the same action (batch version)."""
    # if key in transitions:
    #     # Use reservoir sampling if n_samples <= available transitions
    #     if n_samples <= len(transitions[key]):
    #         return random.sample(transitions[key], n_samples)
    #     else:
    #         # If more samples needed than available, sample with replacement
    #         return [random.choice(transitions[key]) for _ in range(n_samples)]
        
    # TODO: [2025/04/26] if the transition level evaluation can also be good even we don't use random.choice(transitions[key])
    # which means the transition is quite based on the action
    
    desired_action = key[-2][1]  # Extract the last action
    flat_key = flatten_key(key)
    distances, indices = tree.query(flat_key.reshape(1, -1), k=k)
    
    candidates = [keys_list[i] for i in indices[0] if keys_list[i][-2][1] == desired_action]
    if not candidates:
        # If no candidates, return current state
        current_state = get_current_state(key)
        if not used_for_ground_truth_with_neighbors:
            return [current_state] * n_samples
        else:
            return [] # [gt_with_neighbors_update] if used for ground truth with neighbors meet isolated (obs, action), return empty list
    
    # TODO: [2025/04/26] another version of candidates -> make sure it can get k candidates that match the desired action
    # i.e., for all keys with the same action, find the k nearest neighbors -> need to do experiment
    
    all_deltas = []
    for cand_key in candidates:
        s_t = get_current_state(cand_key)
        for s_next in transitions[cand_key]:
            delta = tuple(np.array(s_next, dtype=int) - np.array(s_t, dtype=int))
            all_deltas.append(delta)
    
    # If no deltas found, use (0,0,0)
    if not all_deltas:
        current_state = get_current_state(key)
        return [current_state] * n_samples
    
    # Sample deltas with replacement
    sampled_deltas = random.choices(all_deltas, k=n_samples)
    
    # Apply deltas to current state
    s_t = get_current_state(key)
    results = []
    
    for delta in sampled_deltas:
        # Apply each delta and clip values
        clipped_state = []
        for i, (s, d) in enumerate(zip(s_t, delta)):
            variable_name = list(num_bins_dict.keys())[i]
            max_value = num_bins_dict[variable_name] - 1
            clipped_value = np.clip(s + d, 0, max_value)
            clipped_state.append(int(clipped_value))
        results.append(tuple(clipped_state))
    
    return results


def action_based_approximation_cont(key, transitions, tree, k=100, bins=10, n_samples=1, keys_list=train_cont_keys_list, used_for_ground_truth_with_neighbors=False):  # TODO: shold k be larger? -> k larger means higher diversity
    """Approximate next_state using neighbors with the same action (batch version)."""
    # if key in transitions:
    #     # Use reservoir sampling if n_samples <= available transitions
    #     if n_samples <= len(transitions[key]):
    #         return random.sample(transitions[key], n_samples)
    #     else:
    #         # If more samples needed than available, sample with replacement
    #         return [random.choice(transitions[key]) for _ in range(n_samples)]
        
    # TODO: [2025/04/26] if the transition level evaluation can also be good even we don't use random.choice(transitions[key])
    # which means the transition is quite based on the action
    
    desired_action = key[-2][1]  # Extract the last action
    flat_key = flatten_key(key)
    distances, indices = tree.query(flat_key.reshape(1, -1), k=k)
    
    candidates = [keys_list[i] for i in indices[0] if keys_list[i][-2][1] == desired_action]
    # print(len(candidates), "candidates found for action:", desired_action)  # Debugging
    # print(candidates[0])  # Display first 5 candidates for debugging
    if not candidates:
        # If no candidates, return current state
        # print("Anomalous action")
        current_state = get_current_state(key)
        if not used_for_ground_truth_with_neighbors:
            return [current_state] * n_samples
        else:
            return [] # [gt_with_neighbors_update] if used for ground truth with neighbors meet isolated (obs, action), return empty list
    
    # TODO: [2025/04/26] another version of candidates -> make sure it can get k candidates that match the desired action
    # i.e., for all keys with the same action, find the k nearest neighbors -> need to do experiment
    
    all_deltas = []
    for cand_key in candidates:
        s_t = get_current_state(cand_key)
        for s_next in transitions[cand_key]:
            delta = tuple(np.array(s_next, dtype=int) - np.array(s_t, dtype=int))
            all_deltas.append(delta)
    
    # If no deltas found, use (0,0,0)
    if not all_deltas:
        current_state = get_current_state(key)
        return [current_state] * n_samples
    
    # Sample deltas with replacement
    sampled_deltas = random.choices(all_deltas, k=n_samples)
    
    # Apply deltas to current state
    s_t = get_current_state(key)
    results = []
    
    # [cont_update]
    for delta in sampled_deltas:
        # Apply each delta and clip values
        clipped_state = []
        for i, (s, d) in enumerate(zip(s_t, delta)):
            # variable_name = list(num_bins_dict.keys())[i]
            # max_value = num_bins_dict[variable_name] - 1
            # clipped_value = np.clip(s + d, 0, max_value)
            clipped_value = s + d # [cont_update] remove clipping
            clipped_state.append(int(clipped_value))
        results.append(tuple(clipped_state))
    
    return results


# ================================
# State-Based KNN - Batch Version
# ================================
def state_based_approximation(key, transitions, tree, k=10, bins=10, n_samples=1):
    """Approximate next_state using neighbors with similar state history (batch version)."""
    # if key in transitions:
    #     # Sample from available transitions
    #     if n_samples <= len(transitions[key]):
    #         return random.sample(transitions[key], n_samples)
    #     else:
    #         return [random.choice(transitions[key]) for _ in range(n_samples)]
    
    flat_key = flatten_key(key)
    distances, indices = tree.query(flat_key.reshape(1, -1), k=k)
    
    all_deltas = []
    for idx in indices[0]:
        cand_key = train_keys_list[idx]
        s_t = get_current_state(cand_key)
        for s_next in transitions[cand_key]:
            delta = tuple(np.array(s_next, dtype=int) - np.array(s_t, dtype=int))
            all_deltas.append(delta)
    
    if not all_deltas:
        current_state = get_current_state(key)
        return [current_state] * n_samples
    
    # Sample deltas with replacement
    sampled_deltas = random.choices(all_deltas, k=n_samples)
    
    # Apply deltas to current state
    s_t = get_current_state(key)
    results = []
    
    for delta in sampled_deltas:
        # Apply delta and clip values
        clipped_state = []
        for i, (s, d) in enumerate(zip(s_t, delta)):
            variable_name = list(num_bins_dict.keys())[i]
            max_value = num_bins_dict[variable_name] - 1
            clipped_value = np.clip(s + d, 0, max_value)
            clipped_state.append(int(clipped_value))
        results.append(tuple(clipped_state))
    
    return results

# ================================
# Bayesian Approximation - Batch Version
# ================================
def bayesian_approximation(key, transitions, tree, k=100, bins=10, alpha=1.0, n_samples=1): # TODO: [2025/04/29] bins should be the same as num_bins_dict (?)
    """Approximate next_state distribution using a Bayesian approach with Dirichlet prior (batch version)."""
    # if key in transitions: # TODO: [2025/04/26] should we use this? or don't use random.choice(transitions[key])
    #     # Sample from available transitions
    #     if n_samples <= len(transitions[key]):
    #         return random.sample(transitions[key], n_samples)
    #     else:
    #         return [random.choice(transitions[key]) for _ in range(n_samples)]
    
    flat_key = flatten_key(key)
    distances, indices = tree.query(flat_key.reshape(1, -1), k=k)
    
    all_next_states = []
    for idx in indices[0]:
        all_next_states.extend(transitions[train_keys_list[idx]])
    
    if not all_next_states:
        return [get_current_state(key)] * n_samples
    
    counter = Counter(all_next_states)
    
    # Calculate dirichlet prior
    total = sum(counter.values()) + alpha * sum(num_bins_dict[col] for col in state_cols)
    probs = {state: (count + alpha) / total for state, count in counter.items()}
    # TODO: [ERROR]: use num_bins_dict for mapping the bin value instead of bins
    
    # Convert to lists for random.choices
    states, weights = zip(*probs.items())
    
    # Return n_samples samples based on the calculated weights
    return random.choices(states, weights=weights, k=n_samples)


In [ ]:
# ================================
# XGBoost Classification Approximation - Batch Version
# ================================
def xgb_classification_approximation(key, transitions, tree, k=10, bins=10, retrain=False, n_samples=1):
    """XGBoost-based approximation with randomness for uncertainty (classification version, batch)."""
    # Ensure the model directory exists
    model_dir = os.path.join(mimic_iv_prefix_path, "models")
    os.makedirs(model_dir, exist_ok=True)
    
    # Check if we need to train or load XGBoost classification models
    xgb_cls_models = None
    # model_paths = [os.path.join(model_dir, f"xgb_cls_model_{col}.pkl") for col in state_cols]
    # mapping_path = os.path.join(model_dir, "xgb_cls_mappings.pkl")
    
    # [gt_with_neighbors_update] training smooth with neighbors
    model_paths = [os.path.join(model_dir, f"xgb_cls_model_{col}_ground_truth_with_neighbors.pkl") for col in state_cols]
    mapping_path = os.path.join(model_dir, "xgb_cls_mappings_ground_truth_with_neighbors.pkl")
    
    if not retrain and all(os.path.exists(path) for path in model_paths) and os.path.exists(mapping_path):
        # Load models and mappings
        xgb_cls_models = []
        for path in model_paths:
            with open(path, 'rb') as f:
                xgb_cls_models.append(pickle.load(f))
                
        with open(mapping_path, 'rb') as f:
            mappings = pickle.load(f)
            globals()['next_state_to_class'] = mappings['next_state_to_class']
            globals()['class_to_next_state'] = mappings['class_to_next_state']
            
        # print("Loaded XGBoost classification models from disk")
    else:
        # Train the models
        print("Training XGBoost classification models...")
        xgb_cls_models = train_xgb_models_classification(transitions, state_cols)
        
        # Save the models
        for i, col in enumerate(state_cols):
            with open(model_paths[i], 'wb') as f:
                pickle.dump(xgb_cls_models[i], f)
                
        # Save the mappings
        mappings = {
            'next_state_to_class': globals()['next_state_to_class'],
            'class_to_next_state': globals()['class_to_next_state']
        }
        with open(mapping_path, 'wb') as f:
            pickle.dump(mappings, f)
            
        print("Saved XGBoost classification models to disk")
    
    # Store in globals for reuse
    globals()['xgb_cls_models'] = xgb_cls_models
    
    # Generate predictions
    features = flatten_key(key).reshape(1, -1)
    results = []
    
    for _ in range(n_samples):
        # Get probability distributions for each target variable
        hr_probs = xgb_cls_models[0].predict_proba(features)[0]
        rr_probs = xgb_cls_models[1].predict_proba(features)[0]
        spo2_probs = xgb_cls_models[2].predict_proba(features)[0]
        
        # Sample from probability distributions
        hr_class = np.random.choice(len(hr_probs), p=hr_probs)
        rr_class = np.random.choice(len(rr_probs), p=rr_probs)
        spo2_class = np.random.choice(len(spo2_probs), p=spo2_probs)
        
        # Convert class indices to actual values
        hr = globals()['class_to_next_state'][hr_class][0]
        rr = globals()['class_to_next_state'][rr_class][1]
        spo2 = globals()['class_to_next_state'][spo2_class][2]
        
        # Clip values to valid ranges
        clipped_state = []
        for i, pred in enumerate([hr, rr, spo2]):
            variable_name = list(num_bins_dict.keys())[i]
            max_value = num_bins_dict[variable_name] - 1
            clipped_value = np.clip(pred, 0, max_value)
            clipped_state.append(int(clipped_value))
            
        results.append(tuple(clipped_state))
    
    return results

def train_xgb_models_classification(transitions, state_cols):
    """Train XGBoost classification models for next state prediction."""
    X, y_next_states = [], []
    
    # Extract features (X) and next states (y)
    for key, next_states in transitions.items():
        features = flatten_key(key)
        for next_state in next_states:
            X.append(features)
            # y_next_states.append(next_state)
            y_next_states.append(tuple(next_state)) # [update]
        if GROUND_TRUTH_WITH_NEIGHBORS: # [gt_with_neighbors_update] training smooth with nighbors
            true_next_states = np.array(action_based_approximation(key, transitions, train_tree, k=100, bins=bins, n_samples=1, keys_list=train_keys_list, used_for_ground_truth_with_neighbors=True)) # [gt_with_neighbors_update]
            for true_next_state in true_next_states:
                X.append(features)
                y_next_states.append(tuple(true_next_state))
    
    # Create mappings between next states and class indices
    next_state_to_class = {}
    class_to_next_state = {}
    for next_state in set(y_next_states):
        next_state_tuple = tuple(next_state)
        if next_state_tuple not in next_state_to_class:
            class_index = len(next_state_to_class)
            next_state_to_class[next_state_tuple] = class_index
            class_to_next_state[class_index] = next_state_tuple
    
    # Map next states to class indices
    y = [next_state_to_class[tuple(next_state)] for next_state in y_next_states]
    y = np.array(y)
    X = np.array(X)
    
    # Store mappings for decoding
    globals()['next_state_to_class'] = next_state_to_class
    globals()['class_to_next_state'] = class_to_next_state
    
    # Train models
    models = []
    for i, col in enumerate(state_cols):
        # model = xgb.XGBClassifier(
        #     objective='multi:softprob', 
        #     max_depth=6, 
        #     n_estimators=100, 
        #     learning_rate=0.1,
        #     n_jobs=-1
        # )
        model = xgb.XGBClassifier(
            objective='multi:softprob', 
            max_depth=3, 
            n_estimators=30, 
            learning_rate=0.1,
            n_jobs=-1,
            tree_method='hist',
            subsample=0.8,
            colsample_bytree=0.8
        )
        print(f'Fitting {col} model')
        model.fit(X, y)
        models.append(model)
    
    return models

In [ ]:
import os
import pickle
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ================================
# Random Forest Approximation - Batch Version
# ================================

def rf_approximation_classification(key, transitions, tree, k=10, bins=10, retrain=False, n_samples=1):
    """Random Forest classification-based approximation for next state prediction."""
    # Ensure the model directory exists
    model_dir = os.path.join(mimic_iv_prefix_path, "models")
    os.makedirs(model_dir, exist_ok=True)
    
    # Check if we need to train or load RF classification models
    rf_cls_models = None
    model_paths = [os.path.join(model_dir, f"rf_cls_model_{col}.pkl") for col in state_cols]
    mapping_path = os.path.join(model_dir, "rf_cls_mappings.pkl")
    
    if not retrain and all(os.path.exists(path) for path in model_paths) and os.path.exists(mapping_path):
        # Load models if they exist
        rf_cls_models = []
        for path in model_paths:
            with open(path, 'rb') as f:
                rf_cls_models.append(pickle.load(f))
                
        with open(mapping_path, 'rb') as f:
            mappings = pickle.load(f)
            globals()['next_state_to_class_rf'] = mappings['next_state_to_class']
            globals()['class_to_next_state_rf'] = mappings['class_to_next_state']
    else:
        # Train the models
        print("Training Random Forest classification models...")
        rf_cls_models = train_rf_models_classification(transitions, state_cols)
        
        # Save the models
        for i, col in enumerate(state_cols):
            with open(model_paths[i], 'wb') as f:
                pickle.dump(rf_cls_models[i], f)
                
        # Save the mappings
        mappings = {
            'next_state_to_class': globals()['next_state_to_class_rf'],
            'class_to_next_state_rf': globals()['class_to_next_state_rf']
        }
        with open(mapping_path, 'wb') as f:
            pickle.dump(mappings, f)
        print("Saved Random Forest classification models to disk")
    
    # Store in globals for reuse
    globals()['rf_cls_models'] = rf_cls_models
    
    # Generate predictions
    features = flatten_key(key).reshape(1, -1)
    results = []
    
    for _ in range(n_samples):
        # Get probability distributions for each target variable
        hr_probs = rf_cls_models[0].predict_proba(features)[0]
        rr_probs = rf_cls_models[1].predict_proba(features)[0]
        spo2_probs = rf_cls_models[2].predict_proba(features)[0]
        
        # Sample from probability distributions
        hr_class = np.random.choice(len(hr_probs), p=hr_probs)
        rr_class = np.random.choice(len(rr_probs), p=rr_probs)
        spo2_class = np.random.choice(len(spo2_probs), p=spo2_probs)
        
        # Convert class indices to actual values
        hr = globals()['class_to_next_state_rf'][hr_class][0]
        rr = globals()['class_to_next_state_rf'][rr_class][1]
        spo2 = globals()['class_to_next_state_rf'][spo2_class][2]
        
        # Clip values to valid ranges
        clipped_state = []
        for i, pred in enumerate([hr, rr, spo2]):
            variable_name = list(num_bins_dict.keys())[i]
            max_value = num_bins_dict[variable_name] - 1
            clipped_value = np.clip(pred, 0, max_value)
            clipped_state.append(int(clipped_value))
            
        results.append(tuple(clipped_state))
    
    return results

def train_rf_models_classification(transitions, state_cols):
    """Train Random Forest classification models for each state variable."""
    X, y_next_states = [], []
    
    # Extract features (X) and next states (y)
    for key, next_states in transitions.items():
        features = flatten_key(key)
        for next_state in next_states:
            X.append(features)
            y_next_states.append(next_state)
    
    # Create mappings between next states and class indices
    next_state_to_class_rf = {}
    class_to_next_state_rf = {}
    for next_state in set(y_next_states):
        next_state_tuple = tuple(next_state)
        if next_state_tuple not in next_state_to_class_rf:
            class_index = len(next_state_to_class_rf)
            next_state_to_class_rf[next_state_tuple] = class_index
            class_to_next_state_rf[class_index] = next_state_tuple
    
    # Map next states to class indices
    y = [next_state_to_class_rf[tuple(next_state)] for next_state in y_next_states]
    y = np.array(y)
    X = np.array(X)
    
    # Split data for training and validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Store mappings for decoding
    globals()['next_state_to_class_rf'] = next_state_to_class_rf
    globals()['class_to_next_state_rf'] = class_to_next_state_rf
    
    # Train models
    models = []
    for i, col in enumerate(state_cols):
        print(f'Fitting RF classifier for {col}')
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            n_jobs=-1,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Validate
        val_accuracy = model.score(X_val, y_val)
        print(f"Validation accuracy for {col}: {val_accuracy:.4f}")
        
        models.append(model)
    
    return models

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# ===============================
# Support Vector Classifier Approximation - Batch Version
# ===============================
def svm_approximation_classification(key, transitions, tree, k=10, bins=10, retrain=False, n_samples=1):
    """SVM classification-based approximation for next state prediction."""
    # Ensure the model directory exists
    model_dir = os.path.join(mimic_iv_prefix_path, "models")
    os.makedirs(model_dir, exist_ok=True)
    
    # Check if we need to train or load SVM classification models
    svm_cls_models = None
    model_paths = [os.path.join(model_dir, f"svm_cls_model_{col}.pkl") for col in state_cols]
    mapping_path = os.path.join(model_dir, "svm_cls_mappings.pkl")
    scaler_path = os.path.join(model_dir, "svm_scaler.pkl")
    
    if not retrain and all(os.path.exists(path) for path in model_paths) and os.path.exists(mapping_path) and os.path.exists(scaler_path):
        # Load models if they exist
        svm_cls_models = []
        for path in model_paths:
            with open(path, 'rb') as f:
                svm_cls_models.append(pickle.load(f))
                
        with open(mapping_path, 'rb') as f:
            mappings = pickle.load(f)
            globals()['next_state_to_class_svm'] = mappings['next_state_to_class']
            globals()['class_to_next_state_svm'] = mappings['class_to_next_state']
            
        with open(scaler_path, 'rb') as f:
            globals()['svm_scaler'] = pickle.load(f)
    else:
        # Train the models
        print("Training SVM classification models...")
        svm_cls_models = train_svm_models_classification(transitions, state_cols)
        
        # Save the models
        for i, col in enumerate(state_cols):
            with open(model_paths[i], 'wb') as f:
                pickle.dump(svm_cls_models[i], f)
                
        # Save the mappings
        mappings = {
            'next_state_to_class': globals()['next_state_to_class_svm'],
            'class_to_next_state': globals()['class_to_next_state_svm']
        }
        with open(mapping_path, 'wb') as f:
            pickle.dump(mappings, f)
            
        # Save the scaler
        with open(scaler_path, 'wb') as f:
            pickle.dump(globals()['svm_scaler'], f)
            
        print("Saved SVM classification models to disk")
    
    # Store in globals for reuse
    globals()['svm_cls_models'] = svm_cls_models
    
    # Generate predictions
    features = flatten_key(key).reshape(1, -1)
    # Scale features
    features = globals()['svm_scaler'].transform(features)
    
    results = []
    
    for _ in range(n_samples):
        # Predict classes for each state variable
        hr_class = svm_cls_models[0].predict(features)[0]
        rr_class = svm_cls_models[1].predict(features)[0]
        spo2_class = svm_cls_models[2].predict(features)[0]
        
        # Add some randomness (since SVM doesn't naturally give probabilities)
        # We'll occasionally pick a random class based on prediction certainty
        if random.random() < 0.2:  # 20% chance to pick random class
            classes = list(globals()['class_to_next_state_svm'].keys())
            hr_class = random.choice(classes)
            rr_class = random.choice(classes)
            spo2_class = random.choice(classes)
        
        # Convert class indices to actual values
        hr = globals()['class_to_next_state_svm'][hr_class][0]
        rr = globals()['class_to_next_state_svm'][rr_class][1]
        spo2 = globals()['class_to_next_state_svm'][spo2_class][2]
        
        # Clip values to valid ranges
        clipped_state = []
        for i, pred in enumerate([hr, rr, spo2]):
            variable_name = list(num_bins_dict.keys())[i]
            max_value = num_bins_dict[variable_name] - 1
            clipped_value = np.clip(pred, 0, max_value)
            clipped_state.append(int(clipped_value))
            
        results.append(tuple(clipped_state))
    
    return results

def train_svm_models_classification(transitions, state_cols):
    """Train SVM classification models for each state variable."""
    X, y_next_states = [], []
    
    # Extract features (X) and next states (y)
    for key, next_states in transitions.items():
        features = flatten_key(key)
        for next_state in next_states:
            X.append(features)
            y_next_states.append(next_state)
    
    # Create mappings between next states and class indices
    next_state_to_class_svm = {}
    class_to_next_state_svm = {}
    for next_state in set(y_next_states):
        next_state_tuple = tuple(next_state)
        if next_state_tuple not in next_state_to_class_svm:
            class_index = len(next_state_to_class_svm)
            next_state_to_class_svm[next_state_tuple] = class_index
            class_to_next_state_svm[class_index] = next_state_tuple
    
    # Map next states to class indices
    y = [next_state_to_class_svm[tuple(next_state)] for next_state in y_next_states]
    y = np.array(y)
    X = np.array(X)
    
    # Split data for training and validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    # Store mappings for decoding and scaler
    globals()['next_state_to_class_svm'] = next_state_to_class_svm
    globals()['class_to_next_state_svm'] = class_to_next_state_svm
    globals()['svm_scaler'] = scaler
    
    # Train models
    models = []
    for i, col in enumerate(state_cols):
        print(f'Fitting SVM classifier for {col}')
        # Use linear kernel for speed with large datasets
        model = SVC(
            kernel='linear',
            C=1.0,
            probability=True,  # Enable probability estimates
            random_state=42,
            class_weight='balanced'  # Handle class imbalance
        )
        model.fit(X_train, y_train)
        
        # Validate
        val_accuracy = model.score(X_val, y_val)
        print(f"Validation accuracy for {col}: {val_accuracy:.4f}")
        
        models.append(model)
    
    return models

### RNN / Transformer for approximation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class StateTransitionDataset(Dataset):
    def __init__(self, transitions):
        self.features = []
        self.targets = []
        
        # Convert transitions to PyTorch tensors
        for key, next_states in transitions.items():
            features = flatten_key(key)
            for next_state in next_states:
                self.features.append(features)
                self.targets.append(next_state)
            if GROUND_TRUTH_WITH_NEIGHBORS: # [gt_with_neighbors_update] training smooth with nighbors
                # [gt_with_neighbors_update] since n_samples will affect training data size, change from 30 to 5
                true_next_states = np.array(action_based_approximation(key, transitions, train_tree, k=100, bins=bins, n_samples=5, keys_list=train_keys_list, used_for_ground_truth_with_neighbors=True)) # [gt_with_neighbors_update]
                for true_next_state in true_next_states:
                    self.features.append(features)
                    self.targets.append(true_next_state)
        
        self.features = torch.tensor(self.features, dtype=torch.float32)
        self.targets = torch.tensor(self.targets, dtype=torch.float32)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

### Probabilistic Versions of RNN and Transformer Models

In [ ]:
# Get any key from transitions
first_key = next(iter(transitions))
# Flatten it to get the feature vector
sample_features = flatten_key(first_key)
# Get the dimension
input_dim = len(sample_features)
input_dim

In [ ]:
class ProbabilisticTransformerNextStatePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim_dict, nhead=4, num_layers=2, dropout=0.2):
        """
        output_dim_dict: Dictionary mapping state variable names to number of bins
        """
        super(ProbabilisticTransformerNextStatePredictor, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim_dict = output_dim_dict
        self.variable_names = list(output_dim_dict.keys())
        
        # Input embedding
        self.embedding = nn.Linear(input_dim, hidden_dim)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, 
            nhead=nhead, 
            dim_feedforward=hidden_dim*4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output layers - one for each state variable
        self.output_heads = nn.ModuleDict({
            var_name: nn.Linear(hidden_dim, num_bins) 
            for var_name, num_bins in output_dim_dict.items()
        })
        
    def forward(self, x):
        # Reshape input: [batch_size, seq_len=1, input_dim]
        x = x.unsqueeze(1)
        
        # Embed input
        x = self.embedding(x)
        
        # Pass through transformer encoder
        x = self.transformer_encoder(x)
        
        # Get the output
        x = x.squeeze(1)  # [batch_size, hidden_dim]
        
        # Pass through output layers to get logits for each state variable
        logits = {var_name: head(x) for var_name, head in self.output_heads.items()}
        
        return logits

In [ ]:
class ProbabilisticRNNNextStatePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim_dict, num_layers=2, dropout=0.2):
        """
        output_dim_dict: Dictionary mapping state variable names to number of bins
        """
        super(ProbabilisticRNNNextStatePredictor, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim_dict = output_dim_dict
        self.variable_names = list(output_dim_dict.keys())
        self.num_layers = num_layers
        
        # RNN layers
        self.rnn = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Output layers - one for each state variable
        self.output_heads = nn.ModuleDict({
            var_name: nn.Linear(hidden_dim, num_bins) 
            for var_name, num_bins in output_dim_dict.items()
        })
        
    def forward(self, x):
        # Reshape input: [batch_size, seq_len=1, input_dim]
        x = x.unsqueeze(1)
        
        # Pass through RNN
        rnn_out, _ = self.rnn(x)
        
        # Get the output of last time step
        rnn_out = rnn_out[:, -1, :]  # [batch_size, hidden_dim]
        
        # Pass through output layers to get logits for each state variable
        logits = {var_name: head(rnn_out) for var_name, head in self.output_heads.items()}
        
        return logits

In [ ]:
def train_probabilistic_model(model_class, transitions, state_cols, epochs=50, batch_size=64, lr=0.001, **model_kwargs):
# def train_probabilistic_model(model_class, transitions, state_cols, epochs=50, batch_size=1024, lr=0.001, **model_kwargs): # [gt_with_neighbors_update] changed batch_size from 64 to 1024
    """Train a probabilistic next state prediction model."""
    # Prepare dataset
    dataset = StateTransitionDataset(transitions)
    
    # Calculate dimensions
    input_dim = dataset.features.shape[1]
    
    # Create output_dim_dict mapping state variables to number of bins
    output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
    
    # Split into train, validation, and test sets
    dataset_size = len(dataset)
    train_size = int(0.7 * dataset_size)
    val_size = int(0.15 * dataset_size)
    test_size = dataset_size - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size], 
        generator=torch.Generator().manual_seed(42)
    )
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Initialize model
    model = model_class(
        input_dim=input_dim,
        output_dim_dict=output_dim_dict,
        **model_kwargs
    ).to(device)
    
    # Loss function (Cross Entropy for categorical distributions)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    # Training loop
    best_val_loss = float('inf')
    model_name = model_class.__name__.lower()
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for features, targets in progress_bar:
            features = features.to(device)
            targets = targets.to(device).long()  # Need long type for CrossEntropyLoss
            
            # Forward pass - get logits dictionary
            logits_dict = model(features)
            
            # Calculate loss for each state variable and sum them
            loss = 0
            for i, var_name in enumerate(model.variable_names):
                var_logits = logits_dict[var_name]
                var_targets = targets[:, i]
                loss += criterion(var_logits, var_targets)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item() / len(model.variable_names)})
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for features, targets in val_loader:
                features = features.to(device)
                targets = targets.to(device).long()
                
                logits_dict = model(features)
                
                # Calculate validation loss
                batch_loss = 0
                for i, var_name in enumerate(model.variable_names):
                    var_logits = logits_dict[var_name]
                    var_targets = targets[:, i]
                    batch_loss += criterion(var_logits, var_targets)
                
                val_loss += batch_loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model_path = os.path.join(mimic_iv_prefix_path, "models", f"probabilistic_{model_name}_best_ground_truth_with_neighbors.pth")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_loss': val_loss,
                'variable_names': model.variable_names,
                'output_dim_dict': model.output_dim_dict
            }, model_path)
            print(f"Saved best model with validation loss: {val_loss:.6f}")
    
    return model

In [ ]:
import torch.nn.functional as F
def probabilistic_next_state_prediction(model, key, n_samples=5):
    """Sample next states from the probabilistic model."""
    model.eval()
    
    # Prepare input features
    features = torch.tensor(flatten_key(key), dtype=torch.float32).to(device)
    
    next_states = []
    with torch.no_grad():
        # Get logits dictionary
        logits_dict = model(features.unsqueeze(0))
        
        # Convert logits to probabilities
        probs_dict = {
            var_name: F.softmax(logits, dim=1) 
            for var_name, logits in logits_dict.items()
        }
        
        # Sample from the distributions
        for _ in range(n_samples):
            state = []
            for var_name in model.variable_names:
                # Get probability distribution for this variable
                probs = probs_dict[var_name].squeeze(0)
                
                # Sample from categorical distribution
                dist = torch.distributions.Categorical(probs)
                sample = dist.sample()
                
                state.append(sample.item())
            
            next_states.append(tuple(state))
    
    return next_states

def predict_with_uncertainty(model, key):
    """Get next state prediction with uncertainty metrics."""
    model.eval()
    
    # Prepare input features
    features = torch.tensor(flatten_key(key), dtype=torch.float32).to(device)
    
    result = {}
    with torch.no_grad():
        # Get logits dictionary
        logits_dict = model(features.unsqueeze(0))
        
        # For each state variable
        for var_name in model.variable_names:
            logits = logits_dict[var_name].squeeze(0)
            probs = F.softmax(logits, dim=0)
            
            # Most likely outcome
            most_likely = torch.argmax(probs).item()
            
            # Entropy (uncertainty measure)
            entropy = -torch.sum(probs * torch.log(probs + 1e-10)).item()
            
            # Top 3 most likely values and their probabilities
            values, indices = torch.topk(probs, min(3, len(probs)))
            top_predictions = [(idx.item(), val.item()) for idx, val in zip(indices, values)]
            
            result[var_name] = {
                'prediction': most_likely,
                'entropy': entropy,
                'top_predictions': top_predictions
            }
    
    return result

In [ ]:
def rnn_approximation(key, transitions, tree, k=10, bins=10, n_samples=1):
    """RNN-based approximation of next state."""    
    
    # Check if the model is already loaded in the global namespace
    if 'probabilistic_rnn_model' not in globals():
        model_dir = os.path.join(mimic_iv_prefix_path, "models")
        # model_path = os.path.join(model_dir, 'probabilistic_probabilisticrnnnextstatepredictor_best.pth')
        # [gt_with_neighbors]
        model_path = os.path.join(model_dir, 'probabilistic_probabilisticrnnnextstatepredictor_best_ground_truth_with_neighbors.pth')

        try:
            # Load the checkpoint
            checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            
            # Use the saved output_dim_dict from checkpoint if available
            if 'output_dim_dict' in checkpoint:
                output_dim_dict = checkpoint['output_dim_dict']
            else:
                output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
            
            # Initialize the model
            probabilistic_rnn = ProbabilisticRNNNextStatePredictor(
                # input_dim=StateTransitionDataset(transitions).features.shape[1],
                input_dim=input_dim,
                hidden_dim=128,
                num_layers=2,
                output_dim_dict=output_dim_dict,
                dropout=0.2
            )
            
            # Load the state dict using the correct key
            probabilistic_rnn.load_state_dict(checkpoint['model_state_dict'])
            print("Successfully loaded RNN model from:", model_path)
            globals()['probabilistic_rnn_model'] = probabilistic_rnn
            
        except (FileNotFoundError, KeyError) as e:
            print(f"Error loading RNN model: {e}")
            print("Training a new probabilistic RNN model...")
            probabilistic_rnn = train_probabilistic_model(
                ProbabilisticRNNNextStatePredictor,
                train_transitions, 
                state_cols, 
                hidden_dim=128,
                num_layers=2,
                dropout=0.2
            )
            globals()['probabilistic_rnn_model'] = probabilistic_rnn
    
    # Use the model for prediction
    rnn_samples = probabilistic_next_state_prediction(globals()['probabilistic_rnn_model'], key, n_samples=n_samples)
    
    return rnn_samples

In [ ]:
def transformer_approximation(key, transitions, tree, k=10, bins=10, n_samples=1):
    """Transformer-based approximation of next state."""
    
    # Load probabilistic Transformer model
    if 'probabilistic_transformer_model' not in globals():
        model_dir = os.path.join(mimic_iv_prefix_path, "models")
        # model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best.pth')
        # [gt_with_neighbors]
        model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best_ground_truth_with_neighbors.pth')
        
        try:
            # Load the checkpoint
            checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            
            # Use the saved output_dim_dict from checkpoint if available
            if 'output_dim_dict' in checkpoint:
                output_dim_dict = checkpoint['output_dim_dict']
            else:
                output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
            
            # Initialize the model
            probabilistic_transformer = ProbabilisticTransformerNextStatePredictor(
                # input_dim=StateTransitionDataset(transitions).features.shape[1],
                input_dim=input_dim,
                hidden_dim=128,
                num_layers=2,
                output_dim_dict=output_dim_dict,
                dropout=0.2,
                nhead=4  # Add the nhead parameter that was used in training
            )
            
            # Load the state dict using the correct key
            probabilistic_transformer.load_state_dict(checkpoint['model_state_dict'])
            print("Successfully loaded Transformer model from:", model_path)
            globals()['probabilistic_transformer_model'] = probabilistic_transformer
            
        except (FileNotFoundError, KeyError) as e:
            print(f"Error loading Transformer model: {e}")
            print("Training a new probabilistic Transformer model...")
            probabilistic_transformer = train_probabilistic_model(
                ProbabilisticTransformerNextStatePredictor,
                train_transitions, 
                state_cols, 
                hidden_dim=128, 
                nhead=4,
                num_layers=2,
                dropout=0.2
            )
            globals()['probabilistic_transformer_model'] = probabilistic_transformer
    
    # Use the cached model for prediction
    transformer_samples = probabilistic_next_state_prediction(
        globals()['probabilistic_transformer_model'], 
        key, 
        n_samples=n_samples
    )
    
    return transformer_samples

In [ ]:
# Get any key from transitions
first_key = next(iter(transitions))
# Flatten it to get the feature vector
sample_features = flatten_key(first_key)
# Get the dimension
input_dim = len(sample_features)
input_dim

In [ ]:
# Load probabilistic RNN model
if 'probabilistic_rnn_model' not in globals():
    model_dir = os.path.join(mimic_iv_prefix_path, "models")
    # model_path = os.path.join(model_dir, 'probabilistic_probabilisticrnnnextstatepredictor_best.pth')
    # [gt_with_neighbors]
    model_path = os.path.join(model_dir, 'probabilistic_probabilisticrnnnextstatepredictor_best_ground_truth_with_neighbors.pth')
    
    try:
        # Load the checkpoint
        checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Use the saved output_dim_dict from checkpoint if available
        if 'output_dim_dict' in checkpoint:
            output_dim_dict = checkpoint['output_dim_dict']
        else:
            output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
        
        # Initialize the model
        probabilistic_rnn = ProbabilisticRNNNextStatePredictor(
            # input_dim=StateTransitionDataset(transitions).features.shape[1],
            input_dim=input_dim,
            hidden_dim=128,
            num_layers=2,
            output_dim_dict=output_dim_dict,
            dropout=0.2
        )
        
        # Load the state dict using the correct key
        probabilistic_rnn.load_state_dict(checkpoint['model_state_dict'])
        print("Successfully loaded RNN model from:", model_path)
        globals()['probabilistic_rnn_model'] = probabilistic_rnn
        
    except (FileNotFoundError, KeyError) as e:
        print(f"Error loading RNN model: {e}")
        print("Training a new probabilistic RNN model...")
        probabilistic_rnn = train_probabilistic_model(
            ProbabilisticRNNNextStatePredictor,
            train_transitions, 
            state_cols, 
            hidden_dim=128,
            num_layers=2,
            dropout=0.2
        )
        globals()['probabilistic_rnn_model'] = probabilistic_rnn

# Load probabilistic Transformer model
if 'probabilistic_transformer_model' not in globals():
    model_dir = os.path.join(mimic_iv_prefix_path, "models")
    # model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best.pth')
    # [gt_with_neighbors]
    model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best_ground_truth_with_neighbors.pth')
    
    try:
        # Load the checkpoint
        checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Use the saved output_dim_dict from checkpoint if available
        if 'output_dim_dict' in checkpoint:
            output_dim_dict = checkpoint['output_dim_dict']
        else:
            output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
        
        # Initialize the model
        probabilistic_transformer = ProbabilisticTransformerNextStatePredictor(
            # input_dim=StateTransitionDataset(transitions).features.shape[1],
            input_dim=input_dim,
            hidden_dim=128,
            num_layers=2,
            output_dim_dict=output_dim_dict,
            dropout=0.2,
            nhead=4  # Add the nhead parameter that was used in training
        )
        
        # Load the state dict using the correct key
        probabilistic_transformer.load_state_dict(checkpoint['model_state_dict'])
        print("Successfully loaded Transformer model from:", model_path)
        globals()['probabilistic_transformer_model'] = probabilistic_transformer
        
    except (FileNotFoundError, KeyError) as e:
        print(f"Error loading Transformer model: {e}")
        print("Training a new probabilistic Transformer model...")
        probabilistic_transformer = train_probabilistic_model(
            ProbabilisticTransformerNextStatePredictor,
            train_transitions, 
            state_cols, 
            hidden_dim=128, 
            nhead=4,
            num_layers=2,
            dropout=0.2
        )
        globals()['probabilistic_transformer_model'] = probabilistic_transformer

In [ ]:
# Example key
example_key = next(iter(train_transitions.keys()))
print(f"Example key: {example_key}")
print("True next states:", train_transitions[example_key])

# Sample multiple next states
transformer_samples = probabilistic_next_state_prediction(probabilistic_transformer, example_key, n_samples=5)
rnn_samples = probabilistic_next_state_prediction(probabilistic_rnn, example_key, n_samples=5)

print("Transformer samples:", transformer_samples)
print("RNN samples:", rnn_samples)

# Get predictions with uncertainty
transformer_predictions = predict_with_uncertainty(probabilistic_transformer, example_key)
rnn_predictions = predict_with_uncertainty(probabilistic_rnn, example_key)

# Example of accessing the results
for var_name in state_cols:
    print(f"\nVariable: {var_name}")
    print("Transformer prediction:", transformer_predictions[var_name]['prediction'], 
          "Entropy:", transformer_predictions[var_name]['entropy'])
    print("Top predictions:", transformer_predictions[var_name]['top_predictions'])
    print("RNN prediction:", rnn_predictions[var_name]['prediction'], 
          "Entropy:", rnn_predictions[var_name]['entropy'])

In [ ]:
# Example usage for train (mimic_iv_disc)
sample_key = train_keys_list[0] if train_keys_list else None
if sample_key: # TODO: the baseline showed but no used warning
    print(f"Sample Key: {sample_key}, train_transitions: {train_transitions[sample_key]}")
    print(f"Cont Sample Key: {train_cont_keys_list[0]}, train_transitions: {train_cont_transitions[train_cont_keys_list[0]]}")
    print(f"Keep current state: {keep_current_state_approximation(sample_key, train_transitions, train_tree)}")
    print(f"Random walk: {random_walk_approximation(sample_key, train_transitions, train_tree)}")
    print(f"Action-Based: {action_based_approximation(sample_key, train_transitions, train_tree)}")
    print(f"Action-Based continuous: {action_based_approximation_cont(train_cont_keys_list[0], train_cont_transitions, train_cont_tree)}") # [cont_update]
    print(f"State-Based: {state_based_approximation(sample_key, train_transitions, train_tree)}")
    print(f"Bayesian: {bayesian_approximation(sample_key, train_transitions, train_tree)}")

### Sample output of each strategies

In [ ]:
len(list(train_transitions.items()))

#### use subset of train_transitions for testing if need to retrian the models

In [ ]:
# use subset of train_transitions as train_transitions_subset
train_transitions_subset = dict(random.sample(list(train_transitions.items()), 1000))

In [ ]:
list(train_transitions_subset.keys())[0]

In [ ]:
# use subset for estimate the running time for train model
sample_key = list(train_transitions.keys())[0]
if sample_key: # TODO: the baseline showed but no used warning
    print(f"Sample Key: {sample_key}, train_transitions: {train_transitions[sample_key]}")
    print(f"Keep current state: {keep_current_state_approximation(sample_key, train_transitions, train_tree, n_samples=30)}")
    print(f"Random walk: {random_walk_approximation(sample_key, train_transitions, train_tree, n_samples=30)}")
    print(f"Action-Based: {action_based_approximation(sample_key, train_transitions, train_tree, n_samples=30)}")
    print(f"Action-Based continuous: {action_based_approximation_cont(train_cont_keys_list[0], train_cont_transitions, train_cont_tree, n_samples=30)}") # [cont_update]
    print(f"State-Based: {state_based_approximation(sample_key, train_transitions, train_tree, n_samples=30)}")
    print(f"Bayesian: {bayesian_approximation(sample_key, train_transitions, train_tree, n_samples=30)}")

### (Baseline features with lower priority)
- transition structure: next state with baseline, when sample need to consider baseline first

### Transition level Evaluation
- Internal
    - train (avoid overfitting)
    - test
        - seen key pair
        - unseen key pair
- External
    - seen key pair (common in three datasets)
    - unseen (in MIMIC-III and eICU)

In [ ]:
trainsition_approximation_strategies = {
    "Keep Current State": keep_current_state_approximation,
    "Random Walk": random_walk_approximation,
    "Bayesian": bayesian_approximation,
    "Action-Based KNN": action_based_approximation,
    "Action-Based KNN Continuous": action_based_approximation_cont, # [cont_update]
    "GRU": rnn_approximation, # "Probabilistic RNN"
    "Transformer": transformer_approximation # "Probabilistic Transformer"
}
trainsition_approximation_strategies_names = list(trainsition_approximation_strategies.keys())

##### MMD with multiple n samples

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
def MMD_evaluation(approximation_func, keys, true_transitions, train_transitions, tree, k=100, bins=bins):
    mmd_scores = []
    for key in keys:
        true_next_states = np.array(true_transitions.get(key, []))
        if len(true_next_states) == 0:
            print(f"Skipping key {key} due to no true transitions.")
            continue

        # Get all approximated next states at once (more efficient)
        # approx_next_states = np.array(approximation_func(key, train_transitions, tree, bins, n_samples=30))
        approx_next_states = np.array(approximation_func(key, train_transitions, tree, k, bins, n_samples=30)) # [DEBUG]
        
        if len(approx_next_states) == 0:
            print(f"Skipping key {key} due to no approximated transitions.")
            continue

        # Compute squared distances
        xx = cdist(true_next_states, true_next_states, 'euclidean')**2
        yy = cdist(approx_next_states, approx_next_states, 'euclidean')**2
        xy = cdist(true_next_states, approx_next_states, 'euclidean')**2

        # Median heuristic for bandwidth
        all_states = np.vstack([true_next_states, approx_next_states])
        all_distances = cdist(all_states, all_states, 'euclidean')
        sigma = np.median(all_distances[all_distances > 0])

        # Gaussian kernel
        k_xx = np.exp(-xx / (2 * sigma**2))
        k_yy = np.exp(-yy / (2 * sigma**2))
        k_xy = np.exp(-xy / (2 * sigma**2))

        # Unbiased MMD^2
        m, n = len(true_next_states), len(approx_next_states)
        if m > 1:
            term1 = (np.sum(k_xx) - np.trace(k_xx)) / (m * (m - 1))
        else:
            term1 = 0  # or use biased estimator
        term2 = (np.sum(k_yy) - np.trace(k_yy)) / (n * (n - 1))
        term3 = np.sum(k_xy) / (m * n)
        mmd = np.sqrt(max(0, term1 + term2 - 2 * term3))
        mmd_scores.append(mmd)

    if not mmd_scores:
        print(f"No valid MMD scores computed for keys: {keys}")
        return float('inf')
    return np.mean(mmd_scores)

In [ ]:
# [cont_update]
import numpy as np
from scipy.spatial.distance import cdist
def MMD_evaluation_ground_truth_with_neighbors(approximation_func, keys, true_transitions, train_transitions, tree, ground_truth_tree, k=100, bins=bins, true_keys_list=train_keys_list, is_cont=False):
    mmd_scores = []
    for key in keys:
        # true_next_states = np.array(true_transitions.get(key, []))
        if not is_cont:
            true_next_states = np.array(action_based_approximation(key, true_transitions, ground_truth_tree, k, bins, n_samples=30, keys_list=true_keys_list, used_for_ground_truth_with_neighbors=True)) # [gt_with_neighbors_update]
        else:
            true_next_states = np.array(action_based_approximation_cont(key, true_transitions, ground_truth_tree, k, bins, n_samples=30, keys_list=true_keys_list, used_for_ground_truth_with_neighbors=True)) # [gt_with_neighbors_update]
        if len(true_next_states) == 0:
            print(f"Skipping key {key} due to no true transitions.")
            continue

        # Get all approximated next states at once (more efficient)
        approx_next_states = np.array(approximation_func(key, train_transitions, tree, k, bins, n_samples=30)) # [cont_update] DEBUG, since when approximation it will default use disc train_keys_list, need to use cont when input cont
        # approx_next_states = np.array(approximation_func(key, train_transitions, tree, k, bins, n_samples=30, key_list=train_keys_list)) # [cont_update] DEBUG, since when approximation it will default use disc train_keys_list, need to use cont when input cont
        
        if len(approx_next_states) == 0:
            print(f"Skipping key {key} due to no approximated transitions.")
            continue

        # Compute squared distances
        xx = cdist(true_next_states, true_next_states, 'euclidean')**2
        yy = cdist(approx_next_states, approx_next_states, 'euclidean')**2
        xy = cdist(true_next_states, approx_next_states, 'euclidean')**2

        # Median heuristic for bandwidth
        all_states = np.vstack([true_next_states, approx_next_states])
        all_distances = cdist(all_states, all_states, 'euclidean')
        sigma = np.median(all_distances[all_distances > 0])

        # Gaussian kernel
        k_xx = np.exp(-xx / (2 * sigma**2))
        k_yy = np.exp(-yy / (2 * sigma**2))
        k_xy = np.exp(-xy / (2 * sigma**2))

        # Unbiased MMD^2
        m, n = len(true_next_states), len(approx_next_states)
        if m > 1:
            term1 = (np.sum(k_xx) - np.trace(k_xx)) / (m * (m - 1))
        else:
            term1 = 0  # or use biased estimator
        term2 = (np.sum(k_yy) - np.trace(k_yy)) / (n * (n - 1))
        term3 = np.sum(k_xy) / (m * n)
        mmd = np.sqrt(max(0, term1 + term2 - 2 * term3))
        mmd_scores.append(mmd)

    if not mmd_scores:
        print(f"No valid MMD scores computed for keys: {keys}")
        return float('inf')
    return np.mean(mmd_scores)

#### Internal & External Evaluation

##### filter transitions based on the number of next state

In [ ]:
# subgroup of train_transitions: select those next state occurrance > 1
def filter_transitions(transitions, min_occurrences=2):
    filtered_transitions = {}
    for key, next_states in transitions.items():
        if len(next_states) >= min_occurrences:
            filtered_transitions[key] = next_states
    return filtered_transitions

In [ ]:
# [cont_update]
# train_filtered_transitions = filter_transitions(train_transitions, 5)
# test_filtered_transitions = filter_transitions(test_transitions, 5)
# train_filtered_transitions = filter_transitions(train_transitions, 0) # [WARNING] def filter_transitions vs. def filter_transitions_by_stay_id 
# test_filtered_transitions = filter_transitions(test_transitions, 0)

print(f"train_transitions: {len(train_transitions.keys())}")
print(f"test_transitions: {len(test_transitions.keys())}")
print(f"train_filtered_transitions: {len(train_filtered_transitions.keys())}")
print(f"test_filtered_transitions: {len(test_filtered_transitions.keys())}")

##### filter transitions based on distinct stay_id number appear in this key

In [ ]:
print(f"train_transitions: {len(train_transitions.keys())}")
print(f"test_transitions: {len(test_transitions.keys())}")
print(f"train_filtered_transitions: {len(train_filtered_transitions.keys())}")
print(f"test_filtered_transitions: {len(test_filtered_transitions.keys())}")

- filter_transitions_by_stay_id with min_stay_ids 3
- sample 1000 keys, but test_unseen_keys only have 100

#### Transition level evaluation MMD combine discrete with continuous, all use ground truth with neighbors

In [ ]:
def combine_disc_cont_MMD(methods, scenario_keys, scenario_transitions, scenario_tree, scenario_keys_list,
                    train_transitions, train_tree, bins, k,
                    cont_keys=None, cont_transitions=None, cont_tree=None, cont_keys_list=None,
                    train_cont_transitions=None, train_cont_tree=None, train_cont_keys_list=None):
    results = {}
    for name, func in methods.items():
        if name == "Action-Based KNN Continuous":
            # Use continuous data
            keys = cont_keys if cont_keys is not None else scenario_keys
            transitions = cont_transitions if cont_transitions is not None else scenario_transitions
            tree = cont_tree if cont_tree is not None else scenario_tree
            keys_list = cont_keys_list if cont_keys_list is not None else scenario_keys_list
            train_trans = train_cont_transitions if train_cont_transitions is not None else train_transitions
            train_tr = train_cont_tree if train_cont_tree is not None else train_tree
            train_keys_l = train_cont_keys_list if train_cont_keys_list is not None else train_keys_list
            is_cont = True
        else:
            # Use discrete data
            keys = scenario_keys
            transitions = scenario_transitions
            tree = scenario_tree
            keys_list = scenario_keys_list
            train_trans = train_transitions
            train_tr = train_tree
            train_keys_l = train_keys_list
            is_cont = False
        results[name] = MMD_evaluation_ground_truth_with_neighbors(
            func, keys, transitions, train_trans, train_tr, tree, k, bins, keys_list, is_cont
        )
    return results

In [ ]:
# [cont_update]
k = 100
# Evaluate on training data
# Sample training keys for training evaluation
train_keys = random.sample(list(train_filtered_transitions.keys()), min(test_cases_num, len(train_filtered_transitions)))
train_cont_keys = random.sample(list(train_cont_filtered_transitions.keys()), min(test_cases_num, len(train_cont_filtered_transitions)))

# Separate test keys into seen and unseen with exactly test_cases_num keys each
seen_keys = [key for key in test_filtered_transitions.keys() if key in train_transitions]
unseen_keys = [key for key in test_filtered_transitions.keys() if key not in train_transitions]
# Sample test_cases_num keys from seen and unseen keys
test_seen_keys = random.sample(seen_keys, min(test_cases_num, len(seen_keys)))
test_unseen_keys = random.sample(unseen_keys, min(test_cases_num, len(unseen_keys)))

# Separate test keys into seen and unseen with exactly test_cases_num keys each
cont_seen_keys = [key for key in test_cont_filtered_transitions.keys() if key in train_cont_transitions]
cont_unseen_keys = [key for key in test_cont_filtered_transitions.keys() if key not in train_cont_transitions]
test_cont_seen_keys = random.sample(cont_seen_keys, min(test_cases_num, len(cont_seen_keys)))
test_cont_unseen_keys = random.sample(cont_unseen_keys, min(test_cases_num, len(cont_unseen_keys)))

# --- Training ---
train_results = combine_disc_cont_MMD(
    trainsition_approximation_strategies,
    train_keys, train_transitions, train_tree, train_keys_list,
    train_transitions, train_tree, bins, k,
    cont_keys=train_cont_keys, cont_transitions=train_cont_transitions, cont_tree=train_cont_tree, cont_keys_list=train_cont_keys_list,
    train_cont_transitions=train_cont_transitions, train_cont_tree=train_cont_tree, train_cont_keys_list=train_cont_keys_list
)
print("train done")

# --- Test Seen ---
test_seen_results = combine_disc_cont_MMD(
    trainsition_approximation_strategies,
    test_seen_keys, test_transitions, test_tree, test_keys_list,
    train_transitions, train_tree, bins, k,
    cont_keys=test_cont_seen_keys, cont_transitions=test_cont_transitions, cont_tree=test_cont_tree, cont_keys_list=test_cont_keys_list,
    train_cont_transitions=train_cont_transitions, train_cont_tree=train_cont_tree, train_cont_keys_list=train_cont_keys_list
)
print("test seen done")

# --- Test Unseen ---
test_unseen_results = combine_disc_cont_MMD(
    trainsition_approximation_strategies,
    test_unseen_keys, test_transitions, test_tree, test_keys_list,
    train_transitions, train_tree, bins, k,
    cont_keys=test_cont_unseen_keys, cont_transitions=test_cont_transitions, cont_tree=test_cont_tree, cont_keys_list=test_cont_keys_list,
    train_cont_transitions=train_cont_transitions, train_cont_tree=train_cont_tree, train_cont_keys_list=train_cont_keys_list
)
print("test unseen done")

# --- External Common ---
external_common_results = combine_disc_cont_MMD(
    trainsition_approximation_strategies,
    external_common, eicu_trans, eicu_tree, eicu_keys_list,
    train_transitions, train_tree, bins, k,
    cont_keys=external_common_cont, cont_transitions=eicu_cont_trans, cont_tree=eicu_cont_tree, cont_keys_list=eicu_cont_keys_list,
    train_cont_transitions=train_cont_transitions, train_cont_tree=train_cont_tree, train_cont_keys_list=train_cont_keys_list
)
print("external common done")

# --- External Unique ---
external_unique_results = combine_disc_cont_MMD(
    trainsition_approximation_strategies,
    external_not_mimic_iv, eicu_trans, eicu_tree, eicu_keys_list,
    train_transitions, train_tree, bins, k,
    cont_keys=external_not_mimic_iv_cont, cont_transitions=eicu_cont_trans, cont_tree=eicu_cont_tree, cont_keys_list=eicu_cont_keys_list,
    train_cont_transitions=train_cont_transitions, train_cont_tree=train_cont_tree, train_cont_keys_list=train_cont_keys_list
)
print("external unique done")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(16, 8))
x = np.arange(len(trainsition_approximation_strategies))
width = 0.15

plt.bar(x - 2*width, [train_results[name] for name in trainsition_approximation_strategies], width, label='Internal (Training)')
plt.bar(x - width, [test_seen_results[name] for name in trainsition_approximation_strategies], width, label='Internal (Seen Keys)')
plt.bar(x, [test_unseen_results[name] for name in trainsition_approximation_strategies], width, label='Internal (Unseen Keys)')
plt.bar(x + width, [external_common_results[name] for name in trainsition_approximation_strategies], width, label='External (Seen Keys)')
plt.bar(x + 2*width, [external_unique_results[name] for name in trainsition_approximation_strategies], width, label='External (Unseen Keys)')

plt.title('Method Performance Across Different Evaluation Scenarios')
plt.xticks(x, trainsition_approximation_strategies.keys(), rotation=45, ha='right')
plt.ylabel('MMD Score (Lower is Better)')
plt.legend()
plt.tight_layout()
plt.show()

def print_results(title, results):
    print(f"\n{title}:")
    for method, score in results.items():
        print(f" {method}: {score:.4f}")

print_results("Internal (Training)", train_results)
print_results("Internal (Seen Keys)", test_seen_results)
print_results("Internal (Unseen Keys)", test_unseen_results)
print_results("External (Seen Keys)", external_common_results)
print_results("External (Unseen Keys)", external_unique_results)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# List of scenarios
scenarios = [
    "Internal (Training)",
    "Internal (Seen Keys)",
    "Internal (Unseen Keys)",
    "External (Seen Keys)",
    "External (Unseen Keys)"
]

# List of methods (transition approximation strategies)
methods = list(trainsition_approximation_strategies.keys())

# Gather the data for each scenario, in the order of methods
data = [
    [train_results[m] for m in methods],
    [test_seen_results[m] for m in methods],
    [test_unseen_results[m] for m in methods],
    [external_common_results[m] for m in methods],
    [external_unique_results[m] for m in methods],
]

data = np.array(data)  # shape: (num_scenarios, num_methods)

# Plotting
plt.figure(figsize=(16, 8))
x = np.arange(len(scenarios))
width = 0.1

# Plot each method as a bar in each scenario group
for i, method in enumerate(methods):
    plt.bar(x + (i - len(methods)/2)*width + width/2, data[:, i], width, label=method)

plt.title('Method Performance Across Different Evaluation Scenarios')
plt.xticks(x, scenarios, rotation=30, ha='right')
plt.ylabel('MMD Score (Lower is Better)')
plt.legend(title="Transition Approximation Strategy")
plt.tight_layout()
plt.show()

In [ ]:
# Average next_state number for train seen keys
train_keys_avg_next_state = np.mean([len(train_transitions[key]) for key in train_keys])
print(f"Average number of next states for train keys: {train_keys_avg_next_state:.2f}, train keys: {len(train_keys)}")
# Show average next_state number for train seen keys
test_seen_keys_avg_next_state = np.mean([len(test_transitions[key]) for key in test_seen_keys])
print(f"Average number of next states for test seen keys: {test_seen_keys_avg_next_state:.2f}, test seen keys: {len(test_seen_keys)}")
# show test_unseen_keys average next_state number
test_unseen_keys_avg_next_state = np.mean([len(test_transitions[key]) for key in test_unseen_keys])
print(f"Average number of next states for test unseen keys: {test_unseen_keys_avg_next_state:.2f}, test unseen keys: {len(test_unseen_keys)}")
# show external_common average next_state number
external_common_keys_avg_next_state = np.mean([len(eicu_mimic_iii_mimic_iv_train_trans[key]) for key in external_common])
print(f"Average number of next states for external common keys: {external_common_keys_avg_next_state:.2f}, external common keys: {len(external_common)}")
# show external_unique average next_state number
external_unique_key_avg_next_state = np.mean([len(eicu_mimic_iii_trans[key]) for key in external_not_mimic_iv])
print(f"Average number of next states for external unique keys: {external_unique_key_avg_next_state:.2f}, external unique keys: {len(external_not_mimic_iv)}")

#### example transitions for each evaluation category

In [ ]:
# Print example transitions for each category
# for i in range(1):
#     print(f"train:\nkey: {train_keys[i]}\nvalue: {train_transitions[train_keys[i]]}")
#     print(f"test_seen:\nkey: {test_seen_keys[i]}\nvalue: {test_transitions[test_seen_keys[i]]}")
#     print(f"test_unseen:\nkey: {test_unseen_keys[i]}\nvalue: {test_transitions[test_unseen_keys[i]]}")
#     print(f"external_common:\nkey: {external_common[i]}\nvalue: {eicu_mimic_iii_mimic_iv_train_trans[external_common[i]]}")
#     print(f"external_unique:\nkey: {external_not_mimic_iv[i]}\nvalue: {eicu_mimic_iii_trans[external_not_mimic_iv[i]]}")

# use action_based_approximation to generate next state for each of the above keys
# def sample_multi_run(approximation_func, keys, transitions, tree, bins=bins):
#     approx_next_states = np.array([approximation_func(keys, transitions, tree, bins) for _ in range(10)])
#     # turn it into a list of tuples
#     approx_next_states = [tuple(state) for state in approx_next_states]
#     return approx_next_states

# for i in range(1):
#     print(f"train:\nkey: {train_keys[i]}\nvalue: {train_transitions[train_keys[i]]}")
#     print(f"train_approx:\nvalue: {sample_multi_run(action_based_approximation, train_keys[i], train_transitions, train_tree)}")
#     print(f"test_seen:\nkey: {test_seen_keys[i]}\nvalue: {test_transitions[test_seen_keys[i]]}")
#     print(f"test_seen_approx:\nvalue: {sample_multi_run(action_based_approximation, test_seen_keys[i], test_transitions, train_tree)}")
#     print(f"test_unseen:\nkey: {test_unseen_keys[i]}\nvalue: {test_transitions[test_unseen_keys[i]]}")
#     print(f"test_unseen_approx:\nvalue: {sample_multi_run(action_based_approximation, test_unseen_keys[i], train_transitions, train_tree)}")
#     # print(f"external_common:\nkey: {external_common[i]}\nvalue: {eicu_mimic_iii_mimic_iv_train_trans[external_common[i]]}")
#     # print(f"external_common_approx:\nvalue: {action_based_approximation(eicu_mimic_iii_mimic_iv_train_trans[external_common[i]], eicu_mimic_iii_mimic_iv_train_trans, train_tree)}")
#     # print(f"external_unique:\nkey: {external_not_mimic_iv[i]}\nvalue: {eicu_mimic_iii_trans[external_not_mimic_iv[i]]}")
#     # print(f"external_unique_approx:\nvalue: {action_based_approximation(eicu_mimic_iii_trans[external_not_mimic_iv[i]], eicu_mimic_iii_trans, train_tree)}")



#### Different K for action based approximation

In [ ]:
# Evaluate on training data
# Sample training keys for training evaluation
train_keys = random.sample(list(train_filtered_transitions.keys()), min(test_cases_num, len(train_filtered_transitions)))

# Separate test keys into seen and unseen with exactly test_cases_num keys each
seen_keys = [key for key in test_filtered_transitions.keys() if key in train_transitions]
unseen_keys = [key for key in test_filtered_transitions.keys() if key not in train_transitions]
# Sample test_cases_num keys from seen and unseen keys
test_seen_keys = random.sample(seen_keys, min(test_cases_num, len(seen_keys)))
test_unseen_keys = random.sample(unseen_keys, min(test_cases_num, len(unseen_keys)))
# test_seen_keys = seen_keys[:min(test_cases_num, len(seen_keys))] # random.sample(seen_keys, min(test_cases_num, len(seen_keys)))
# test_unseen_keys = unseen_keys[:min(test_cases_num, len(unseen_keys))] # random.sample(unseen_keys, min(test_cases_num, len(unseen_keys)))

In [ ]:
# [cont_update] [ground_truth_with_neighbors] 
import numpy as np
import matplotlib.pyplot as plt

def visualize_mmd_action_based_approximation_ground_truth_with_neighbors(k_values, data_sets, train_transitions, tree, bins):
    """
    Visualize MMD scores for the 'action based approximation' method with different k values across multiple datasets.
    
    Parameters:
    - k_values (list): List of k values to evaluate (e.g., [3, 10, 30, 50, 100]).
    - data_sets (list): List of tuples, each containing (label, keys, true_transitions).
    - train_transitions (dict): Dictionary of training transitions for approximation.
    - tree (object): KD-tree object for nearest neighbor search.
    - bins (int): Number of bins used for discretization.
    
    Returns:
    - None: Displays a single plot of MMD scores vs. k values for all datasets.
    """
    # Function to create an approximation function with a fixed k
    def make_action_based_approx(k):
        def approx_func(key, transitions, tree, k, bins, n_samples):
            return action_based_approximation(key, transitions, tree, k=k, bins=bins, n_samples=n_samples)
        return approx_func
    
    # Create a single figure for all datasets
    plt.figure(figsize=(10, 8))
    
    # Iterate over each dataset
    for label, keys, true_transitions, true_tree, true_keys_list in data_sets:
        mmd_scores = []
        for k in k_values:
            # Create approximation function with current k
            approx_func = make_action_based_approx(k)
            # Compute MMD score
            # mmd = MMD_evaluation(approx_func, keys, true_transitions, train_transitions, tree, bins)
            mmd = MMD_evaluation_ground_truth_with_neighbors(approx_func, keys, true_transitions, train_transitions, tree, true_tree, k, bins, true_keys_list)
            mmd_scores.append(mmd)
            print(f"MMD for {label}, k={k}: {mmd:.4f}")
        
        # Plot the MMD scores for this dataset
        plt.plot(k_values, mmd_scores, marker='o', linestyle='-', label=label)
    
    # Customize the plot
    plt.xlabel('k (Number of Neighbors)')
    plt.ylabel('MMD Score')
    plt.title('MMD vs k for Action Based Approximation Across Datasets')
    plt.grid(True)
    plt.xticks(k_values)
    plt.legend()
    plt.show()

# Define the datasets to visualize together
data_sets = [
    ("Internal (Training)", train_keys, train_transitions, train_tree, train_keys_list),
    ("Internal (Seen Keys)", test_seen_keys, test_transitions, test_tree, test_keys_list),
    ("Internal (Unseen Keys)", test_unseen_keys, test_transitions, test_tree, test_keys_list),
    ("External (Seen Keys)", external_common, eicu_trans, eicu_tree, eicu_keys_list),
    ("External (Unseen Keys)", external_not_mimic_iv, eicu_trans, eicu_tree, eicu_keys_list)
]

# Example usage
k_values = [3, 10, 30, 50, 100, 150, 200]
visualize_mmd_action_based_approximation_ground_truth_with_neighbors(
    k_values=k_values,
    data_sets=data_sets,
    train_transitions=train_transitions,
    tree=train_tree,
    bins=bins
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_mmd_action_based_approximation(k_values, data_sets, train_transitions, tree, bins):
    """
    Visualize MMD scores for the 'action based approximation' method with different k values across multiple datasets.
    
    Parameters:
    - k_values (list): List of k values to evaluate (e.g., [3, 10, 30, 50, 100]).
    - data_sets (list): List of tuples, each containing (label, keys, true_transitions).
    - train_transitions (dict): Dictionary of training transitions for approximation.
    - tree (object): KD-tree object for nearest neighbor search.
    - bins (int): Number of bins used for discretization.
    
    Returns:
    - None: Displays a single plot of MMD scores vs. k values for all datasets.
    """
    # Function to create an approximation function with a fixed k
    def make_action_based_approx(k):
        def approx_func(key, transitions, tree, k, bins, n_samples):
            return action_based_approximation(key, transitions, tree, k=k, bins=bins, n_samples=n_samples)
        return approx_func
    
    # Create a single figure for all datasets
    plt.figure(figsize=(10, 8))
    
    # Iterate over each dataset
    for label, keys, true_transitions in data_sets:
        mmd_scores = []
        for k in k_values:
            # Create approximation function with current k
            approx_func = make_action_based_approx(k)
            # Compute MMD score
            mmd = MMD_evaluation(approx_func, keys, true_transitions, train_transitions, tree, k, bins)
            mmd_scores.append(mmd)
            print(f"MMD for {label}, k={k}: {mmd:.4f}")
        
        # Plot the MMD scores for this dataset
        plt.plot(k_values, mmd_scores, marker='o', linestyle='-', label=label)
    
    # Customize the plot
    plt.xlabel('k (Number of Neighbors)')
    plt.ylabel('MMD Score')
    plt.title('MMD vs k for Action Based Approximation Across Datasets')
    plt.grid(True)
    plt.xticks(k_values)
    plt.legend()
    plt.show()

# Define the datasets to visualize together
# data_sets = [
#     ("Internal (Training)", train_keys, train_transitions),
#     ("Internal (Seen Keys)", test_seen_keys, test_transitions),
#     ("Internal (Unseen Keys)", test_unseen_keys, test_transitions),
#     ("External (Seen Keys)", external_common, eicu_mimic_iii_mimic_iv_train_trans),
#     ("External (Unseen Keys)", external_not_mimic_iv, eicu_mimic_iii_trans)
# ]
data_sets = [
    ("Internal (Training)", train_keys, train_transitions),
    ("Internal (Seen Keys)", test_seen_keys, test_transitions),
    ("Internal (Unseen Keys)", test_unseen_keys, test_transitions),
    ("External (Seen Keys)", external_common, eicu_trans), # [cont_update] not sure if use eicu_trans will cause skip due to key not found in true keys list
    ("External (Unseen Keys)", external_not_mimic_iv, eicu_trans)
]

# Example usage
k_values = [3, 10, 30, 50, 100, 150, 200]
visualize_mmd_action_based_approximation(
    k_values=k_values,
    data_sets=data_sets,
    train_transitions=train_transitions,
    tree=train_tree,
    bins=bins
)

## Guideline to Bin for Python Gym environment Reward
- rewards based on ventilation weaning (extubation) guidelines

In [ ]:
import json

with open('../data/mimic_iv/min_max_values_guidelines.json', 'r') as f:
    min_max_values_guidelines = json.load(f)
# Convert to DataFrame
min_max_values_guidelines_raw_df = pd.DataFrame(min_max_values_guidelines)
# Display the DataFrame
min_max_values_guidelines_raw_df

In [ ]:
def convert_guideline_to_bins(min_max_df, bin_edges_dict, state_cols, action_cols):
    """
    Convert guideline min and max values into bin values based on bin edges for state and action variables.

    Args:
        min_max_df (pd.DataFrame): Input DataFrame with raw guideline min and max values.
        bin_edges_dict (dict): Dictionary containing bin edges for state and action variables.
        state_cols (list): List of state variable columns.
        action_cols (list): List of action variable columns.

    Returns:
        pd.DataFrame: DataFrame with guideline min and max values converted to bin values.
    """
    # Create a copy of the input DataFrame
    bin_df = min_max_df.copy()

    # Process state variables
    for col in state_cols:
        if col in bin_edges_dict:
            bin_edges = bin_edges_dict[col]
            # Convert min and max values to bins
            bin_df[col] = bin_df[col].apply(
                lambda x: pd.cut(
                    [x], bins=bin_edges, labels=False, include_lowest=True
                )[0]
            )

    # Process action variables
    for col in action_cols:
        if col in bin_edges_dict:
            bin_edges = bin_edges_dict[col]
            # Convert min and max values to bins
            bin_df[col] = bin_df[col].apply(
                lambda x: pd.cut(
                    [x], bins=bin_edges, labels=False, include_lowest=True
                )[0]
            )

    return bin_df

In [ ]:
# Convert guideline values into bins
min_max_values_guidelines_bin_s1_df = convert_guideline_to_bins(min_max_values_guidelines_raw_df, bin_edges_dict_s1, state_cols, action_cols)
min_max_values_guidelines_bin_s2_df = convert_guideline_to_bins(min_max_values_guidelines_raw_df, bin_edges_dict_s2, state_cols, action_cols)

print("Guideline Bin Values (Strategy 1):")
min_max_values_guidelines_bin_s1_df[state_cols+action_cols]


In [ ]:
print("Guideline Bin Values (Strategy 2):")
min_max_values_guidelines_bin_s2_df[state_cols+action_cols]

In [ ]:
def convert_min_max_df_to_dict(min_max_df):
    """
    Convert a DataFrame with 'min' and 'max' rows to a JSON-serializable dictionary.
    
    Parameters:
    -----------
    min_max_df : pandas.DataFrame
        DataFrame with 'min' and 'max' rows and columns for each variable
        
    Returns:
    --------
    str
        JSON string representation of the min/max values
    """
    # Convert the dataframe to a dictionary format
    result = {}
    for column in min_max_df.columns:
        result[column] = {
            'min': int(min_max_df.loc['min', column]),  # Convert to Python int
            'max': int(min_max_df.loc['max', column])   # Convert to Python int
        }
    
    return result

In [ ]:
print(min_max_values_guidelines_bin_s1_df[state_cols+action_cols])

In [ ]:
print(min_max_values_guidelines_bin_s2_df[state_cols+action_cols])

In [ ]:
# Example usage:
min_max_values_guidelines_bin_s1_dict = convert_min_max_df_to_dict(min_max_values_guidelines_bin_s1_df)
min_max_values_guidelines_bin_s2_dict = convert_min_max_df_to_dict(min_max_values_guidelines_bin_s2_df)

In [ ]:
if bins_strategy == "s1":
    bin_edges_dict = bin_edges_dict_s1
    min_max_values_guidelines_bin_dict = min_max_values_guidelines_bin_s1_dict # TODO: need to set it as selected one
elif bins_strategy == "s2":
    bin_edges_dict = bin_edges_dict_s2
    min_max_values_guidelines_bin_dict = min_max_values_guidelines_bin_s2_dict # TODO: need to set it as selected one
min_max_values_guidelines_bin_dict

In [ ]:
bin_edges_dict

## Benchmark of different severity init obs for different offline policy doctor agents
(Finding Special Initial Observations for PatientEnvironment Evaluation)

In [ ]:
# For PatientEnvironment Evaluation Reward
EVAL_ACTION_PENALTY_FLAG = False         # TODO: For Reward Design Experiment
EVAL_ACTION_STABLE_PENALTY_FLAG = True   # Penalize if the action is the same as the previous one
EVAL_WEANING_REWARD_FLAG = True          # TODO: For Reward Design Experiment
EVAL_WEANING_REWARD = 10                 # Reward for weaning the patient off ventilation
EVAL_MAX_TRAJ_LENGTH = 120               # [TODO] Maximum trajectory length for the environment, depends on the dataset distribution
EVAL_ANALYSIS_REWARD_CLIP_MAX_TRAJ_LENGTH_FLAG = True  # Clip the reward to the maximum trajectory length for analysis

ACTION_PENALTY_FLAG = False         # TODO: For Reward Design Experiment # [update] all exp don't use action penalty
ACTION_STABLE_PENALTY_FLAG = True   # Penalize if the action is the same as the previous one
STATE_REWARD_FLAG = True         # TODO: For Reward Design Experiment
STATE_STABLE_PENALTY_FLAG = True   # Penalize if the action is the same as the previous one
WEANING_REWARD_FLAG = True          # TODO: For Reward Design Experiment
TIMESTAMP_PENALTY_FLAG = True  # Penalize if the timestamp is not the same as the previous one
WEANING_REWARD = 10                 # Reward for weaning the patient off ventilation
MAX_TRAJ_LENGTH = 120               # [TODO] Maximum trajectory length for the environment, depends on the dataset distribution
ANALYSIS_REWARD_CLIP_MAX_TRAJ_LENGTH_FLAG = True  # Clip the reward to the maximum trajectory length for analysis
ALL_USE_UNSEEN_FLAG = True
EXP_FOLDER_PREFIX = "preprocess_aggregate_update_all_use_unseen_without_action_penalty_with_action_stable_penalty_weaning_reward_10_lr_1e-5" # "without_internal_reward_weaning_reward_10_lr_1e-5" # "without_action_penalty_with_action_stable_penalty_weaning_reward_10_lr_1e-5" # "with_internal_reward_without_weaning_reward_lr_1e-5" # "without_action_penalty_without_action_stable_penalty_weaning_reward_10_lr_1e-5"
# "preprocess_aggregate_update_all_use_unseen_without_action_penalty_with_action_stable_penalty_weaning_reward_10_lr_1e-5"
# "preprocess_aggregate_update_all_use_unseen_without_internal_reward_weaning_reward_10_lr_1e-5"
# "preprocess_aggregate_update_all_use_unseen_with_internal_reward_without_weaning_reward_lr_1e-5"
# mkdir if not exists
import os
if not os.path.exists(f"../models/training_log/{EXP_FOLDER_PREFIX}"):
    os.makedirs(f"../models/training_log/{EXP_FOLDER_PREFIX}")

### Extracting Initial Observations

In [ ]:
import numpy as np
import pandas as pd

def get_initial_obs(df, stay_id, state_cols, action_cols, baseline_cols, obs_hrs=3):
    group = df[df['stay_id'] == stay_id].sort_values('hours_in')
    if len(group) < obs_hrs:
        return None
    rows = group.iloc[:obs_hrs].to_dict('records')
    # History: obs_hrs - 1 state-action pairs
    history = [(tuple(rows[i][col] for col in state_cols), 
                tuple(rows[i][col] for col in action_cols)) 
               for i in range(obs_hrs - 1)]
    # Current state at t = obs_hrs - 1
    current_state = tuple(rows[obs_hrs - 1][col] for col in state_cols)
    # Baseline (constant across rows)
    baseline = tuple(rows[0][col] for col in baseline_cols)
    # Flatten into a single array
    flat_history = [item for sublist in history for item in (sublist[0] + sublist[1])]
    obs = np.array(flat_history + list(current_state) + list(baseline), dtype=np.int32)
    return obs

# Example usage
stay_id = 30003598
init_obs = get_initial_obs(train_df, stay_id, state_cols, action_cols, baseline_cols)
print(init_obs)

### Categorizing Special Initial Observations

In [ ]:
def compute_trajectory_metrics(df, state_cols, action_cols, baseline_cols, 
                               min_max_values_guidelines_bin_dict,
                               obs_hrs=3, stable_hr=2, stable_threshold=2, 
                               timestamp_reward_step=1):
    """
    Compute trajectory metrics per stay:
      - Uses the same penalty logic as the environment.
      - Clips trajectory when weaning (6 consecutive zero-penalty steps) occurs.
      - Calculates total reward, steps to weaning for each stay.
      - Prints summary: reward distribution, average steps, and proportion not weaned.
    """
    import numpy as np
    import pandas as pd

    metrics_list = []
    total_rewards = []
    steps_list = []
    steps_to_weaning_list = []
    num_stays = 0
    num_not_weaned = 0
    
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in')
        if len(group) < obs_hrs + 1:
            continue
        
        rows = group.to_dict('records')
        # Seed history
        history = []
        for i in range(obs_hrs - 1):
            history.append((tuple(rows[i][col] for col in state_cols),
                            tuple(rows[i][col] for col in action_cols)))
        
        current_state = tuple(rows[obs_hrs-1][col] for col in state_cols)
        previous_action = tuple(rows[obs_hrs-2][col] for col in action_cols) # TODO: [update reward]
        current_action = tuple(rows[obs_hrs-2][col] for col in action_cols) # TODO: [update reward]
        total_reward = 0
        weaning_count = 0
        weaning_met = False
        steps_to_weaning = None
        action_diversity_set = set()
        
        # TODO: [update reward Part 2]
        # Iterate through trajectory
        for t in range(obs_hrs - 1, len(rows) - 1):
            if weaning_met:
                break
            step = t - (obs_hrs - 1)  # zero-index step
            if step >= EVAL_MAX_TRAJ_LENGTH and EVAL_ANALYSIS_REWARD_CLIP_MAX_TRAJ_LENGTH_FLAG: # clip trajectory length
                break
            previous_action = current_action
            current_action = tuple(rows[t][col] for col in action_cols)
            next_state = tuple(rows[t+1][col] for col in state_cols)

            action_diversity_set.add(current_action)
            
            # State penalty
            state_penalty = -1 if any(
                not (min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= 
                     min_max_values_guidelines_bin_dict[col]['max'])
                for i, col in enumerate(state_cols)
            ) else 0

            state_reward = sum(
                1 if min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)
            
            # Action penalty
            # action_penalty = -1 if any(
            #     not (min_max_values_guidelines_bin_dict[col]['min'] <= a <= 
            #          min_max_values_guidelines_bin_dict[col]['max'])
            #     for a, col in zip(current_action, action_cols)
            # ) else 0
            action_penalty = sum(
                -1 if current_action[i] < min_max_values_guidelines_bin_dict[col]['min'] or current_action[i] > min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)
            
            # Stable penalty
            # stable_penalty = 0
            # if len(history) >= stable_hr:
            #     stable = True
            #     for i in range(-stable_hr + 1, 0):
            #         for j in range(len(state_cols)):
            #             diff = abs(history[i][0][j] - history[i-1][0][j])
            #             if diff >= stable_threshold:
            #                 stable = False
            #                 break
            #         if not stable:
            #             break
            #     if stable:
            #         last_state = history[-1][0]
            #         for j in range(len(state_cols)):
            #             diff = abs(current_state[j] - last_state[j])
            #             if diff >= stable_threshold:
            #                 stable = False
            #                 break
            #     stable_penalty = -1 if not stable else 0

            state_stable_penalty = sum(
                -1 if abs(next_state[i] - current_state[i]) >= stable_threshold else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)
            
            action_stable_penalty = sum(
                -1 if abs(previous_action[i] - current_action[i]) >= stable_threshold * 2 else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)

            # Timestamp penalty
            timestamp_penalty = -1 if (step + 1) % timestamp_reward_step == 0 else 0
            
            # Check weaning condition
            # if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0:
            if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0 and action_stable_penalty == 0: # TODO: try add action_stable_penalty
                weaning_count += 1
            else:
                weaning_count = 0

            # Accumulate reward
            if not EVAL_ACTION_PENALTY_FLAG:
                action_penalty = 0
            if not EVAL_ACTION_STABLE_PENALTY_FLAG:
                action_stable_penalty = 0
            reward = state_reward + action_penalty + state_stable_penalty + action_stable_penalty + timestamp_penalty
            total_reward += reward
            
            # if weaning_count >= 6: # TODO: 6 is set by my self
            # if weaning_count >= 6 and step >= 23: # TODO: 6 is set by my self # TODO: [update reward]
            if weaning_count >= 6: # TODO: 6 is set by my self
                # total_reward += 10 # TODO: [update reward]
                if EVAL_WEANING_REWARD_FLAG:
                    total_reward += WEANING_REWARD
                weaning_met = True
                steps_to_weaning = step + 1
                break
            # [update reward] TODO: elif last step and weaning_cout >= 2
            # elif step == len(rows) - obs_hrs - 1 and weaning_count >= 2:
            #     total_reward += 100
            #     weaning_met = True
            #     steps_to_weaning = step + 1

            # Update history and state for next iteration
            history = history[1:] + [(current_state, current_action)]
            current_state = next_state
        
        # If not weaned, mark steps_to_weaning as total steps
        if not weaning_met:
            if EVAL_ANALYSIS_REWARD_CLIP_MAX_TRAJ_LENGTH_FLAG:
                steps_to_weaning = min(len(rows) - obs_hrs, EVAL_MAX_TRAJ_LENGTH)
            else:
                steps_to_weaning = len(rows) - obs_hrs
            num_not_weaned += 1
            total_reward -= EVAL_WEANING_REWARD
        
        metrics_list.append({
            'stay_id': stay_id,
            'total_reward': total_reward,
            'meet_weaning': weaning_met,
            'steps': steps_to_weaning,
            'action_diversity': len(action_diversity_set),
            'action': action_diversity_set
        })
        total_rewards.append(total_reward)
        steps_list.append(steps_to_weaning)
        if weaning_met:
            steps_to_weaning_list.append(steps_to_weaning)
        num_stays += 1
    
    metrics_df = pd.DataFrame(metrics_list)
    
    # Summary statistics
    avg_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)
    avg_steps = np.mean(steps_list)
    std_steps = np.std(steps_list)
    avg_steps_to_weaning = np.mean(steps_to_weaning_list) if steps_to_weaning_list else 0
    std_steps_to_weaning = np.std(steps_to_weaning_list) if steps_to_weaning_list else 0
    prop_not_weaned = num_not_weaned / num_stays if num_stays > 0 else None
    
    print(f"Total stays evaluated: {num_stays}")
    print(f"Reward (mean ± std): {avg_reward:.2f} ± {std_reward:.2f}")
    print(f"Average steps: {avg_steps:.2f} ± {std_steps:.2f}")
    print(f"Average steps to weaning: {avg_steps_to_weaning:.2f} ± {std_steps_to_weaning:.2f}")
    print(f"Proportion not weaned: {prop_not_weaned:.2%}")
    
    return metrics_df

In [ ]:
metrics_df = compute_trajectory_metrics(train_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
metrics_train_df = compute_trajectory_metrics(train_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
metrics_test_df = compute_trajectory_metrics(test_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
metrics_eICU_df = compute_trajectory_metrics(eICU_disc, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)

With ANALYSIS_REWARD_CLIP_MAX_TRAJ_LENGTH_FLAG, analysis small, median, large reward

In [ ]:
# Reward smallest 100
# metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=True)[:100]
metrics_df.sort_values(by="total_reward", ascending=True)[:100]
print("Smallest 100 rewards:")
print(f"Mean of Total Reward: {metrics_df.sort_values(by='total_reward', ascending=True)[:100]['total_reward'].mean():.2f}")
print(f"Std of Total Reward: {metrics_df.sort_values(by='total_reward', ascending=True)[:100]['total_reward'].std():.2f}")
print(f"Meet Extubation: {metrics_df.sort_values(by='total_reward', ascending=True)[:100]['meet_weaning'].mean():.2%}")
print(f"Mean of Steps: {metrics_df.sort_values(by='total_reward', ascending=True)[:100]['steps'].mean():.2f}")
print(f"Std of Steps: {metrics_df.sort_values(by='total_reward', ascending=True)[:100]['steps'].std():.2f}")
print(f"Mean of Steps to Weaning: {metrics_df.sort_values(by='total_reward', ascending=True)[:100][metrics_df.sort_values(by='total_reward', ascending=True)[:100]['meet_weaning'] == True]['steps'].mean():.2f}")
print(f"Std of Steps to Weaning: {metrics_df.sort_values(by='total_reward', ascending=True)[:100][metrics_df.sort_values(by='total_reward', ascending=True)[:100]['meet_weaning'] == True]['steps'].std():.2f}")
print(f"Action Diversity: {len(set.union(*metrics_df.sort_values(by='total_reward', ascending=True)[:100]['action']))}")

In [ ]:
def real_data_bemchmark_metrics(metrics_df, category, n_samples=100):
    if category == "small":
        ascending = True
        subset = metrics_df.sort_values(by='total_reward', ascending=ascending)[:n_samples]
    elif category == "median":
        ascending = True
        sorted_df = metrics_df.sort_values(by='total_reward', ascending=ascending)
        start_idx = len(metrics_df)//2-(n_samples//2)
        end_idx = len(metrics_df)//2+(n_samples//2)
        subset = sorted_df[start_idx:end_idx]
    elif category == "large":
        ascending = False
        subset = metrics_df.sort_values(by='total_reward', ascending=ascending)[:n_samples]
    elif category == "all":
        subset = metrics_df
    
    # Calculate metrics
    total_reward_mean = subset['total_reward'].mean()
    total_reward_std = subset['total_reward'].std()
    meet_extubation = subset['meet_weaning'].mean()
    avg_hours_mean = subset['steps'].mean()
    avg_hours_std = subset['steps'].std()
    
    # Calculate steps to weaning for cases that meet weaning criteria
    weaning_subset = subset[subset['meet_weaning'] == True]
    if len(weaning_subset) > 0:
        avg_hours_to_meet_mean = weaning_subset['steps'].mean()
        avg_hours_to_meet_std = weaning_subset['steps'].std()
    else:
        avg_hours_to_meet_mean = np.nan
        avg_hours_to_meet_std = np.nan
    
    action_diversity = len(set.union(*subset['action']))
    
    # Create result dictionary
    result = {
        'Category': category,
        'N_Samples': len(subset) if category == "all" else n_samples,
        'Total Reward (Mean±Std)': f"{total_reward_mean:.2f}±{total_reward_std:.2f}",
        'Meet Extubation': f"{meet_extubation:.2%}",
        'Avg Hours (Mean±Std)': f"{avg_hours_mean:.2f}±{avg_hours_std:.2f}",
        'Avg Hours to Meet (Mean±Std)': f"{avg_hours_to_meet_mean:.2f}±{avg_hours_to_meet_std:.2f}" if not np.isnan(avg_hours_to_meet_mean) else "N/A",
        'Action Diversity': action_diversity
    }
    
    # Convert to DataFrame
    result_df = pd.DataFrame([result])
    
    return result_df

# If you want to combine multiple categories into one DataFrame:
def get_all_metrics(metrics_df, n_samples=100):
    categories = ["small", "median", "large", "all"]
    all_results = []
    
    for category in categories:
        if category == "all":
            result_df = real_data_bemchmark_metrics(metrics_df, category)
        else:
            result_df = real_data_bemchmark_metrics(metrics_df, category, n_samples)
        all_results.append(result_df)
    
    combined_df = pd.concat(all_results, ignore_index=True)
    return combined_df

In [ ]:
# For a single category
# real_data_bemchmark_metrics_result_df = real_data_bemchmark_metrics(metrics_df, "small", 100)
# real_data_bemchmark_metrics_result_df

# For all categories combined
n_samples = 100
all_metrics_df = get_all_metrics(metrics_df, n_samples)
all_metrics_train_df = get_all_metrics(metrics_train_df, n_samples)
all_metrics_test_df = get_all_metrics(metrics_test_df, n_samples)
all_metrics_eICU_df = get_all_metrics(metrics_eICU_df, n_samples)
all_metrics_df

In [ ]:
all_metrics_test_df

In [ ]:
all_metrics_eICU_df

n_samples as 1000

In [ ]:
n_samples = 1000
all_metrics_df = get_all_metrics(metrics_df, n_samples)
all_metrics_test_df = get_all_metrics(metrics_test_df, n_samples)
all_metrics_eICU_df = get_all_metrics(metrics_eICU_df, n_samples)
all_metrics_df

In [ ]:
all_metrics_test_df

In [ ]:
all_metrics_eICU_df

In [ ]:
import pandas as pd
import numpy as np

def create_combined_metrics_csv(metrics_train_df, metrics_test_df, metrics_eICU_df, filename='combined_metrics.csv'):
    """
    Create a single CSV file combining all metrics for different datasets and sample sizes
    """
    
    all_combined_results = []
    
    # Define sample sizes
    sample_sizes = [100, 1000]
    
    # Define datasets with their corresponding dataframes
    datasets = {
        'Train': metrics_train_df,
        'Test': metrics_test_df,
        'eICU': metrics_eICU_df
    }
    
    # Process each dataset and sample size combination
    for dataset_name, df in datasets.items():
        for n_samples in sample_sizes:
            # Get metrics for current dataset and sample size
            metrics_df = get_all_metrics(df, n_samples)
            
            # Add dataset name and sample size columns
            metrics_df['Dataset'] = dataset_name
            metrics_df['Sample_Size'] = n_samples
            
            # Reorder columns to put Dataset and Sample_Size first
            cols = ['Dataset', 'Sample_Size'] + [col for col in metrics_df.columns if col not in ['Dataset', 'Sample_Size']]
            metrics_df = metrics_df[cols]
            
            all_combined_results.append(metrics_df)
    
    # Combine all results
    final_combined_df = pd.concat(all_combined_results, ignore_index=True)
    
    # Save to CSV
    final_combined_df.to_csv(filename, index=False)
    
    print(f"Combined metrics saved to {filename}")
    print(f"Total rows: {len(final_combined_df)}")
    print("\nPreview of the combined data:")
    print(final_combined_df.head(10))
    
    return final_combined_df

# Usage:
combined_metrics = create_combined_metrics_csv(
    metrics_train_df, 
    metrics_test_df, 
    metrics_eICU_df, 
    filename='../models/real_data_benchmark_metrics.csv'
)

In [ ]:
# metrics_df[metrics_df["meet_weaning"] == True]
# metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=True).head(50)
# metrics_df[(metrics_df["steps_to_weaning"] <= 48)].sort_values(by="total_reward", ascending=True).head(50)
# metrics_df.sort_values(by="total_reward", ascending=True)[int(len(metrics_df)/2)-10:int(len(metrics_df)/2)+11]
# metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="steps_to_weaning", ascending=True).describe()
metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=True).describe()

In [ ]:
# Reward smallest 100
metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=True)[:100].describe()

In [ ]:
# Reward largest 100
metrics_eICU_df[metrics_eICU_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=False)[:100].describe()
# metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=False)[40:60]

In [ ]:
# Reward median 100
metrics_test_df[metrics_test_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=True)[int(len(metrics_test_df)/2)-50:int(len(metrics_test_df)/2)+50].describe()
# metrics_df[metrics_df["meet_weaning"] == True].sort_values(by="total_reward", ascending=False)[40:60]

In [ ]:
metrics_train_df

In [ ]:
metrics_train_df[metrics_train_df["steps_to_weaning"] > 96]

In [ ]:
metrics_test_df

In [ ]:
metrics_train_df[metrics_train_df["steps_to_weaning"]==1]


In [ ]:
len(metrics_train_df[metrics_train_df["steps_to_weaning"]==1])/len(metrics_train_df)

In [ ]:
len(metrics_train_df[metrics_train_df["steps_to_weaning"]==6])/len(metrics_train_df)

In [ ]:
metrics_train_df[metrics_train_df["steps_to_weaning"]==6]

In [ ]:
metrics_df.describe()

In [ ]:
metrics_df

In [ ]:
def plot_metrics(metrics_df, title):
    # relationship between total reward and steps to weaning
    plt.figure(figsize=(10, 6))
    
    # Separate data based on meet_weaning status
    meet_weaning_true = metrics_df[metrics_df['meet_weaning'] == True]
    meet_weaning_false = metrics_df[metrics_df['meet_weaning'] == False]
    # Plot with different colors
    plt.scatter(meet_weaning_true['total_reward'], meet_weaning_true['steps'], 
                alpha=0.5, color='#1f77b4', label='Meet Extubation: True')
                # alpha=0.5, color='#1f77b4', label='Meet Weaning: True')
    plt.scatter(meet_weaning_false['total_reward'], meet_weaning_false['steps'], 
                alpha=0.5, color='#d62728', label='Meet Extubation: False')
                # alpha=0.5, color='#d62728', label='Meet Weaning: False')
    
    plt.title(f'{title} Total Reward vs Trajectory Length')
    plt.xlabel('Total Reward')
    # plt.ylabel('Steps to Extubation')
    plt.ylabel('Trajectory Length')
    plt.legend()
    plt.grid()
    plt.show()

plot_metrics(metrics_train_df, 'Train')
plot_metrics(metrics_test_df, 'Test')
plot_metrics(metrics_eICU_df, 'eICU')

clip by MAX_TRAJ_LENGTH = 120

In [ ]:
plot_metrics(metrics_train_df, 'Train')
plot_metrics(metrics_test_df, 'Test')
plot_metrics(metrics_eICU_df, 'eICU')

In [ ]:
plot_metrics(metrics_train_df, 'Train')
plot_metrics(metrics_test_df, 'Test')
plot_metrics(metrics_eICU_df, 'eICU')

In [ ]:
plot_metrics(metrics_train_df, 'Train')
plot_metrics(metrics_test_df, 'Test')
plot_metrics(metrics_eICU_df, 'eICU')

### Get Evaluation Initial Observation Categories (7 categories)

In [ ]:
def get_eval_init_obs_categories(train_df, metrics_df, state_cols, action_cols, baseline_cols, n_samples=100):
    # Total Reward Categories
    n = len(metrics_df)
    
    # Use min(n_samples, available samples) to handle cases where dataset is smaller than n_samples
    small_sample_size = min(n_samples, int(0.1 * n))
    large_sample_size = min(n_samples, int(0.1 * n))
    
    small_reward_patients = metrics_df.nsmallest(small_sample_size, 'total_reward')['stay_id']
    
    # For median, take samples from the middle 10% range
    # median_idx = int(0.45 * n), int(0.55 * n)
    median_idx = int(n//2 - n_samples//2), int(n//2 + n_samples//2)
    median_candidates = metrics_df.sort_values('total_reward').iloc[median_idx[0]:median_idx[1]]
    median_sample_size = min(n_samples, len(median_candidates))
    median_reward_patients = median_candidates.sample(n=median_sample_size, random_state=42)['stay_id']
    
    large_reward_patients = metrics_df.nlargest(large_sample_size, 'total_reward')['stay_id']

    small_reward_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in small_reward_patients]
    median_reward_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in median_reward_patients]
    large_reward_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in large_reward_patients]

    # Steps to Weaning Categories
    short_sample_size = min(n_samples, int(0.1 * n))
    long_sample_size = min(n_samples, int(0.1 * n))
    
    short_to_wean_patients = metrics_df.nsmallest(short_sample_size, 'steps')['stay_id']
    
    # median_wean_idx = int(0.45 * n), int(0.55 * n)
    median_wean_idx = int(n//2 - n_samples//2), int(n//2 + n_samples//2)
    median_wean_candidates = metrics_df.sort_values('steps').iloc[median_wean_idx[0]:median_wean_idx[1]]
    median_wean_sample_size = min(n_samples, len(median_wean_candidates))
    median_to_wean_patients = median_wean_candidates.sample(n=median_wean_sample_size, random_state=42)['stay_id']
    
    long_to_wean_patients = metrics_df.nlargest(long_sample_size, 'steps')['stay_id']

    short_to_wean_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in short_to_wean_patients]
    median_to_wean_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in median_to_wean_patients]
    long_to_wean_init_obs = [get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) for sid in long_to_wean_patients]

    # Common Initial Observations
    from collections import Counter
    initial_obs_list = [tuple(get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols)) 
                        for sid in train_df['stay_id'].unique() if get_initial_obs(train_df, sid, state_cols, action_cols, baseline_cols) is not None]
    obs_counter = Counter(initial_obs_list)
    common_init_obs = [obs for obs, count in obs_counter.items() if count > 1]  # Appears in multiple patients

    # Example usage
    # TODO: update as only small, median, and large reward
    categories = {
        'small_reward': small_reward_init_obs,
        'median_reward': median_reward_init_obs,
        'large_reward': large_reward_init_obs,
        # 'long_to_wean': long_to_wean_init_obs,
        # 'median_to_wean': median_to_wean_init_obs,
        # 'short_to_wean': short_to_wean_init_obs,
        # 'common': common_init_obs
    }

    # Filter out None values from init_obs lists
    categories = {k: [obs for obs in v if obs is not None] for k, v in categories.items()}
    return categories

In [ ]:
eval_init_obs_categories = get_eval_init_obs_categories(train_df, metrics_train_df, state_cols, action_cols, baseline_cols)

In [ ]:
len(eval_init_obs_categories["median_reward"])

### Real data Reward on different Evaluation Initial Observation Categories

In [ ]:
def get_mean_std_of_reward_and_steps(metrics_df):
    # 1. Mean and standard deviation of `total_reward`
    mean_reward = metrics_df['total_reward'].mean()
    std_reward = metrics_df['total_reward'].std()
    # 2. Mean and standard deviation of `steps_to_weaning`
    mean_steps_to_weaning = metrics_df['steps'].mean()
    std_steps_to_weaning = metrics_df['steps'].std()
    # 3. Action diversity across all 100 rows
    # Combine all action sets into one set
    all_actions = set().union(*metrics_df['action'])
    action_diversity = len(all_actions)

    return {
        'mean_reward': mean_reward,
        'std_reward': std_reward,
        'mean_steps_to_weaning': mean_steps_to_weaning,
        'std_steps_to_weaning': std_steps_to_weaning,
        'action_diversity': action_diversity
    }

def real_data_reward_across_eval_init_obs_categories(metrics_df, n_samples=100):
    n = len(metrics_df)
    small_reward_patients_n_samples_rows = metrics_df.nsmallest(min(n_samples, int(0.1 * n)), 'total_reward')[:n_samples]
    median_idx = int(n//2 - n_samples//2), int(n//2 + n_samples//2)
    median_reward_patients_n_samples_rows = metrics_df.sort_values('total_reward').iloc[median_idx[0]:median_idx[1]][:n_samples]
    large_reward_patients_n_samples_rows = metrics_df.nlargest(min(n_samples, int(0.1 * n)), 'total_reward')[:n_samples]
    small_reward_stats_results = get_mean_std_of_reward_and_steps(small_reward_patients_n_samples_rows)
    median_reward_stats_results = get_mean_std_of_reward_and_steps(median_reward_patients_n_samples_rows)
    large_reward_stats_results = get_mean_std_of_reward_and_steps(large_reward_patients_n_samples_rows)

    short_to_wean_patients_n_samples_rows = metrics_df.nsmallest(min(n_samples, int(0.1 * n)), 'steps')[:n_samples]
    median_wean_idx = int(n//2 - n_samples//2), int(n//2 + n_samples//2)
    median_to_wean_patients_n_samples_rows = metrics_df.sort_values('steps').iloc[median_wean_idx[0]:median_wean_idx[1]][:n_samples]
    long_to_wean_patients_n_samples_rows = metrics_df.nlargest(min(n_samples, int(0.1 * n)), 'steps')[:n_samples]
    short_to_wean_stats_results = get_mean_std_of_reward_and_steps(short_to_wean_patients_n_samples_rows)
    median_to_wean_stats_results = get_mean_std_of_reward_and_steps(median_to_wean_patients_n_samples_rows)
    long_to_wean_stats_results = get_mean_std_of_reward_and_steps(long_to_wean_patients_n_samples_rows)

    # Combine results into a single dictionary
    results = {
        'small_reward': small_reward_stats_results,
        'median_reward': median_reward_stats_results,
        'large_reward': large_reward_stats_results,
        'long_to_wean': long_to_wean_stats_results,
        'median_to_wean': median_to_wean_stats_results,
        'short_to_wean': short_to_wean_stats_results
    }
    return results

def plot_reward_steps(metrics_df, title):
    # Plotting the distribution of total reward and steps to weaning
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    # Total Reward Distribution
    ax[0].hist(metrics_df['total_reward'], bins=30, color='blue', alpha=0.7)
    ax[0].set_title(f'{title} Total Reward Distribution')
    ax[0].set_xlabel('Total Reward')
    ax[0].set_ylabel('Frequency')
    
    # Steps to Weaning Distribution
    ax[1].hist(metrics_df['steps_to_weaning'], bins=30, color='green', alpha=0.7)
    ax[1].set_title(f'{title} Steps to Weaning Distribution')
    ax[1].set_xlabel('Steps to Weaning')
    ax[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
train_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_train_df, n_samples=100)
test_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_test_df, n_samples=100)
eICU_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_eICU_df, n_samples=100)
train_df_reward_stats

In [ ]:
test_df_reward_stats

In [ ]:
eICU_df_reward_stats

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert the dictionary data to a more convenient format for visualization
def extract_metric_data(stats_dict, dataset_name):
    rows = []
    for category, metrics in stats_dict.items():
        for metric_name, value in metrics.items():
            # Convert numpy.float64 to regular float for better compatibility 
            if isinstance(value, np.float64):
                value = float(value)
            rows.append({
                'Dataset': dataset_name,
                'Category': category,
                'Metric': metric_name,
                'Value': value
            })
    return pd.DataFrame(rows)

# Create the datasets
train_data = extract_metric_data(train_df_reward_stats, 'Train')
test_data = extract_metric_data(test_df_reward_stats, 'Test')
eicu_data = extract_metric_data(eICU_df_reward_stats, 'eICU')

# Combine the datasets
combined_data = pd.concat([train_data, test_data, eicu_data], ignore_index=True)

# Handling different metrics appropriately
metrics_to_plot = ['mean_reward', 'std_reward', 'mean_steps_to_weaning', 
                  'std_steps_to_weaning', 'action_diversity']

# Create subplots for each metric
fig, axes = plt.subplots(len(metrics_to_plot), 1, figsize=(14, 20))
fig.suptitle('Comparison of Metrics Across Datasets and Categories', fontsize=16)

for i, metric in enumerate(metrics_to_plot):
    # Filter for this metric
    metric_data = combined_data[combined_data['Metric'] == metric].copy()
    
    # Pivot data for heatmap
    pivot_data = metric_data.pivot_table(
        index='Category', 
        columns='Dataset', 
        values='Value'
    )
    
    # Sort categories for better visualization
    # You can modify this sorting logic as needed
    category_order = [
        'large_reward', 'median_reward', 'small_reward',
        'short_to_wean', 'median_to_wean', 'long_to_wean'
    ]
    pivot_data = pivot_data.reindex(category_order)
    
    # Determine colormap based on metric
    # Green is good for rewards, red is good for fewer steps
    if metric == 'mean_reward':
        cmap = 'RdYlGn'  # Red-Yellow-Green: Red for negative, Green for positive rewards
    elif 'steps' in metric:
        cmap = 'RdYlGn_r'  # Reversed: Green for fewer steps, Red for more steps
    elif metric == 'action_diversity':
        cmap = 'viridis'  # Neutral colormap for diversity
    else:
        cmap = 'coolwarm'  # Default colormap
    
    # Plot heatmap
    ax = axes[i]
    sns.heatmap(pivot_data, annot=True, fmt=".2f", cmap=cmap, ax=ax, linewidths=0.5)
    
    # Format the heatmap
    ax.set_title(f'{metric}', fontsize=14)
    ax.set_ylabel('')
    if i < len(metrics_to_plot) - 1:
        ax.set_xlabel('')
    else:
        ax.set_xlabel('Dataset')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

# Let's also create a radar chart to visualize the multidimensional aspects
# Just for the large_reward, median_reward, and small_reward categories

# Function to create radar chart
def create_radar_chart(categories, metrics, data, title):
    num_metrics = len(metrics)
    angles = np.linspace(0, 2*np.pi, num_metrics, endpoint=False).tolist()
    angles += angles[:1]  # Close the polygon
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Add metrics labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics)
    
    # Add data for each category and dataset
    for dataset in ['Train', 'Test', 'eICU']:
        for category in categories:
            values = []
            for metric in metrics:
                value = data[(data['Dataset'] == dataset) & 
                             (data['Category'] == category) & 
                             (data['Metric'] == metric)]['Value'].values[0]
                values.append(value)
            values += values[:1]  # Close the polygon
            
            ax.plot(angles, values, linewidth=2, label=f"{dataset} - {category}")
            ax.fill(angles, values, alpha=0.1)
    
    ax.set_title(title, fontsize=15)
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    return fig

# Define categories and metrics for radar chart
radar_categories = ['large_reward', 'median_reward', 'small_reward']
radar_metrics = ['mean_reward', 'mean_steps_to_weaning', 'action_diversity']

# Normalize data for radar chart (different scales)
radar_data = combined_data.copy()
for metric in radar_metrics:
    metric_subset = radar_data[radar_data['Metric'] == metric]
    min_val = metric_subset['Value'].min()
    max_val = metric_subset['Value'].max()
    radar_data.loc[radar_data['Metric'] == metric, 'Value'] = (
        (radar_data.loc[radar_data['Metric'] == metric, 'Value'] - min_val) / (max_val - min_val)
    )

# Create radar chart
radar_fig = create_radar_chart(
    radar_categories, 
    radar_metrics,
    radar_data,
    'Normalized Comparison of Reward Categories Across Datasets'
)

plt.tight_layout()
plt.show()

# Let's also create a bar chart for mean_reward across all categories and datasets
plt.figure(figsize=(14, 8))
reward_data = combined_data[combined_data['Metric'] == 'mean_reward']
sns.barplot(x='Category', y='Value', hue='Dataset', data=reward_data)
plt.title('Mean Reward Comparison Across Categories and Datasets', fontsize=15)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# And a bar chart for mean_steps_to_weaning across all categories and datasets
plt.figure(figsize=(14, 8))
steps_data = combined_data[combined_data['Metric'] == 'mean_steps_to_weaning']
sns.barplot(x='Category', y='Value', hue='Dataset', data=steps_data)
plt.title('Mean Steps to Weaning Comparison Across Categories and Datasets', fontsize=15)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
train_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_train_df, top_n=100)
test_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_test_df, top_n=100)
eICU_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_eICU_df, top_n=100)
train_df_reward_stats

In [ ]:
train_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_train_df, top_n=100)
test_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_test_df, top_n=100)
eICU_df_reward_stats = real_data_reward_across_eval_init_obs_categories(metrics_eICU_df, top_n=100)
train_df_reward_stats

## Gym Patient Environment (with d3rlpy)

In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from collections import defaultdict
import pandas as pd
from sklearn.neighbors import KDTree
import itertools
import d3rlpy
from d3rlpy.algos import DQNConfig
from d3rlpy.models.q_functions import MeanQFunctionFactory
from d3rlpy.models import DefaultEncoderFactory


In [ ]:
import numpy as np

def transform_to_reset_format(data, state_cols, action_cols, baseline_cols):
    # Calculate lengths of each section
    state_len = len(state_cols)
    action_len = len(action_cols)
    baseline_len = len(baseline_cols)

    # Split the data into states, actions, and baselines
    num_states = (len(data) - baseline_len) // (state_len + action_len)
    states = []
    actions = []
    for i in range(num_states):
        start_idx = i * (state_len + action_len)
        states.append(tuple(data[start_idx:start_idx + state_len]))
        actions.append(tuple(data[start_idx + state_len:start_idx + state_len + action_len]))
    # last state is the current state
    current_state = tuple(data[-(state_len + baseline_len):-baseline_len])
    states.append(current_state)
    # Extract baselines
    baselines = tuple(data[-baseline_len:])

    # Create the final format
    result = []
    for i in range(num_states+1):
        # Use the last action if the current action is missing
        action = actions[i] if i < len(actions) else actions[-1]
        result.append((states[i], action))
    
    # Add the baselines
    result.append(baselines)

    return tuple(result)

test_data = (
    np.int32(1), np.int32(2), np.int32(3),  # state 1
    np.int32(4), np.int32(5),               # action 1
    np.int32(6), np.int32(7), np.int32(8),  # state 2
    np.int32(9), np.int32(0),               # action 2
    np.int32(1), np.int32(2), np.int32(3),  # state 3 (current state)
    np.int32(4), np.int32(5)                # baselines
)

result = transform_to_reset_format(test_data, state_cols, action_cols, baseline_cols)
print(result)

In [ ]:
# === PatientEnvironment Class ===
class PatientEnvironment(gym.Env):
    """
    A Gym environment for patient simulation using historical medical data.
    """
    def __init__(self, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, stable_hr=2, stable_threshold=2, timestamp_reward_step=1):
        super(PatientEnvironment, self).__init__()
        
        # Configuration
        self.train_transitions = train_transitions
        self.state_cols = state_cols
        self.action_cols = action_cols
        self.baseline_cols = baseline_cols
        self.num_bins_dict = num_bins_dict
        self.min_max_values_guidelines_bin_dict = min_max_values_guidelines_bin_dict
        self.obs_hrs = obs_hrs
        self.stable_hr = stable_hr
        self.stable_threshold = stable_threshold
        self.timestamp_reward_step = timestamp_reward_step
        
        # Define observation space
        state_bins = [num_bins_dict[col] for col in state_cols]
        action_bins = [num_bins_dict[col] for col in action_cols]
        baseline_bins = [num_bins_dict[col] for col in baseline_cols]
        nvec = []
        for _ in range(obs_hrs - 1):  # History of obs_hrs-1 state-action pairs
            nvec.extend(state_bins)
            nvec.extend(action_bins)
        nvec.extend(state_bins)  # Current state
        nvec.extend(baseline_bins)  # Baseline
        self.observation_space = spaces.MultiDiscrete(nvec)
        
        # Define action space
        self.action_bins = action_bins
        num_actions = np.prod(action_bins)
        self.action_space = spaces.Discrete(num_actions)
        
        # Action mapping
        self.id_to_action = {i: a for i, a in enumerate(itertools.product(*[range(b) for b in action_bins]))}
        self.action_to_id = {a: i for i, a in self.id_to_action.items()}
        
        # State variables
        self.history = []  # List of (state, action) tuples
        self.current_state = None
        self.previous_action = None
        self.current_action = None
        self.current_baseline = None
        self.meet_weaning_history = []
        self.steps = 0
        
        # KDTree for approximation
        self.train_keys_list = list(train_transitions.keys())
        self.train_X = np.array([self._flatten_key(key) for key in self.train_keys_list])
        self.tree = KDTree(self.train_X, metric='manhattan')

    # def reset(self):
    #     """Reset the environment to an initial state."""
    #     initial_key = random.choice(self.train_keys_list)
    #     self.history = list(initial_key[:self.obs_hrs - 1])  # obs_hrs-1 state-action pairs
    #     self.current_state = initial_key[self.obs_hrs - 1][0]  # Current state
    #     self.current_baseline = initial_key[-1]  # Baseline
    #     self.meet_weaning_history = []
    #     self.steps = 0
    #     return self._get_obs()

    def reset(self, initial_key=None):
        """Reset the environment to an initial state."""
        if initial_key is None:
            initial_key = random.choice(self.train_keys_list)
        else:
            # for i in range(len(initial_key)):
            #     initial_key[i] = int(initial_key[i])
            state_len = len(state_cols)
            action_len = len(action_cols)
            baseline_len = len(baseline_cols)

            # Split the data into states, actions, and baselines
            num_states = (len(initial_key) - baseline_len) // (state_len + action_len)
            states = []
            actions = []
            for i in range(num_states):
                start_idx = i * (state_len + action_len)
                states.append(tuple(initial_key[start_idx:start_idx + state_len]))
                actions.append(tuple(initial_key[start_idx + state_len:start_idx + state_len + action_len]))
            # last state is the current state
            current_state = tuple(initial_key[-(state_len + baseline_len):-baseline_len])
            states.append(current_state)
            # Extract baselines
            baselines = tuple(initial_key[-baseline_len:])

            # Create the final format
            result = []
            for i in range(num_states+1):
                # Use the last action if the current action is missing
                action = actions[i] if i < len(actions) else actions[-1]
                result.append((states[i], action))
            
            # Add the baselines
            result.append(baselines)

            initial_key = tuple(result)

        # print(f"initial_key: {initial_key}")
        self.history = list(initial_key[:self.obs_hrs - 1])  # obs_hrs-1 state-action pairs
        # print(f"self.history: {self.history}")
        self.current_state = initial_key[self.obs_hrs - 1][0]  # Current state
        # print(f"self.current_state: {self.current_state}")
        self.current_action = initial_key[self.obs_hrs - 2][1]  # Previous action
        # print(f"self.previous_action: {self.previous_action}")
        self.current_baseline = initial_key[-1]  # Baseline
        # print(f"self.current_baseline: {self.current_baseline}")
        self.meet_weaning_history = []
        self.steps = 0
        return self._get_obs()

    def step(self, action):
        """Take a step in the environment."""
        self.previous_action = self.current_action
        self.current_action = self.id_to_action[action]
        key = tuple(self.history) + ((self.current_state, self.current_action),) + (self.current_baseline,) # TODO: self debug
        if key in self.train_transitions and not ALL_USE_UNSEEN_FLAG: # [cont_update]
            next_state = random.choice(self.train_transitions[key])
            nonsense_action = False
        else:
            next_state, nonsense_action = self._action_based_approximation_filter_nonsense_action(key)
        
        # Update state
        self.history = self.history[1:] + [(self.current_state, self.current_action)]
        self.steps += 1
        reward = self._calculate_reward(self.current_action, next_state) # TODO: since we need to get the current state before update
        self.current_state = next_state
        done = self._check_termination() or nonsense_action
        meet_weaning = False

        # termination have three cases:
        # 1. meet weaning condition
        # 2. nonsense action
        # 3. steps exceed max trajectory length

        # 1. meet weaning condition
        if (len(self.meet_weaning_history) >= 6 and all(self.meet_weaning_history[-6:])):
            # print("meet weaning condition")
            meet_weaning = True
            reward += EVAL_WEANING_REWARD
        # 2. nonsense action
        elif nonsense_action:
            # print("nonsense action")
            reward -= EVAL_WEANING_REWARD
        # 3. steps exceed max trajectory length
        elif self.steps >= EVAL_MAX_TRAJ_LENGTH:
            # print("steps exceed max trajectory length")
            reward -= EVAL_WEANING_REWARD

        return self._get_obs(), reward, done, {"nonsense_action": nonsense_action, "meet_weaning": meet_weaning} # TODO: add info dict for nonsense action

    def _get_obs(self):
        """Return the current observation as a flattened array."""
        flat = []
        for s, a in self.history:
            flat.extend(s)
            flat.extend(a)
        flat.extend(self.current_state)
        flat.extend(self.current_baseline)
        return np.array(flat, dtype=np.int32)

    def _calculate_reward(self, current_action, next_state):
        """
        Calculate the reward based on next_state, action, and history.

        Penalties:
        - state_penalty: -1 if any next_state value is outside its guideline range (else 0)
        - action_penalty: -1 if any action is outside its guideline range (else 0)
        - stable_penalty: -1 if any jump >= stable_threshold in the last stable_hr steps (else 0)
        - timestamp_penalty: -1 every timestamp_reward_step steps (else 0)

        A one-time +100 reward (weaning bonus) is granted once the state_penalty,
        action_penalty, and stable_penalty are all zero for 6 consecutive steps,
        after step >= 23.  The episode then terminates via _check_termination().
        """
        # TODO: [update reward Part 1]
        # TODO: [update reward] reward shaping
        # State penalty: -1 if any vital sign is outside guidelines
        state_penalty = -1 if any(
            not (self.min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <=
                self.min_max_values_guidelines_bin_dict[col]['max'])
            for i, col in enumerate(self.state_cols)
        ) else 0

        state_reward = sum(
            1 if min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= min_max_values_guidelines_bin_dict[col]['max'] else 0
            for i, col in enumerate(self.state_cols)
        ) / len(self.state_cols)

        
        action_penalty = sum(
            -1 if current_action[i] < min_max_values_guidelines_bin_dict[col]['min'] or current_action[i] > min_max_values_guidelines_bin_dict[col]['max'] else 0
            for i, col in enumerate(action_cols)
        ) / len(action_cols)


        state_stable_penalty = sum(
            -1 if abs(next_state[i] - self.current_state[i]) >= self.stable_threshold else 0
            for i, col in enumerate(state_cols)
        ) / len(state_cols)

        action_stable_penalty = sum(
            -1 if abs(self.current_action[i] - self.previous_action[i]) >= self.stable_threshold * 2 else 0 # TODO: [update reward] *2 not sure, hope can guide not (8, 5), (8, 5), (1, 0)
            for i, col in enumerate(action_cols)
        ) / len(action_cols)


        # Timestamp penalty: -1 every timestamp_reward_step steps
        timestamp_penalty = -1 if (self.steps + 1) % self.timestamp_reward_step == 0 else 0

        # Record whether all three penalties are zero (weaning condition) at this step
        meet_condition = (state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0 and action_stable_penalty == 0) # TODO: [update reward] add action_stable_penalty
        # meet_condition = (state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0)
        # Total reward is sum of all penalties (non-positive unless all are zero)
        
        if not EVAL_ACTION_PENALTY_FLAG:
            action_penalty = 0
        if not EVAL_ACTION_STABLE_PENALTY_FLAG:
            action_stable_penalty = 0

        reward = state_reward + action_penalty + state_stable_penalty + action_stable_penalty + timestamp_penalty

        self.meet_weaning_history.append(1 if meet_condition else 0)

        return reward


    def _check_termination(self):
        """Check if the episode should terminate."""
        return (len(self.meet_weaning_history) >= 6 and all(self.meet_weaning_history[-6:])) or self.steps >= EVAL_MAX_TRAJ_LENGTH # TODO: [update reward] add and self.steps >= 23 s.t. it at least 24 hr (not sure reasonable)
        # return (len(self.meet_weaning_history) >= 6 and all(self.meet_weaning_history[-6:])) or self.steps >= 120 # TODO: [update reward] add and self.steps >= 23 s.t. it at least 24 hr (not sure reasonable)
        # return (len(self.meet_weaning_history) >= 6 and all(self.meet_weaning_history[-6:])) and self.steps >= 23 or self.steps >= 120 # TODO: [update reward] add and self.steps >= 23 s.t. it at least 24 hr (not sure reasonable)

    def _action_based_approximation(self, key):
        """Approximate next state when key is not in transitions."""
        desired_action = key[-2][1]
        flat_key = self._flatten_key(key)
        distances, indices = self.tree.query(flat_key.reshape(1, -1), k=100) # TODO: number of neighbors, should be selected by experiment
        candidates = [self.train_keys_list[i] for i in indices[0] if self.train_keys_list[i][-2][1] == desired_action]
        # print(f"len of candidates: {len(candidates)}")
        # print(f"key: {key}")
        # print(f"candidates: {candidates}")
        if not candidates:
            return key[-2][0]
        all_deltas = [tuple(np.array(s_next) - np.array(cand_key[-2][0])) 
                      for cand_key in candidates for s_next in self.train_transitions[cand_key]]
        delta = random.choice(all_deltas) if all_deltas else (0,) * len(self.state_cols)
        s_t = key[-2][0]
        return tuple(np.clip(s_t[i] + delta[i], 0, self.num_bins_dict[self.state_cols[i]] - 1) 
                    for i in range(len(s_t)))
    
    def _action_based_approximation_filter_nonsense_action(self, key):
        """Approximate next state when key is not in transitions."""
        desired_action = key[-2][1]
        flat_key = self._flatten_key(key)
        distances, indices = self.tree.query(flat_key.reshape(1, -1), k=100) # TODO: number of neighbors, should be selected by experiment
        candidates = [self.train_keys_list[i] for i in indices[0] if self.train_keys_list[i][-2][1] == desired_action]
        # print(f"len of candidates: {len(candidates)}")
        # print(f"key: {key}")
        # print(f"candidates: {candidates}")
        if not candidates:
            return key[-2][0], True # TODO: currently return that last state, but should penalty reward since this pair not seen in dataset with 100 around neighbors
        all_deltas = [tuple(np.array(s_next) - np.array(cand_key[-2][0])) 
                      for cand_key in candidates for s_next in self.train_transitions[cand_key]]
        delta = random.choice(all_deltas) if all_deltas else (0,) * len(self.state_cols)
        s_t = key[-2][0]
        return tuple(np.clip(s_t[i] + delta[i], 0, self.num_bins_dict[self.state_cols[i]] - 1) 
                    for i in range(len(s_t))), False
    
    def _transformer_approximation(self, key, transitions, n_samples=1):
        """Transformer-based approximation of next state."""
        
        # Load probabilistic Transformer model
        if 'probabilistic_transformer_model' not in globals():
            model_dir = os.path.join(mimic_iv_prefix_path, "models")
            # model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best.pth')
            model_path = os.path.join(model_dir, 'probabilistic_probabilistictransformernextstatepredictor_best_ground_truth_with_neighbors.pth') # [gt_with_neighbors_update]
            
            try:
                # Load the checkpoint
                checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
                
                # Use the saved output_dim_dict from checkpoint if available
                if 'output_dim_dict' in checkpoint:
                    output_dim_dict = checkpoint['output_dim_dict']
                else:
                    output_dim_dict = {var_name: num_bins_dict[var_name] for var_name in state_cols}
                
                # Initialize the model
                probabilistic_transformer = ProbabilisticTransformerNextStatePredictor(
                    # input_dim=StateTransitionDataset(transitions).features.shape[1],
                    input_dim=input_dim,
                    hidden_dim=128,
                    num_layers=2,
                    output_dim_dict=output_dim_dict,
                    dropout=0.2,
                    nhead=4  # Add the nhead parameter that was used in training
                )
                
                # Load the state dict using the correct key
                probabilistic_transformer.load_state_dict(checkpoint['model_state_dict'])
                print("Successfully loaded Transformer model from:", model_path)
                globals()['probabilistic_transformer_model'] = probabilistic_transformer
                
            except (FileNotFoundError, KeyError) as e:
                print(f"Error loading Transformer model: {e}")
                print("Training a new probabilistic Transformer model...")
                probabilistic_transformer = train_probabilistic_model(
                    ProbabilisticTransformerNextStatePredictor,
                    train_transitions, 
                    state_cols, 
                    hidden_dim=128, 
                    nhead=4,
                    num_layers=2,
                    dropout=0.2
                )
                globals()['probabilistic_transformer_model'] = probabilistic_transformer
        
        # Use the cached model for prediction
        transformer_samples = probabilistic_next_state_prediction(
            globals()['probabilistic_transformer_model'], 
            key, 
            n_samples=n_samples
        )
        
        return transformer_samples[0]

    def _flatten_key(self, key):
        """Flatten a key for KDTree."""
        flat = []
        for s, a in key[:-1]:
            flat.extend(s)
            flat.extend(a)
        flat.extend(key[-1])
        return np.array(flat, dtype=np.int32)


In [ ]:
def flatten_state(history, current_state, baseline):
    flat = []
    for s, a in history:
        flat.extend(s)
        flat.extend(a)
    flat.extend(current_state)
    flat.extend(baseline)
    return np.array(flat, dtype=np.int32)


def preprocess_data(df, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, stable_hr=2, stable_threshold=2, timestamp_reward_step=1, satisfy_weaning_reward=10):
    """
    Preprocess dataframe into a D3RLPY MDPDataset, using the consistent reward logic:
      - state_penalty = -1 if any vital out of guideline range (else 0)
      - action_penalty = -1 if any action out of guideline range (else 0)
      - state_stable_penalty = -1 if any jump >= threshold over last stable_hr states (else 0)
      - timestamp_penalty = -1 every timestamp_reward_step steps (else 0)
      - satisfy_weaning_reward = +100 granted once when all three penalties are 0 for 6 consecutive steps (then terminal).
    """
    from itertools import product
    import numpy as np
    import d3rlpy
    
    # Build action -> ID map
    action_bins = [num_bins_dict[col] for col in action_cols]
    action_to_id = {tuple(a): i
                    for i, a in enumerate(product(*[range(b) for b in action_bins]))}
    
    transitions = []
    not_met_weaning_count = 0
    met_weaning_count = 0
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in')
        if len(group) < obs_hrs + 1:
            continue
        
        history = []
        rows = group.to_dict('records')
        # Seed history with first (obs_hrs-1) state-action pairs
        for i in range(obs_hrs - 1):
            s = tuple(rows[i][col] for col in state_cols)
            a = tuple(rows[i][col] for col in action_cols)
            current_action = a
            history.append((s, a))
        baseline = tuple(rows[0][col] for col in baseline_cols)
        
        weaning_met = False
        weaning_count = 0
        
        transition = []
        # Iterate from obs_hrs-1 to the penultimate row
        # TODO: [update reward Part 3]
        for t in range(obs_hrs - 1, len(rows) - 1):
            # if weaning_met: # TODO: [update reward] if want clip at the first time met weaning condition
            #     break  # Stop once weaning is achieved (trajectory clipped)
            previous_action = current_action
            step = t - (obs_hrs - 1)  # zero-indexed step count
            current_state = tuple(rows[t][col] for col in state_cols)
            current_action = tuple(rows[t][col] for col in action_cols)
            next_state    = tuple(rows[t+1][col] for col in state_cols)
            
            # 1) State penalty
            state_penalty = -1 if any(
                next_state[i] < min_max_values_guidelines_bin_dict[col]['min'] or
                next_state[i] > min_max_values_guidelines_bin_dict[col]['max']
                for i, col in enumerate(state_cols)
            ) else 0

            state_reward = sum(
                1 if min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)
            
            # 2) Action penalty
            
            action_penalty = sum(
                -1 if current_action[i] < min_max_values_guidelines_bin_dict[col]['min'] or current_action[i] > min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)
            
            # 3) Stable penalty # TODO: [update reward] make sure it only check next state - current state
            # for each of the state_cols, next_state - current_state >= stable_threshold then state_stable_penalty = -1, don't need to check history
            state_stable_penalty = sum(
                -1 if abs(next_state[i] - current_state[i]) >= stable_threshold else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)

            action_stable_penalty = sum(
                -1 if abs(previous_action[i] - current_action[i]) >= stable_threshold * 2 else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)
            
            # 4) Timestamp penalty (every timestamp_reward_step)
            if TIMESTAMP_PENALTY_FLAG:
                timestamp_penalty = 1 if (step + 1) % timestamp_reward_step == 0 else 0 # TODO: [update reward] --> set 0 due to transition MDP don't have sequential concept
            else:
                timestamp_penalty = 0 if (step + 1) % timestamp_reward_step == 0 else 0 # TODO: [update reward] --> set 0 due to transition MDP don't have sequential concept
                                                                                    # TODO: update --> to align with the Patient Environment
            
            # 5) Check if all penalties are zero (weaning condition)
            # if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0:
            if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0 and action_stable_penalty == 0:
                weaning_count += 1
            else:
                weaning_count = 0

            # Base reward (sum of penalties)
            # reward = state_penalty + action_penalty + state_stable_penalty + timestamp_penalty
            if not ACTION_PENALTY_FLAG:
                action_penalty = 0
            if not ACTION_STABLE_PENALTY_FLAG:
                action_stable_penalty = 0
            if not STATE_REWARD_FLAG:
                state_reward = 0
            if not STATE_STABLE_PENALTY_FLAG:
                state_stable_penalty = 0

            reward = state_reward + action_penalty + state_stable_penalty + action_stable_penalty + timestamp_penalty
            
            # 6) Grant +100 and clip if sustained for 6 steps (with min step 23)
            terminal = False
            # if weaning_count >= 6 and step >= 23: # TODO: [update reward]
            if weaning_count >= 6: # TODO: [update reward]
                weaning_met = True
                if WEANING_REWARD_FLAG:
                    reward += WEANING_REWARD
            #     reward += satisfy_weaning_reward
            #     terminal = True
            if t == len(rows) - 2:
                terminal = True
                if weaning_count < 6 and WEANING_REWARD_FLAG: # TODO: [update reward] not sure will cause problem
                    reward -= WEANING_REWARD
            # [update reward] TODO: else if the traj end with last step satisfy weaning condition, then reward += satisfy_weaning_reward
            # since the real data might wean without keep 6 steps stable, but the trajectory path may be good
            # elif t == len(rows) - 2 and weaning_count >= 2: # TODO: not sure - 2 or - 1?
            #     reward += satisfy_weaning_reward
            #     terminal = True
            #     weaning_met = True
            
            # Flatten observation and next_observation
            obs = flatten_state(history, current_state, baseline)
            next_history = history[-(stable_hr-2):] if len(history) >= stable_hr - 1 else history[1:]
            next_history = next_history + [(current_state, current_action)]
            next_obs = flatten_state(next_history, next_state, baseline)
            
            # TODO: [update reward] if the traj end without satisfy weaning condition, maybe not add into the MDP dataset
            transition.append({
                'observation':       obs,
                'action':            action_to_id[current_action],
                'reward':            reward,
                'next_observation':  next_obs,
                'terminal':          terminal
            })
            
            # Update history
            history = history[1:] + [(current_state, current_action)]
        # Append transitions if weaning condition met
        if weaning_met:
            transitions.extend(transition)
            met_weaning_count += 1
        else:
            transitions.extend(transition) # TODO: [update reward] since the MDP is "transition", so met or not doesn't matter
            not_met_weaning_count += 1
    print(f"Number of trajectories meeting weaning condition: {met_weaning_count}")
    print(f"Number of trajectories not meeting weaning condition: {not_met_weaning_count}")
    print(f"Total number of trajectories: {met_weaning_count + not_met_weaning_count}")
    print(f"percentage of trajectories meeting weaning condition: {met_weaning_count / (met_weaning_count + not_met_weaning_count) * 100:.2f}%")
    print(f"percentage of trajectories not meeting weaning condition: {not_met_weaning_count / (met_weaning_count + not_met_weaning_count) * 100:.2f}%")
    
    # Collate into arrays for MDPDataset
    observations      = np.stack([t['observation'] for t in transitions])
    actions           = np.array([t['action']           for t in transitions], dtype=np.int32)
    rewards           = np.array([t['reward']           for t in transitions], dtype=np.float32)
    next_observations = np.stack([t['next_observation'] for t in transitions])
    terminals         = np.array([t['terminal']         for t in transitions], dtype=bool)
    
    return d3rlpy.dataset.MDPDataset(observations, actions, rewards, terminals)


In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from collections import defaultdict
import pandas as pd
from sklearn.neighbors import KDTree
import itertools
import d3rlpy
import torch
from d3rlpy.algos import DQNConfig, DoubleDQNConfig, DiscreteSACConfig, DecisionTransformerConfig
from d3rlpy.algos import DiscreteDecisionTransformerConfig, DiscreteDecisionTransformer
from d3rlpy.models.encoders import DefaultEncoderFactory
from d3rlpy.models.q_functions import MeanQFunctionFactory
from d3rlpy.models import DefaultEncoderFactory
from d3rlpy import PositionEncodingType

In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from collections import defaultdict
import pandas as pd
from sklearn.neighbors import KDTree
import itertools
import d3rlpy
import torch
from d3rlpy.algos import DQNConfig, DoubleDQNConfig, DiscreteSACConfig, DecisionTransformerConfig, NFQ, NFQConfig
from d3rlpy.algos import DiscreteDecisionTransformerConfig, DiscreteDecisionTransformer
from d3rlpy.algos.transformer.inputs import TorchTransformerInput
from d3rlpy.algos import DiscreteBC, DiscreteBCConfig
from d3rlpy.models.encoders import DefaultEncoderFactory
from d3rlpy.models.q_functions import MeanQFunctionFactory
from d3rlpy.models import DefaultEncoderFactory
from d3rlpy import PositionEncodingType
from d3rlpy.algos import DiscreteBCQ, DiscreteCQL
from d3rlpy.algos import DiscreteBCQConfig, DiscreteCQLConfig
from d3rlpy.optimizers import OptimizerFactory
from torch.optim import Adam

In [ ]:
import os
import glob

def get_newest_json(algo_name):
    # Construct the search pattern for the algorithm's JSON files
    search_pattern = f"./d3rlpy_logs/{algo_name}_*/params.json"
    # Use glob to find all matching files
    json_files = glob.glob(search_pattern)
    # Sort the files by their modification time in descending order
    json_files.sort(key=os.path.getmtime, reverse=True)
    # Return the newest file (if any)
    return json_files[0] if json_files else None

def get_newest_loss(algo_name):
    # Construct the search pattern for the algorithm's csv files
    search_pattern = f"./d3rlpy_logs/{algo_name}_*/loss.csv"
    # Use glob to find all matching files
    loss_csv_files = glob.glob(search_pattern)
    # Sort the files by their modification time in descending order
    loss_csv_files.sort(key=os.path.getmtime, reverse=True)
    # Return the newest file (if any)
    return loss_csv_files[0] if loss_csv_files else None



def train_and_evaluate(df, dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='DQN', data_source='test', retrain=False):
    """
    Train or load a D3RLPY agent, then evaluate it.
    """
    # Device selection
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Prepare model directory
    model_dir = f"../models/training_log/{EXP_FOLDER_PREFIX}/models"
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, f"{algo}_{data_source}.pkl")


    # Create encoder factory with batch normalization and dropout
    encoder_factory = DefaultEncoderFactory(use_batch_norm=True, dropout_rate=0.5)
    # Create optimizer factory with Adam optimizer
    # optim_factory = OptimizerFactory(Adam)
    
    # Define algorithm mapping
    # [2025/06/09] Current used settings
    """ algo_map = {
        'DQN': (d3rlpy.algos.DQN, DQNConfig(learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048, encoder_factory=encoder_factory)),
        'DoubleDQN': (d3rlpy.algos.DoubleDQN, DoubleDQNConfig(learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048, encoder_factory=encoder_factory)),
        'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(actor_learning_rate=3e-4, critic_learning_rate=3e-4, batch_size=2048, gamma=0.98, actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory, target_update_interval=1000)), # [2025/05/21] current best
        'DecisionTransformer': (d3rlpy.algos.DiscreteDecisionTransformer, DiscreteDecisionTransformerConfig( context_size=1, batch_size=128, learning_rate=6e-4, max_timestep=int(df.groupby('stay_id')['hours_in'].max().max() + 1), num_heads=8, num_layers=6)), # context_size=obs_hrs, # TODO: Prior sequence length. 
        'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=128, learning_rate=1e-5, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory)),
        'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=1024, learning_rate=1e-5, gamma=0.75, target_update_interval=1000, encoder_factory=encoder_factory, alpha=0.1)), # TODO: this performance is good
        'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(batch_size=1024, learning_rate=1e-6, gamma=0.99, encoder_factory=encoder_factory, beta=0.1)),
        'NFQ': (d3rlpy.algos.NFQ, d3rlpy.algos.NFQConfig(batch_size=1024, learning_rate=1e-6, gamma=0.99, encoder_factory=encoder_factory)),
    } """
    # same parameter settings v1
    learning_rate = 1e-5
    # learning_rate = 1e-6
    gamma = 0.98
    target_update_interval = 1000
    batch_size = 1024 
    algo_map = {
        'DQN': (d3rlpy.algos.DQN, DQNConfig(learning_rate=learning_rate, gamma=gamma, target_update_interval=target_update_interval, batch_size=batch_size, encoder_factory=encoder_factory)),
        'DoubleDQN': (d3rlpy.algos.DoubleDQN, DoubleDQNConfig(learning_rate=learning_rate, gamma=gamma, target_update_interval=target_update_interval, batch_size=batch_size, encoder_factory=encoder_factory)),
        'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(actor_learning_rate=learning_rate, critic_learning_rate=learning_rate, batch_size=batch_size, gamma=gamma, actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory, target_update_interval=target_update_interval)), # [2025/05/21] current best
        'DecisionTransformer': (d3rlpy.algos.DiscreteDecisionTransformer, DiscreteDecisionTransformerConfig( context_size=1, batch_size=batch_size, learning_rate=learning_rate, max_timestep=int(df.groupby('stay_id')['hours_in'].max().max() + 1), num_heads=8, num_layers=6)), # context_size=obs_hrs, # TODO: Prior sequence length. 
        'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=batch_size, learning_rate=learning_rate, gamma=gamma, target_update_interval=target_update_interval, encoder_factory=encoder_factory)),
        'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=batch_size, learning_rate=learning_rate, gamma=gamma, target_update_interval=target_update_interval, encoder_factory=encoder_factory, alpha=0.1)), # TODO: this performance is good
        'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(batch_size=batch_size, learning_rate=learning_rate, gamma=gamma, encoder_factory=encoder_factory, beta=0.1)),
        'NFQ': (d3rlpy.algos.NFQ, d3rlpy.algos.NFQConfig(batch_size=batch_size, learning_rate=learning_rate, gamma=gamma, encoder_factory=encoder_factory)),
    }

    """ # D3RLPY default settings for baseline
    target_update_interval = 1000
    algo_map = {
        'DQN': (d3rlpy.algos.DQN, DQNConfig(target_update_interval=target_update_interval)),
        'DoubleDQN': (d3rlpy.algos.DoubleDQN, DoubleDQNConfig(target_update_interval=target_update_interval)),
        'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(target_update_interval=target_update_interval)), # [2025/05/21] current best
        # 'DecisionTransformer': (d3rlpy.algos.DiscreteDecisionTransformer, DiscreteDecisionTransformerConfig( context_size=1, batch_size=batch_size, learning_rate=learning_rate, max_timestep=int(df.groupby('stay_id')['hours_in'].max().max() + 1), num_heads=8, num_layers=6)), # context_size=obs_hrs, # TODO: Prior sequence length. 
        'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(target_update_interval=target_update_interval)),
        'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(target_update_interval=target_update_interval)), # TODO: this performance is good
        'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig()),
        'NFQ': (d3rlpy.algos.NFQ, d3rlpy.algos.NFQConfig()),
    } """

    """ algo_map = {
        # 'DQN': (d3rlpy.algos.DQN, DQNConfig(learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048)), # TODO: [update reward] add L2 regularization
        # 'DQN': (d3rlpy.algos.DQN, DQNConfig(gamma=0.99, target_update_interval=1000, batch_size=1024)), # TODO: [update reward] not sure target_update_interval
        # 'DQN': (d3rlpy.algos.DQN, DQNConfig(learning_rate=1e-3, gamma=0.99, target_update_interval=1000, batch_size=1024)), # TODO: [update reward] not sure target_update_interval
        # 'DQN': (d3rlpy.algos.DQN, DQNConfig( learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048, encoder_factory=encoder_factory, optim_factory=optim_factory)),
        'DQN': (d3rlpy.algos.DQN, DQNConfig(learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048, encoder_factory=encoder_factory)),
        # 'DoubleDQN': (d3rlpy.algos.DoubleDQN, DoubleDQNConfig(learning_rate=1e-3, gamma=0.99, target_update_interval=1000, batch_size=1024)),
        'DoubleDQN': (d3rlpy.algos.DoubleDQN, DoubleDQNConfig(learning_rate=1e-4, gamma=0.98, target_update_interval=1000, batch_size=2048, encoder_factory=encoder_factory)),
        # 'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(actor_learning_rate=3e-4, critic_learning_rate=3e-4, batch_size=1024, gamma=0.99)),
        'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(actor_learning_rate=3e-4, critic_learning_rate=3e-4, batch_size=2048, gamma=0.98, actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory, target_update_interval=1000)), # [2025/05/21] current best
        # 'DiscreteSAC': (d3rlpy.algos.DiscreteSAC, DiscreteSACConfig(actor_learning_rate=1e-5, critic_learning_rate=1e-5, batch_size=2048, gamma=0.98, actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory, target_update_interval=1000)),
        'DecisionTransformer': (d3rlpy.algos.DiscreteDecisionTransformer, DiscreteDecisionTransformerConfig(
                context_size=1,
                # context_size=obs_hrs, # TODO: Prior sequence length.
                batch_size=128,
                learning_rate=6e-4,
                max_timestep=int(df.groupby('stay_id')['hours_in'].max().max() + 1),
                num_heads=8,
                num_layers=6
            )
        ),
        # 'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(learning_rate=1e-4, batch_size=2048, gamma=0.98, encoder_factory=encoder_factory, target_update_interval=1000)),
        # 'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=2048, gamma=0.98)),
        # 'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=1024, gamma=0.99)),
        # 'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=32, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory)), # [2025/05/21] this is current best, but batch size 32 like not reasonable
        'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=128, learning_rate=1e-5, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory)),
        # 'DiscreteBCQ': (d3rlpy.algos.DiscreteBCQ, d3rlpy.algos.DiscreteBCQConfig(batch_size=1024, learning_rate=1e-5, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory)),
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(learning_rate=1e-4, batch_size=2048, gamma=0.98, encoder_factory=encoder_factory, target_update_interval=1000)),
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=2048, gamma=0.98)),
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=1024, gamma=0.99)),
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=128, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory)),
        # 'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(learning_rate=1e-4, batch_size=2048, gamma=0.98, encoder_factory=encoder_factory)),
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=128, gamma=0.99, target_update_interval=10000, encoder_factory=encoder_factory)), # TODO: seems like using larger target_update_interval (default 8000), before target network update the reward getting better
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=1024, learning_rate=1e-5, gamma=0.99, target_update_interval=1000, encoder_factory=encoder_factory, alpha=0.1)), # TODO: gamma = 0.75 is batter than 0.99
        'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=1024, learning_rate=1e-5, gamma=0.75, target_update_interval=1000, encoder_factory=encoder_factory, alpha=0.1)), # TODO: this performance is good
        # 'DiscreteCQL': (d3rlpy.algos.DiscreteCQL, d3rlpy.algos.DiscreteCQLConfig(batch_size=32, gamma=0.98, target_update_interval=1000, encoder_factory=encoder_factory, alpha=0.7)),
        'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(batch_size=1024, learning_rate=1e-6, gamma=0.99, encoder_factory=encoder_factory, beta=0.1)),
        # 'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(learning_rate=1e-4, batch_size=64, gamma=0.98, encoder_factory=encoder_factory)),
        # 'DiscreteBC': (d3rlpy.algos.DiscreteBC, d3rlpy.algos.DiscreteBCConfig(learning_rate=1e-4, batch_size=2048, gamma=0.98)),
        # 'NFQ': (d3rlpy.algos.NFQ, d3rlpy.algos.NFQConfig(batch_size=1024, gamma=0.99, encoder_factory=encoder_factory)),
        'NFQ': (d3rlpy.algos.NFQ, d3rlpy.algos.NFQConfig(batch_size=1024, learning_rate=1e-6, gamma=0.99, encoder_factory=encoder_factory)),
    } """

    if algo not in algo_map:
        raise ValueError(f"Unknown algorithm: {algo}")

    AlgClass, config = algo_map[algo]

    # Instantiate agent
    agent = AlgClass(config=config, device=device, enable_ddp=False)
    env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

    # Load existing or train new
    if not retrain and os.path.exists(model_path):
        print(f"Loading existing model from {model_path}")
        json_path = get_newest_json(algo)
        if json_path:
            if algo == 'DQN':
                agent = d3rlpy.algos.DQN.from_json(json_path)
            elif algo == 'DoubleDQN':
                agent = d3rlpy.algos.DoubleDQN.from_json(json_path)
            elif algo == 'DiscreteSAC':
                agent = d3rlpy.algos.DiscreteSAC.from_json(json_path)
            elif algo == 'DecisionTransformer':
                agent = d3rlpy.algos.DiscreteDecisionTransformer.from_json(json_path)
            elif algo == 'DiscreteBCQ':
                agent = d3rlpy.algos.DiscreteBCQ.from_json(json_path)
            elif algo == 'DiscreteCQL':
                agent = d3rlpy.algos.DiscreteCQL.from_json(json_path)
            elif algo == 'DiscreteBC':
                agent = d3rlpy.algos.DiscreteBC.from_json(json_path)
            elif algo == 'NFQ':
                agent = d3rlpy.algos.NFQ.from_json(json_path)
            else:
                raise ValueError(f"Unknown algorithm: {algo}")
            print(f"Loading model from {json_path}")
            agent.load_model(model_path)
    else:
        # Train agent
        # agent.fit(dataset, n_steps=10, n_steps_per_epoch=5)
        # agent.fit(dataset, n_steps=1000000, n_steps_per_epoch=100000)
        # agent.fit(dataset, n_steps=50000, n_steps_per_epoch=10000)
        # agent.fit(dataset, n_steps=10000, n_steps_per_epoch=1000)
        # agent.fit(dataset, n_steps=100000, n_steps_per_epoch=10000)
        # n_steps = 10000 # TODO: increase # TODO: [update reward] not sure
        """ n_steps = 20 # TODO: increase # TODO: [update reward] not sure
        n_steps_per_epoch = 10 """
        n_steps = 20000 # TODO: increase # TODO: [update reward] not sure
        n_steps_per_epoch = 1000
        
        cur_best_reward = -np.inf
        rewards_summary = []
        action_diversity_summary = []
        for epoch in range(int(n_steps/n_steps_per_epoch)):
            print(f"Epoch {epoch+1}/{n_steps/n_steps_per_epoch}")
            agent.fit(dataset, n_steps=n_steps_per_epoch, n_steps_per_epoch=n_steps_per_epoch)
            # evaluate the agent
            action_diversity_set = set()
            evaluation_trajectories = []  # List to store all trajectory steps

            for category, init_obs_list in eval_init_obs_categories.items():
                if not init_obs_list:
                    print(f"Skipping {category}: No valid initial observations")
                    continue

                # Full evaluation for reward statistics and trajectories
                episode_returns = []
                traj_len = []
                traj_len_without_nonsense_actions = []
                nonsense_actions = []
                meet_weaning_list = []
                meet_weaning_traj_len_list = []
                action_diversity_in_category_set = set()
                for idx in range(min(100, len(init_obs_list))): # TODO: set back to 100
                # for idx in range(min(100, len(init_obs_list))):
                    obs = env.reset(init_obs_list[idx])
                    done = False
                    total_reward = 0
                    trajectory = []  # Trajectory for this episode
                    stay_id = f'{epoch}_{category}_{idx}'
                    # stay_id = f'eval_{algo}_{data_source}_epoch{epoch}_{category}_{idx}'

                    # Extract sizes
                    state_size = len(env.state_cols)  # e.g., 3 for heart_rate, resp_rate, spo2
                    action_size = len(env.action_cols)  # e.g., 2 for fio2, respiratory_rate_set
                    baseline_size = len(env.baseline_cols)  # e.g., 2 for gender_M, age
                    obs_hrs = env.obs_hrs  # Assuming env has obs_hrs attribute

                    # At t=0, include historical states and current state from initial obs
                    for h in range(obs_hrs):  # h=0 for s_1, h=1 for s_2, ..., h=obs_hrs-1 for s_current
                        state_start = h * (state_size + action_size)
                        state = obs[state_start:state_start + state_size]  # Extract state
                        # Action: use a_h for s_h, last historical action for s_current # TODO: this is not correct
                        if h < obs_hrs - 1:
                            action_start = state_start + state_size
                            action = obs[action_start:action_start + action_size]
                        else:
                            action_idx = agent.predict(np.expand_dims(obs, axis=0))[0]  # Predict action for s_current
                            action = env.id_to_action[action_idx]  # Assuming this maps action index to tuple
                            # q_value = agent.predict_value(np.expand_dims(obs, axis=0), action_idx) # TODO: for tracking q_value
                            # print(f"action: {action} q_value: {q_value}")
                        # Baseline variables
                        baseline_start = len(obs) - baseline_size
                        baseline = obs[baseline_start:baseline_start + baseline_size]

                        trajectory.append({
                            'stay_id': stay_id,
                            'hours_in': h,
                            **{col: state[i] for i, col in enumerate(env.state_cols)},
                            **{col: action[i] for i, col in enumerate(env.action_cols)},
                            **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
                        })

                    # Continue for remaining steps
                    t = obs_hrs
                    while not done:
                        action_idx = agent.predict(np.expand_dims(obs, axis=0))[0] # TODO: or use "sample_action" -> since some mtehod will be not deterministic
                        action_tuple = env.id_to_action[action_idx]  # Assuming this maps action index to tuple
                        action_diversity_set.add(action_idx)
                        action_diversity_in_category_set.add(action_idx)
                        next_obs, reward, done, info = env.step(action_idx)
                        total_reward += reward

                        # Extract current state and baseline from next_obs
                        state_start = (obs_hrs - 1) * (state_size + action_size)
                        state = next_obs[state_start:state_start + state_size]  # s_current
                        baseline_start = len(next_obs) - baseline_size
                        baseline = next_obs[baseline_start:baseline_start + baseline_size]

                        # Store trajectory
                        trajectory.append({
                            'stay_id': stay_id,
                            'hours_in': t,
                            **{col: state[i] for i, col in enumerate(env.state_cols)},
                            **{col: action_tuple[i] for i, col in enumerate(env.action_cols)},
                            **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
                        })

                        obs = next_obs
                        t += 1

                    episode_returns.append(total_reward)
                    evaluation_trajectories.extend(trajectory)  # Add this episode's steps to the full list
                    traj_len.append(len(trajectory))
                    nonsense_actions.append(info['nonsense_action'])
                    if not info['nonsense_action']:
                        traj_len_without_nonsense_actions.append(len(trajectory))
                    meet_weaning_list.append(info['meet_weaning'])
                    if info['meet_weaning']:
                        meet_weaning_traj_len_list.append(len(trajectory))

                if len(episode_returns) == 0:
                    print(f"Skipping {category}: No valid episode returns")
                    continue

                mean_reward = np.mean(episode_returns)
                std_reward = np.std(episode_returns)
                rewards_summary.append({
                    'algo': algo,
                    'data_source': data_source,
                    'epoch': epoch,
                    'category': category,
                    'mean_reward': mean_reward,
                    'std_reward': std_reward,
                    'meet_weaning_percentage': np.mean(meet_weaning_list) if meet_weaning_list else 0,
                    'mean_traj_len': np.mean(traj_len),
                    'std_traj_len': np.std(traj_len),
                    'mean_traj_len_without_nonsense_actions': np.mean(traj_len_without_nonsense_actions) if traj_len_without_nonsense_actions else 0,
                    'std_traj_len_without_nonsense_actions': np.std(traj_len_without_nonsense_actions) if traj_len_without_nonsense_actions else 0,
                    'mean_traj_len_to_meet_weaning': np.mean(meet_weaning_traj_len_list) if meet_weaning_traj_len_list else 0,
                    'std_traj_len_to_meet_weaning': np.std(meet_weaning_traj_len_list) if meet_weaning_traj_len_list else 0,
                    'action_diversity': len(action_diversity_in_category_set), # TODO: action_diversity_set will be "cumulative" across all categories
                    'mean_nonsense_actions': np.mean(nonsense_actions) # not sure mean of boolean
                })
                print(f"{algo} {data_source} - {category}: Average Total Reward = {mean_reward}")
                # print(f"Mean trajectory length: {np.mean(traj_len)}")
                # print(f"Mean trajectory length without nonsense actions: {np.mean(traj_len_without_nonsense_actions)}")
                # print(f"Mean nonsense actions: {np.mean(nonsense_actions)}")

                if category == 'small_reward' and mean_reward > cur_best_reward:
                    cur_best_reward = mean_reward
                    print(f"Saving model to {model_path}")
                    agent.save_model(model_path)

            # Record action diversity for the epoch
            action_diversity = len(action_diversity_set)
            action_diversity_summary.append({
                'algo': algo,
                'data_source': data_source,
                'epoch': epoch,
                'action_diversity': action_diversity
            })
            print(f"Action diversity: {action_diversity}")
            print(f"Actions taken: {action_diversity_set}")

            # Save all evaluation trajectories to a CSV file
            df_trajectories = pd.DataFrame(evaluation_trajectories)
            df_trajectories.to_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_{data_source}_epoch{epoch}.csv', index=False)
            print(f"Saved evaluation trajectories to 'eval_trajectories_{algo}_{data_source}_epoch{epoch}.csv'")
            # Save rewards summary to a CSV file
            df_rewards = pd.DataFrame(rewards_summary)
            df_rewards.to_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv', index=False)
            # Save action diversity summary to a CSV file
            df_action_diversity = pd.DataFrame(action_diversity_summary)
            # TODO: can merge rewards_summary and action_diversity_summary
            df_action_diversity.to_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/action_diversity_summary_{algo}_{data_source}.csv', index=False)
            # save model
            # agent.save_model(f"../models/{algo}_{data_source}_epoch{epoch}.pkl") # TODO: save if needed
            """ if mean_reward > cur_best_reward: # TODO: only use common reward for finding the best model, since it is the last one to be eval
                cur_best_reward = mean_reward
                print(f"Saving model to {model_path}")
                agent.save_model(model_path) """
        # # Save model
        # print(f"Saving model to {model_path}")
        # agent.save_model(model_path)

    return agent, env


### Preprare data set for train_df, test_df, and eICU_disc for train d3rlpy agent

In [ ]:
train_df_dataset = preprocess_data(train_df, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)
test_df_dataset = preprocess_data(test_df, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)
eICU_disc_dataset = preprocess_data(eICU_disc, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

## Physician Policy (Naive Agent action transitions)

In [ ]:
from collections import defaultdict
import random
import itertools
import numpy as np

class NaiveAgent:
    """
    A naive agent that interacts with the PatientEnvironment.
    It uses a dictionary to map state-action-baseline keys to a list of possible actions.
    """
    def __init__(self, df, obs_hrs, state_cols, action_cols, baseline_cols=None):
        """
        Initialize the naive agent by creating its internal dictionary.

        Args:
            df (pd.DataFrame): Input dataframe containing the data.
            obs_hrs (int): Number of observation hours to include in the state history.
            state_cols (list): List of columns to include in the state.
            action_cols (list): List of columns to include in the action.
            baseline_cols (list, optional): List of baseline columns (e.g., age, gender) to include. Defaults to None.
        """
        self.naive_agent_dict = self._create_naive_agent_dict(df, obs_hrs, state_cols, action_cols, baseline_cols)

    def _create_naive_agent_dict(self, df, obs_hrs, state_cols, action_cols, baseline_cols=None):
        """
        Create the naive agent dictionary.

        Args:
            df (pd.DataFrame): Input dataframe containing the data.
            obs_hrs (int): Number of observation hours to include in the state history.
            state_cols (list): List of columns to include in the state.
            action_cols (list): List of columns to include in the action.
            baseline_cols (list, optional): List of baseline columns (e.g., age, gender) to include. Defaults to None.

        Returns:
            dict: A dictionary where keys are tuples of (state, action) history and baseline,
                  and values are lists of possible actions.
        """
        naive_agent = defaultdict(list)
        
        # Group by stay_id
        for stay_id, group in df.groupby('stay_id'):
            group = group.sort_values('hours_in')
            rows = group.to_dict('records')
            
            # Sliding window with length obs_hrs+1 to get (state/action history, current_action)
            for i in range(len(rows) - obs_hrs):
                window = rows[i:i+obs_hrs+1]
                
                # Build state_with_pre_state_action_baseline: for each of the first obs_hrs rows
                key = []
                for j in range(obs_hrs):
                    state = tuple(window[j][col] for col in state_cols)  # Extract state variables
                    if j < obs_hrs - 1:
                        action = tuple(window[j][col] for col in action_cols)  # Extract action variables
                        # key.append((state, action))
                        key.append(state)
                        key.append(action)
                    else:
                        # key.append((state))
                        key.append(state)
                
                # Add baseline variables (if provided) to the key
                if baseline_cols:
                    baseline = tuple(window[0][col] for col in baseline_cols)  # Use the first row for baseline variables
                    key.append(baseline)

                # The current action for the last row
                curr_action = tuple(window[obs_hrs][col] for col in action_cols)
                
                # Store action: key -> action_list
                # naive_agent[tuple(key)].append(curr_action)
                naive_agent[tuple(itertools.chain.from_iterable(key))].append(curr_action)
                
                
        
        return naive_agent

    def sample_action(self, state_with_pre_state_action_baseline, last_action):
        """
        Sample an action from the naive agent dictionary for a given key.
        If the key does not exist, return the last action.

        Args:
            state_with_pre_state_action_baseline (tuple): The state-action-baseline key.
            last_action (tuple): The last action to return if the key does not exist.

        Returns:
            tuple: A sampled action or the last action if the key does not exist.
        """
        if tuple(state_with_pre_state_action_baseline) in self.naive_agent_dict:
            return random.choice(self.naive_agent_dict[state_with_pre_state_action_baseline])
        return last_action
    
    def predict(self, state_with_pre_state_action_baseline):
        action_bins = [num_bins_dict[col] for col in action_cols]
        action_to_id = {a: i for i, a in enumerate(itertools.product(*[range(b) for b in action_bins]))}
        
        state_with_pre_state_action_baseline = tuple(state_with_pre_state_action_baseline[0])
        if tuple(state_with_pre_state_action_baseline) in self.naive_agent_dict:
            return [action_to_id[random.choice(self.naive_agent_dict[state_with_pre_state_action_baseline])]]
        else:
            state_len = len(state_cols)
            action_len = len(action_cols)
            baseline_len = len(baseline_cols)

            # Split the data into states, actions, and baselines
            num_states = (len(state_with_pre_state_action_baseline) - baseline_len) // (state_len + action_len)
            states = []
            actions = []
            for i in range(num_states):
                start_idx = i * (state_len + action_len)
                states.append(tuple(state_with_pre_state_action_baseline[start_idx:start_idx + state_len]))
                actions.append(tuple(state_with_pre_state_action_baseline[start_idx + state_len:start_idx + state_len + action_len]))
            # last state is the current state
            current_state = tuple(state_with_pre_state_action_baseline[-(state_len + baseline_len):-baseline_len])
            states.append(current_state)

            # Create the final format
            result = []
            for i in range(num_states + 1):
                # Use the last action if the current action is missing
                if i < len(actions):
                    action = actions[i]
                else:
                    last_action = actions[-1]
                    # Convert np.int64 (or any NumPy integer) to plain Python integers
                    last_action = tuple(int(x) if isinstance(x, np.integer) else x for x in last_action)
                    # print("last_action")
                    return [action_to_id[last_action]]

In [ ]:
naive_agent = NaiveAgent(train_df, obs_hrs, state_cols, action_cols, baseline_cols)
# naive_agent = NaiveAgent(test_df, obs_hrs, state_cols, action_cols, baseline_cols)
# naive_agent = NaiveAgent(eICU_disc, obs_hrs, state_cols, action_cols, baseline_cols)

In [ ]:
naive_agent.naive_agent_dict[tuple([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7])]

In [ ]:
# naive_agent.predict(tuple(tuple([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7])))
# naive_agent.predict(np.expand_dims([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7], axis=0))
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)
obs = env.reset()
print(obs)
naive_agent.predict(np.expand_dims(obs, axis=0))
# naive_agent.predict(tuple(env.reset()))

## D3rlpy Offline Policy Reinforcement Learning

### BC

In [ ]:
agent_bc_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBC', data_source='train', retrain=True)


In [ ]:
agent_bc_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBC', data_source='train', retrain=True)


In [ ]:
agent_bc_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBC', data_source='train')
agent_bc_test, env = train_and_evaluate(test_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBC', data_source='test')
agent_bc_eICU, env = train_and_evaluate(eICU_disc, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBC', data_source='eICU')

### NFQ

In [ ]:
agent_nfq_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='NFQ', data_source='train')
agent_nfq_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='NFQ', data_source='test')
agent_nfq_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='NFQ', data_source='eICU')

### DQN

In [ ]:
agent_dqn_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DQN', data_source='train')
agent_dqn_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DQN', data_source='test')
agent_dqn_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DQN', data_source='eICU')

### Double DQN

In [ ]:
agent_ddqn_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DoubleDQN', data_source='train')
agent_ddqn_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DoubleDQN', data_source='test')
agent_ddqn_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DoubleDQN', data_source='eICU')

### DiscreteSAC

In [ ]:
agent_sac_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteSAC', data_source='train')
agent_sac_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteSAC', data_source='test')
agent_sac_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteSAC', data_source='eICU')

### BCQ


In [ ]:
agent_bcq_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBCQ', data_source='train')
agent_bcq_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBCQ', data_source='test')
agent_bcq_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteBCQ', data_source='eICU')

### CQL

In [ ]:
agent_cql_train, env = train_and_evaluate(train_df, train_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteCQL', data_source='train')
agent_cql_test, env = train_and_evaluate(test_df, test_df_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteCQL', data_source='test')
agent_cql_eICU, env = train_and_evaluate(eICU_disc, eICU_disc_dataset, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, algo='DiscreteCQL', data_source='eICU')

## Offline Policy Evaluation (FQE)

In [ ]:
import numpy as np
import pandas as pd
import torch
import d3rlpy
from d3rlpy.algos import DQNConfig, DoubleDQNConfig, DiscreteSACConfig, DiscreteBCQConfig, DiscreteCQLConfig
from d3rlpy.ope import DiscreteFQE

def load_agent(algo, data_source, dataset):
    """
    Load a trained agent from a saved model file.
    
    Args:
        algo (str): Algorithm name (e.g., 'DQN', 'DoubleDQN').
        data_source (str): Data source the agent was trained on ('train', 'test', 'eICU').
        dataset (MDPDataset): Dataset to build the agent's model structure.
    
    Returns:
        Agent object loaded with the saved model.
    """
    model_path = f"../models/{algo}_{data_source}.pkl"
    
    # Define configurations matching those in train_and_evaluate
    algo_configs = {
        'DQN': DQNConfig(learning_rate=1e-3, gamma=0.99, target_update_interval=1000, batch_size=32),
        'DoubleDQN': DoubleDQNConfig(learning_rate=1e-3, gamma=0.99, target_update_interval=1000, batch_size=32),
        'DiscreteSAC': DiscreteSACConfig(actor_learning_rate=3e-4, critic_learning_rate=3e-4, batch_size=32, gamma=0.99),
        'DiscreteBCQ': DiscreteBCQConfig(batch_size=32, gamma=0.99),
        'DiscreteBC': DiscreteBCConfig(batch_size=32, learning_rate=1e-3),
        'DiscreteCQL': DiscreteCQLConfig(batch_size=32, gamma=0.99)
    }
    
    if algo not in algo_configs:
        raise ValueError(f"Unknown algorithm: {algo}")
    
    config = algo_configs[algo]
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Instantiate the agent
    if algo == 'DQN':
        agent = d3rlpy.algos.DQN(config=config, device=device, enable_ddp=False)
    elif algo == 'DoubleDQN':
        agent = d3rlpy.algos.DoubleDQN(config=config, device=device, enable_ddp=False)
    elif algo == 'DiscreteSAC':
        agent = d3rlpy.algos.DiscreteSAC(config=config, device=device, enable_ddp=False)
    elif algo == 'DiscreteBCQ':
        agent = d3rlpy.algos.DiscreteBCQ(config=config, device=device, enable_ddp=False)
    elif algo == 'DiscreteCQL':
        agent = d3rlpy.algos.DiscreteCQL(config=config, device=device, enable_ddp=False)
    elif algo == 'DiscreteBC':
        agent = d3rlpy.algos.DiscreteBC(config=config, device=device, enable_ddp=False)
    
    # Build with dataset to set observation and action spaces
    agent.build_with_dataset(dataset)
    
    # Load the saved model
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    agent.load_model(model_path)
    print(f"Loaded {algo} agent trained on {data_source} from {model_path}")
    
    return agent

def get_terminals_from_dataset(dataset):
    terminals = []
    for episode in dataset.episodes:
        for transition in episode.transitions:
            # Assume transition is a Transition object with a terminal attribute
            terminals.append(getattr(transition, 'terminal', False))
    return np.array(terminals, dtype=bool)

# def evaluate_with_fqe(agent, dataset, fqe_config, n_steps=10000):
#     """
#     Evaluate an agent on a dataset using DiscreteFQE.
    
#     Args:
#         agent: Trained agent with a predict method.
#         dataset (MDPDataset): Dataset to evaluate on.
#         fqe_config: Configuration for the FQE Q-function.
#         n_steps (int): Number of training steps for FQE.
    
#     Returns:
#         float: Estimated policy value.
#     """
#     # Initialize FQE
#     fqe = DiscreteFQE(algo=agent, config=fqe_config)
    
#     # Fit FQE on the dataset
#     fqe.fit(dataset, n_steps=n_steps)
    
#     # Identify initial observations (start of episodes)
#     initial_indices = np.where(dataset.terminals[:-1] == 1)[0] + 1
#     if len(initial_indices) == 0:
#         initial_indices = [0]
#     else:
#         initial_indices = np.concatenate(([0], initial_indices))
#     initial_obs = dataset.observations[initial_indices]
    
#     # Get actions chosen by the agent's policy
#     actions = agent.predict(initial_obs)
    
#     # Predict Q-values and select those for the chosen actions
#     q_values = fqe.predict(initial_obs)  # Shape: (n_samples, n_actions)
#     selected_q_values = q_values[np.arange(len(actions)), actions]
    
#     # Compute average Q-value as the estimated policy value
#     estimated_value = selected_q_values.mean()
    
#     return estimated_value
def evaluate_with_fqe(agent, dataset, eval_init_obs_categories, fqe_config, n_steps=10000):
    """
    Evaluate an agent on a dataset using DiscreteFQE.
    
    Args:
        agent: Trained agent with a predict method.
        dataset (MDPDataset): Dataset to evaluate on.
        episode_starts (list): List of indices indicating the start of each episode.
        fqe_config: Configuration for the FQE Q-function.
        n_steps (int): Number of training steps for FQE.
    
    Returns:
        float: Estimated policy value.
    """
    # Initialize FQE
    fqe = DiscreteFQE(algo=agent, config=fqe_config)
    
    # Fit FQE on the dataset
    fqe.fit(dataset, n_steps=n_steps)
    
    # obs = env.reset()
    # q_value = fqe.predict(obs)
    # # Identify initial observations using episode_starts
    # initial_obs = dataset.observations[episode_starts]
    
    # # Get actions chosen by the agent's policy
    # actions = agent.predict(initial_obs)
    
    # # Predict Q-values and select those for the chosen actions
    # q_values = fqe.predict(initial_obs)  # Shape: (n_samples, n_actions)
    # selected_q_values = q_values[np.arange(len(actions)), actions]
    
    # # Compute average Q-value as the estimated policy value
    # estimated_value = selected_q_values.mean()
    
    return fqe

In [ ]:
# Datasets (assumed to be predefined MDPDataset objects)
# datasets = {
#     'train': (train_df_dataset, train_episode_starts),
#     'test': (test_df_dataset, test_episode_starts),
#     'eICU': (eICU_disc_dataset, eICU_episode_starts)
# }
datasets = {
    'train': train_df_dataset,
    'test': test_df_dataset,
    'eICU': eICU_disc_dataset
}

# FQE Configuration
fqe_config = DQNConfig(
    learning_rate=1e-3,
    gamma=0.99,
    target_update_interval=1000,
    batch_size=32
)

# Algorithms and data sources
algorithms = ['DQN', 'DoubleDQN', 'DiscreteSAC', 'DiscreteBCQ', 'DiscreteCQL', 'DiscreteBC']
data_sources = ['train', 'test', 'eICU']

In [ ]:
import os

# Collect results
results = []

for algo in algorithms:
    for train_data_source in data_sources:
        # Load the agent trained on train_data_source
        try:
            agent = load_agent(algo, train_data_source, train_df_dataset) # TODO: train_df_dataset -> it's only for building the model structure
        except FileNotFoundError as e:
            print(e)
            continue
        
        # Evaluate on each dataset
        for eval_data_source in data_sources:
            dataset = datasets[eval_data_source]
            # dataset, episode_starts = datasets[eval_data_source]
            # estimated_value = evaluate_with_fqe(agent, dataset, episode_starts, fqe_config, n_steps=10000)
            # fqe_agent = evaluate_with_fqe(agent, dataset, episode_starts, fqe_config, n_steps=10000)
            fqe_agent = DiscreteFQE(algo=agent, config=fqe_config)
            # fqe_agent.fit(dataset, n_steps=100, n_steps_per_epoch=10)
            fqe_agent.fit(dataset, n_steps=10000, n_steps_per_epoch=1000)
            # estimated_value = fqe_agent.predict(env.reset())
            print(f"Estimated policy value for {eval_data_source}:")
            q_value_dict = {}
            action_diversity = set()
            for category, init_obs_list in eval_init_obs_categories.items():
                q_value_list = []
                for obs in init_obs_list:
                    action = fqe_agent.predict(np.expand_dims(obs, axis=0))
                    action_diversity.add(tuple(action))
                    q_value = fqe_agent.predict_value(np.expand_dims(obs, axis=0), action)
                    q_value_list.append(q_value)
                q_value_mean = np.mean(q_value_list)
                print(f"mean Q of {category}: {q_value_mean}")
                q_value_dict[category] = q_value_mean
            # print(f"Estimated policy value for {eval_data_source}: {estimated_value}")
            results.append({
                'algo': algo,
                'train_data_source': train_data_source,
                'eval_data_source': eval_data_source,
                'q_value_dict': q_value_dict,
                'action_diversity': len(action_diversity)
            })
            print(f"{algo} trained on {train_data_source}, evaluated on {eval_data_source}, action diversity {len(action_diversity)}: {q_value_dict}")
            # save fqe model
            fqe_agent.save_model(f"../models/fqe_{algo}_{train_data_source}_{eval_data_source}.pkl")
            print(f"FQE model saved to '../models/fqe_{algo}_{train_data_source}_{eval_data_source}.pkl'")

# Convert to DataFrame and save
df_results = pd.DataFrame(results)
df_results.to_csv('../models/fqe_evaluation_results.csv', index=False)
print("Results saved to 'fqe_evaluation_results.csv'")

### visualize fqe results

In [ ]:
fqe_evaluation_results_df = pd.read_csv('../models/fqe_evaluation_results.csv')
fqe_evaluation_results_df

In [ ]:
fqe_evaluation_results_df["q_value_dict"]

In [ ]:
fqe_evaluation_results_df = pd.read_csv('../models/fqe_evaluation_results.csv')


In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
fqe_evaluation_results_df = pd.read_csv('../models/fqe_evaluation_results.csv')

# Convert 'q_value_dict' from string to dictionary
fqe_evaluation_results_df['q_value_dict'] = fqe_evaluation_results_df['q_value_dict'].apply(lambda x: ast.literal_eval(x))

# Define the reward categories
reward_categories = ['small_reward', 'median_reward', 'large_reward', 'long_to_wean', 'median_to_wean', 'short_to_wean', 'common']

# Aggregate Q-values by algorithm and evaluation data source
def aggregate_q_values(df):
    aggregated_data = {}
    for algo in df['algo'].unique():
        for eval_data in df['eval_data_source'].unique():
            subset = df[(df['algo'] == algo) & (df['eval_data_source'] == eval_data)]
            q_values = {reward: np.mean([entry[reward] for entry in subset['q_value_dict']]) for reward in reward_categories}
            aggregated_data[(algo, eval_data)] = q_values
    return aggregated_data

aggregated_q_values = aggregate_q_values(fqe_evaluation_results_df)

# Convert to DataFrame for heatmap plotting
heatmap_data = pd.DataFrame(aggregated_q_values).T
heatmap_data.index = pd.MultiIndex.from_tuples(heatmap_data.index, names=['Algorithm', 'Eval Data'])

# Plot the heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, cmap='coolwarm', annot=True, fmt='.2f', cbar=True)
plt.title('Q-Value Heatmap by Algorithm and Eval Data Source')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Load the CSV

# 2) Parse the q_value_dict column into actual dicts
def parse_q_dict(s):
    # safely evaluate the string representation of the dict
    return {k: float(v) for k, v in ast.literal_eval(s).items()}

fqe_evaluation_results_df['q_dict'] = fqe_evaluation_results_df['q_value_dict'].apply(parse_q_dict)

# 3) Unpack into long form
rows = []
for _, row in fqe_evaluation_results_df.iterrows():
    algo = row['algo']
    train_src = row['train_data_source']
    eval_src = row['eval_data_source']
    diversity = float(row['action_diversity'])
    qdict = row['q_dict']
    for metric_name, value in qdict.items():
        rows.append({
            'Algo': algo,
            'Train Source': train_src,
            'Eval Source': eval_src,
            'Metric': metric_name,
            'Value': value
        })
    # add action_diversity as its own metric
    rows.append({
        'Algo': algo,
        'Train Source': train_src,
        'Eval Source': eval_src,
        'Metric': 'action_diversity',
        'Value': diversity
    })

long_fqe_evaluation_results_df = pd.DataFrame(rows)

# 4) Decide which metrics to plot and their order
metrics_to_plot = [
    'small_reward', 'median_reward', 'large_reward',
    'short_to_wean', 'median_to_wean', 'long_to_wean',
    'action_diversity'
]

# 5) Plot one heatmap per metric, comparing Train vs Eval for each algorithm
for algo in long_fqe_evaluation_results_df['Algo'].unique():
    algo_fqe_evaluation_results_df = long_fqe_evaluation_results_df[long_fqe_evaluation_results_df['Algo'] == algo]
    
    fig, axes = plt.subplots(len(metrics_to_plot), 1, figsize=(12, 4 * len(metrics_to_plot)))
    fig.suptitle(f'{algo}: FQE Evaluation Metrics\n', fontsize=18)
    
    for i, metric in enumerate(metrics_to_plot):
        sub = algo_fqe_evaluation_results_df[algo_fqe_evaluation_results_df['Metric'] == metric]
        pivot = sub.pivot_table(
            index='Train Source',
            columns='Eval Source',
            values='Value'
        ).reindex(index=['train', 'test', 'eICU'], columns=['train', 'test', 'eICU'])
        
        # pick colormap
        if 'reward' in metric:
            cmap = 'RdYlGn'
        elif 'to_wean' in metric:
            cmap = 'RdYlGn_r'
        else:
            cmap = 'viridis'
        
        ax = axes[i]
        sns.heatmap(pivot, annot=True, fmt=".2f", cmap=cmap,
                    linewidths=0.5, cbar=i == 0, ax=ax)
        ax.set_title(metric, fontsize=14)
        ax.set_ylabel('Train on')
        ax.set_xlabel('Eval on')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
results

In [ ]:
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)


In [ ]:
eval_init_obs_categories["small_reward"][0]

In [ ]:
obs = env.reset()
obs

In [ ]:
q_value_list = []
for category, init_obs_list in eval_init_obs_categories.items():
    for obs in init_obs_list:
        action = fqe_agent.predict(np.expand_dims(obs, axis=0))
        q_value = fqe_agent.predict_value(np.expand_dims(obs, axis=0), action)
        q_value_list.append(q_value)
    print(f"mean Q of {category}: {np.mean(q_value_list)}")


In [ ]:
fix_obs = env.reset()
fix_obs

In [ ]:
fix_obs = eval_init_obs_categories["small_reward"][0]


In [ ]:
action = fqe_agent.predict(np.expand_dims(obs, axis=0))
action

In [ ]:
action = fqe_agent.predict(np.expand_dims(env.reset(), axis=0))
action

In [ ]:
fqe_agent.predict_value(np.expand_dims(obs, axis=0), action)

In [ ]:
print(f"obs: {fix_obs}")
for i in range(30):
    action = fqe_agent.predict(np.expand_dims(env.reset(), axis=0))
    value = fqe_agent.predict_value(np.expand_dims(fix_obs, axis=0), action)
    print(f"action: {env.id_to_action[action[0]]}, q_value: {value}")

In [ ]:
print(f"obs: {fix_obs}")
for i in range(5):
    action = fqe_agent.sample_action(np.expand_dims(fix_obs, axis=0))
    # action = fqe_agent.predict(np.expand_dims(fix_obs, axis=0))
    value = fqe_agent.predict_value(np.expand_dims(fix_obs, axis=0), action)
    print(f"action: {env.id_to_action[action[0]]}, q_value: {value}")

In [ ]:
# print(f"obs: {fix_obs}")
for category, init_obs_list in eval_init_obs_categories.items():
    for i in range(5):
        print(f"obs: {init_obs_list[i]}")
        action = fqe_agent.predict(np.expand_dims(init_obs_list[i], axis=0))
        # action = fqe_agent.predict(np.expand_dims(fix_obs, axis=0))
        value = fqe_agent.predict_value(np.expand_dims(fix_obs, axis=0), action)
        print(f"category: {category} action: {env.id_to_action[action[0]]}, q_value: {value}")

## Naive Agent (action transitions)

In [ ]:
naive_agent = NaiveAgent(train_df, obs_hrs, state_cols, action_cols, baseline_cols)

In [ ]:
naive_agent.naive_agent_dict[tuple([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7])]
# naive_agent.naive_agent_dict[tuple(env.reset())]

In [ ]:
# naive_agent.predict(tuple(tuple([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7])))
# naive_agent.predict(np.expand_dims([5, 7, 5, 5, 6, 5, 7, 5, 5, 6, 5, 7, 5, 0, 7], axis=0))
obs = env.reset()
print(obs)
naive_agent.predict(np.expand_dims(obs, axis=0))
# naive_agent.predict(tuple(env.reset()))

### Evaluate Naive Agent on Patient Environment with Random init

In [ ]:
def naive_agent_evaluate_with_random(df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='DQN', data_source='test', retrain=False):
    """
    Train or load a D3RLPY agent, then evaluate it.
    """
    agent = NaiveAgent(df, obs_hrs, state_cols, action_cols, baseline_cols)
    env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)
    cur_best_reward = -np.inf
    rewards_summary = []
    action_diversity_summary = []
    action_diversity_set = set()
    evaluation_trajectories = []  # List to store all trajectory steps

    # for category, init_obs_list in eval_init_obs_categories.items():
    #     if not init_obs_list:
    #         print(f"Skipping {category}: No valid initial observations")
    #         continue

    # Full evaluation for reward statistics and trajectories
    episode_returns = []
    # for idx in range(min(100, len(init_obs_list))):
    for idx in range(int(df["stay_id"].nunique())):
        # obs = env.reset(init_obs_list[idx])
        obs = env.reset()
        done = False
        total_reward = 0
        trajectory = []  # Trajectory for this episode
        stay_id = f'{category}_{idx}'
        # stay_id = f'eval_{algo}_{data_source}_epoch{epoch}_{category}_{idx}'

        # Extract sizes
        state_size = len(env.state_cols)  # e.g., 3 for heart_rate, resp_rate, spo2
        action_size = len(env.action_cols)  # e.g., 2 for fio2, respiratory_rate_set
        baseline_size = len(env.baseline_cols)  # e.g., 2 for gender_M, age
        obs_hrs = env.obs_hrs  # Assuming env has obs_hrs attribute

        # At t=0, include historical states and current state from initial obs
        for h in range(obs_hrs):  # h=0 for s_1, h=1 for s_2, ..., h=obs_hrs-1 for s_current
            state_start = h * (state_size + action_size)
            state = obs[state_start:state_start + state_size]  # Extract state
            # Action: use a_h for s_h, last historical action for s_current
            if h < obs_hrs - 1:
                action_start = state_start + state_size
                action = obs[action_start:action_start + action_size]
            else:
                # For s_current, use the last historical action (a_(obs_hrs-1))
                last_action_start = (obs_hrs - 1) * (state_size + action_size) - action_size
                action = obs[last_action_start:last_action_start + action_size]
            # Baseline variables
            baseline_start = len(obs) - baseline_size
            baseline = obs[baseline_start:baseline_start + baseline_size]

            trajectory.append({
                'stay_id': stay_id,
                'hours_in': h,
                **{col: state[i] for i, col in enumerate(env.state_cols)},
                **{col: action[i] for i, col in enumerate(env.action_cols)},
                **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
            })

        # Continue for remaining steps
        t = obs_hrs
        while not done:
            action_idx = agent.predict(np.expand_dims(obs, axis=0))[0]
            action_tuple = env.id_to_action[action_idx]  # Assuming this maps action index to tuple
            action_diversity_set.add(action_idx)
            next_obs, reward, done, _ = env.step(action_idx)
            total_reward += reward

            # Extract current state and baseline from next_obs
            state_start = (obs_hrs - 1) * (state_size + action_size)
            state = next_obs[state_start:state_start + state_size]  # s_current
            baseline_start = len(next_obs) - baseline_size
            baseline = next_obs[baseline_start:baseline_start + baseline_size]

            # Store trajectory
            trajectory.append({
                'stay_id': stay_id,
                'hours_in': t,
                **{col: state[i] for i, col in enumerate(env.state_cols)},
                **{col: action_tuple[i] for i, col in enumerate(env.action_cols)},
                **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
            })

            obs = next_obs
            t += 1

        episode_returns.append(total_reward)
        evaluation_trajectories.extend(trajectory)  # Add this episode's steps to the full list

    # if len(episode_returns) == 0:
    #     print(f"Skipping {category}: No valid episode returns")
    #     continue

    mean_reward = np.mean(episode_returns)
    std_reward = np.std(episode_returns)
    rewards_summary.append({
        'algo': algo,
        'data_source': data_source,
        'category': category,
        'mean_reward': mean_reward,
        'std_reward': std_reward
    })
    # rewards_summary.append({  # TODO: save more info of navie agent, also can add action diversity
    #     'algo': algo,
    #     'data_source': data_source,
    #     'category': category,
    #     'mean_reward': mean_reward,
    #     'std_reward': std_reward,
    #     'mean_traj_len': np.mean(traj_len),
    #     'std_traj_len': np.std(traj_len),
    #     'mean_traj_len_without_nonsense_actions': np.mean(traj_len_without_nonsense_actions) if traj_len_without_nonsense_actions else 0,
    #     'std_traj_len_without_nonsense_actions': np.std(traj_len_without_nonsense_actions) if traj_len_without_nonsense_actions else 0,
    #     'mean_nonsense_actions': np.mean(nonsense_actions) # not sure mean of boolean
    # })


    
    print(f"{algo} {data_source} - {category}: Average Total Reward = {mean_reward}")

    # Record action diversity for the epoch
    action_diversity = len(action_diversity_set)
    action_diversity_summary.append({
        'algo': algo,
        'data_source': data_source,
        'action_diversity': action_diversity
    })
    print(f"Action diversity: {action_diversity}")
    print(f"Actions taken: {action_diversity_set}")

    # Save all evaluation trajectories to a CSV file
    df_trajectories = pd.DataFrame(evaluation_trajectories)
    # df_trajectories.to_csv(f'../models/training_log/eval_trajectories_{algo}_{data_source}.csv', index=False)
    print(f"Saved evaluation trajectories to 'eval_trajectories_{algo}_{data_source}.csv'")
    # Save rewards summary to a CSV file
    df_rewards = pd.DataFrame(rewards_summary)
    # df_rewards.to_csv(f'../models/training_log/rewards_summary_{algo}_{data_source}.csv', index=False)
    # Save action diversity summary to a CSV file
    df_action_diversity = pd.DataFrame(action_diversity_summary)
    # df_action_diversity.to_csv(f'../models/training_log/action_diversity_summary_{algo}_{data_source}.csv', index=False)

    return df_trajectories, df_rewards, df_action_diversity

In [ ]:
naive_agent_gen_trajectories, naive_agent_gen_reward, naive_agent_gen_action_diversity = naive_agent_evaluate_with_random(train_df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='train', retrain=False)


In [ ]:
naive_agent_gen_trajectories[naive_agent_gen_trajectories["hours_in"] == 50]

In [ ]:
1118/7443

### Evaluate Naive Agent on Patient Environment with different severity categories

In [ ]:
def naive_agent_evaluate(df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='DQN', data_source='test', retrain=False):
    agent = NaiveAgent(df, obs_hrs, state_cols, action_cols, baseline_cols)
    env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)
    cur_best_reward = -np.inf
    rewards_summary = []
    action_diversity_summary = []
    action_diversity_set = set()
    evaluation_trajectories = []  # List to store all trajectory steps

    for category, init_obs_list in eval_init_obs_categories.items():
        if not init_obs_list:
            print(f"Skipping {category}: No valid initial observations")
            continue

        # Full evaluation for reward statistics and trajectories
        episode_returns = []
        meet_weaning_list = []
        steps_list = []
        steps_to_weaning_list = []
        nonsense_action_list = []
        action_diversity_category_set = set()
        
        for idx in range(min(100, len(init_obs_list))):
            obs = env.reset(init_obs_list[idx])
            done = False
            total_reward = 0
            trajectory = []  # Trajectory for this episode
            stay_id = f'{category}_{idx}'
            # stay_id = f'eval_{algo}_{data_source}_epoch{epoch}_{category}_{idx}'

            # Extract sizes
            state_size = len(env.state_cols)  # e.g., 3 for heart_rate, resp_rate, spo2
            action_size = len(env.action_cols)  # e.g., 2 for fio2, respiratory_rate_set
            baseline_size = len(env.baseline_cols)  # e.g., 2 for gender_M, age
            obs_hrs = env.obs_hrs  # Assuming env has obs_hrs attribute

            # At t=0, include historical states and current state from initial obs
            for h in range(obs_hrs):  # h=0 for s_1, h=1 for s_2, ..., h=obs_hrs-1 for s_current
                state_start = h * (state_size + action_size)
                state = obs[state_start:state_start + state_size]  # Extract state
                # Action: use a_h for s_h, last historical action for s_current
                if h < obs_hrs - 1:
                    action_start = state_start + state_size
                    action = obs[action_start:action_start + action_size]
                else:
                    # For s_current, use the last historical action (a_(obs_hrs-1))
                    last_action_start = (obs_hrs - 1) * (state_size + action_size) - action_size
                    action = obs[last_action_start:last_action_start + action_size]
                # Baseline variables
                baseline_start = len(obs) - baseline_size
                baseline = obs[baseline_start:baseline_start + baseline_size]

                trajectory.append({
                    'stay_id': stay_id,
                    'hours_in': h,
                    **{col: state[i] for i, col in enumerate(env.state_cols)},
                    **{col: action[i] for i, col in enumerate(env.action_cols)},
                    **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
                })

            # Continue for remaining steps
            t = obs_hrs
            while not done:
                action_idx = agent.predict(np.expand_dims(obs, axis=0))[0]
                action_tuple = env.id_to_action[action_idx]  # Assuming this maps action index to tuple
                action_diversity_set.add(action_idx)
                action_diversity_category_set.add(action_idx)
                next_obs, reward, done, info = env.step(action_idx)
                total_reward += reward

                # Extract current state and baseline from next_obs
                state_start = (obs_hrs - 1) * (state_size + action_size)
                state = next_obs[state_start:state_start + state_size]  # s_current
                baseline_start = len(next_obs) - baseline_size
                baseline = next_obs[baseline_start:baseline_start + baseline_size]

                # Store trajectory
                trajectory.append({
                    'stay_id': stay_id,
                    'hours_in': t,
                    **{col: state[i] for i, col in enumerate(env.state_cols)},
                    **{col: action_tuple[i] for i, col in enumerate(env.action_cols)},
                    **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
                })

                obs = next_obs
                t += 1
            # {"nonsense_action": nonsense_action, "meet_weaning": meet_weaning}
            meet_weaning_list.append(info["meet_weaning"])
            nonsense_action_list.append(info["nonsense_action"])
            steps_list.append(t)
            if info["meet_weaning"]:
                steps_to_weaning_list.append(t)

            episode_returns.append(total_reward)
            evaluation_trajectories.extend(trajectory)  # Add this episode's steps to the full list

        if len(episode_returns) == 0:
            print(f"Skipping {category}: No valid episode returns")
            continue

        mean_reward = np.mean(episode_returns)
        std_reward = np.std(episode_returns)
        rewards_summary.append({
            'algo': algo,
            'data_source': data_source,
            'category': category,
            'mean_reward': mean_reward,
            'std_reward': std_reward,
            'meet_weaning_percentage': np.mean(meet_weaning_list),
            'mean_traj_len': np.mean(steps_list),
            'std_traj_len': np.std(steps_list),
            'mean_traj_len_to_meet_weaning': np.mean(steps_to_weaning_list),
            'std_traj_len_to_meet_weaning': np.std(steps_to_weaning_list),
            'action_diversity': len(action_diversity_category_set),
            'mean_nonsense_actions': np.mean(nonsense_action_list)

        })


        
        print(f"{algo} {data_source} - {category}: Average Total Reward = {mean_reward}")

        # Record action diversity for the epoch
        action_diversity = len(action_diversity_set)
        action_diversity_summary.append({
            'algo': algo,
            'data_source': data_source,
            'action_diversity': action_diversity
        })
        print(f"Action diversity: {action_diversity}")
        print(f"Actions taken: {action_diversity_set}")

    # Save all evaluation trajectories to a CSV file
    df_trajectories = pd.DataFrame(evaluation_trajectories)
    df_trajectories.to_csv(f'../models/training_log/naive_agent/eval_trajectories_{algo}_{data_source}.csv', index=False)
    print(f"Saved evaluation trajectories to 'eval_trajectories_{algo}_{data_source}.csv'")
    # Save rewards summary to a CSV file
    df_rewards = pd.DataFrame(rewards_summary)
    df_rewards.to_csv(f'../models/training_log/naive_agent/rewards_summary_{algo}_{data_source}.csv', index=False)
    # Save action diversity summary to a CSV file
    df_action_diversity = pd.DataFrame(action_diversity_summary)
    df_action_diversity.to_csv(f'../models/training_log/naive_agent/action_diversity_summary_{algo}_{data_source}.csv', index=False)
        # save model
        # if mean_reward > cur_best_reward: # TODO: only use common reward for finding the best model, since it is the last one to be eval
        #     cur_best_reward = mean_reward
        #     print(f"Saving model to {model_path}")
        #     agent.save_model(model_path)
    # # Save model
    # print(f"Saving model to {model_path}")
    # agent.save_model(model_path)

    return agent, env, df_rewards

In [ ]:
agent_naive_train, env, reward_summary_train = naive_agent_evaluate(train_df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='train', retrain=False)
agent_naive_test, env, reward_summary_test = naive_agent_evaluate(test_df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='test', retrain=False)
agent_naive_eICU, env, reward_summary_eICU = naive_agent_evaluate(eICU_disc, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='eICU', retrain=False)

In [ ]:
agent_naive_test, env, reward_summary_test = naive_agent_evaluate(test_df, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='test', retrain=False)
# agent_naive_eICU, env, reward_summary_eICU = naive_agent_evaluate(eICU_disc, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs=3, algo='naive_agent', data_source='eICU', retrain=False)

In [ ]:
reward_summary_train


In [ ]:
reward_summary_test


In [ ]:
reward_summary_eICU


In [ ]:
# Assuming your original dataframe is called 'df'
# Create the transformed dataframe
def turn_into_benchmark_table_format(df):
    transformed_data = []

    for _, row in df.iterrows():
        # Calculate formatted values
        total_reward = f"{row['mean_reward']:.2f}±{row['std_reward']:.2f}"
        meet_extubation = f"{row['meet_weaning_percentage']*100:.0f}%"
        nonsense_action = f"{row['mean_nonsense_actions']*100:.0f}%"
        avg_hours = f"{row['mean_traj_len']:.2f}±{row['std_traj_len']:.2f}"
        
        # Handle avg hours to meet - only show if weaning rate > 0
        if row['meet_weaning_percentage'] > 0:
            avg_hours_to_meet = f"{row['mean_traj_len_to_meet_weaning']:.2f}±{row['std_traj_len_to_meet_weaning']:.2f}"
        else:
            avg_hours_to_meet = "N/A"
        
        transformed_row = {
            'Category': row['category'].replace('_reward', ''),
            'N_Samples': 100,  # You may need to adjust this based on your actual sample sizes
            'Total Reward (Mean±Std)': total_reward,
            'Meet Extubation': meet_extubation,
            'Avg Hours (Mean±Std)': avg_hours,
            'Avg Hours to Meet (Mean±Std)': avg_hours_to_meet,
            'Action Diversity': int(row['action_diversity']),
            'Anomaly Action': nonsense_action
        }
        
        transformed_data.append(transformed_row)

    # Create the new dataframe
    new_df = pd.DataFrame(transformed_data)

    # If you want to add an "all" row (you'll need to calculate these values from your full dataset)
    # all_row = {
    #     'Category': 'all',
    #     'N_Samples': 7443,
    #     'Total Reward (Mean±Std)': '6.05±8.87',
    #     'Meet Extubation': '91.94%',
    #     'Avg Hours (Mean±Std)': '19.39±20.62',
    #     'Avg Hours to Meet (Mean±Std)': '16.02±14.42',
    #     'Action Diversity': 89
    # }
    # new_df = pd.concat([new_df, pd.DataFrame([all_row])], ignore_index=True)

    return new_df

In [ ]:
turn_into_benchmark_table_format(reward_summary_train)

In [ ]:
turn_into_benchmark_table_format(reward_summary_test)


In [ ]:
turn_into_benchmark_table_format(reward_summary_eICU)

In [ ]:
naive_agent_train_rewards = pd.read_csv('../models/training_log/naive_agent/rewards_summary_naive_agent_train.csv')
naive_agent_test_rewards = pd.read_csv('../models/training_log/naive_agent/rewards_summary_naive_agent_test.csv')
naive_agent_eICU_rewards = pd.read_csv('../models/training_log/naive_agent/rewards_summary_naive_agent_eICU.csv')
naive_agent_train_rewards

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the data from the provided CSVs
naive_agent_train_rewards = pd.read_csv('../models/training_log/naive_agentrewards_summary_naive_agent_train.csv')
naive_agent_test_rewards = pd.read_csv('../models/training_log/naive_agentrewards_summary_naive_agent_test.csv')
naive_agent_eICU_rewards = pd.read_csv('../models/training_log/naive_agentrewards_summary_naive_agent_eICU.csv')

# Add dataset column to each dataframe
naive_agent_train_rewards['dataset'] = 'Train'
naive_agent_test_rewards['dataset'] = 'Test'
naive_agent_eICU_rewards['dataset'] = 'eICU'

# Combine the dataframes
combined_data = pd.concat([naive_agent_train_rewards, naive_agent_test_rewards, naive_agent_eICU_rewards])

# Create a pivot table for the heatmap
heatmap_data = combined_data.pivot_table(
    index='category', 
    columns='dataset', 
    values='mean_reward',
    aggfunc='mean'
)

# Set up the matplotlib figure
plt.figure(figsize=(12, 8))

# Create a heatmap using seaborn
sns.heatmap(heatmap_data, annot=True, cmap="RdYlGn", center=0, fmt=".2f", linewidths=.5)

plt.title('Mean Rewards by Category and Dataset', fontsize=16)
plt.tight_layout()
plt.savefig('mean_rewards_heatmap.png', dpi=300)
plt.close()

# Create another heatmap for standard deviation
std_heatmap_data = combined_data.pivot_table(
    index='category', 
    columns='dataset', 
    values='std_reward',
    aggfunc='mean'
)

plt.figure(figsize=(12, 8))
sns.heatmap(std_heatmap_data, annot=True, cmap="Blues", fmt=".2f", linewidths=.5)
plt.title('Standard Deviation of Rewards by Category and Dataset', fontsize=16)
plt.tight_layout()
plt.savefig('std_rewards_heatmap.png', dpi=300)
plt.close()

# Let's also create a more comprehensive visualization
# Create a figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Plot mean rewards heatmap
sns.heatmap(heatmap_data, annot=True, cmap="RdYlGn", center=0, fmt=".2f", 
            linewidths=.5, ax=axes[0], cbar_kws={'label': 'Mean Reward'})
axes[0].set_title('Mean Rewards', fontsize=14)

# Plot std deviation heatmap
sns.heatmap(std_heatmap_data, annot=True, cmap="Blues", fmt=".2f", 
            linewidths=.5, ax=axes[1], cbar_kws={'label': 'Standard Deviation'})
axes[1].set_title('Standard Deviation of Rewards', fontsize=14)

plt.suptitle('Reward Statistics by Category and Dataset', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.savefig('combined_rewards_heatmap.png', dpi=300)

# Create a barplot for comparison across categories
plt.figure(figsize=(14, 10))
sns.barplot(x='category', y='mean_reward', hue='dataset', data=combined_data)
plt.title('Mean Rewards by Category Across Datasets', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('rewards_barplot.png', dpi=300)

print("Visualizations created successfully!")

# Display the summary statistics for each dataset
print("\nSummary Statistics:")
for dataset in ['Train', 'Test', 'eICU']:
    subset = combined_data[combined_data['dataset'] == dataset]
    print(f"\n{dataset} Dataset:")
    print(f"Average Mean Reward: {subset['mean_reward'].mean():.2f}")
    print(f"Average Std Reward: {subset['std_reward'].mean():.2f}")
    print(f"Best Category: {subset.loc[subset['mean_reward'].idxmax(), 'category']} ({subset['mean_reward'].max():.2f})")
    print(f"Worst Category: {subset.loc[subset['mean_reward'].idxmin(), 'category']} ({subset['mean_reward'].min():.2f})")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the data from the provided CSVs
naive_agent_train_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_train.csv')
naive_agent_test_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_test.csv')
naive_agent_eICU_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_eICU.csv')

# Add dataset column to each dataframe
naive_agent_train_rewards['Dataset'] = 'Train'
naive_agent_test_rewards['Dataset'] = 'Test'
naive_agent_eICU_rewards['Dataset'] = 'eICU'

# Combine the dataframes
combined_rewards = pd.concat([naive_agent_train_rewards, naive_agent_test_rewards, naive_agent_eICU_rewards])

# Rename columns to be consistent with the first part if needed
combined_rewards = combined_rewards.rename(columns={
    'category': 'Category',
    'mean_reward': 'mean_reward',
    'std_reward': 'std_reward',
    'mean_steps': 'mean_steps_to_weaning',
    'std_steps': 'std_steps_to_weaning'
})

# Define the metrics to plot (matching the first part)
metrics_to_plot = ['mean_reward', 'std_reward', 'mean_steps_to_weaning', 
                   'std_steps_to_weaning']

# Check if action_diversity exists in columns, if not we'll handle it separately
if 'action_diversity' in combined_rewards.columns:
    metrics_to_plot.append('action_diversity')

# Create subplots for each metric
fig, axes = plt.subplots(len(metrics_to_plot), 1, figsize=(14, 20))
fig.suptitle('Naive Agent: Comparison of Metrics Across Datasets and Categories', fontsize=16)

# Get the category order - keep it consistent with part 1 if possible
# You might need to adjust this based on what categories are in your data
category_order = combined_rewards['Category'].unique().tolist()
# Try to match the original order if those categories exist
preferred_order = [
    'large_reward', 'median_reward', 'small_reward',
    'short_to_wean', 'median_to_wean', 'long_to_wean'
]
category_order = [cat for cat in preferred_order if cat in category_order]
# Add any missing categories
for cat in combined_rewards['Category'].unique():
    if cat not in category_order:
        category_order.append(cat)

for i, metric in enumerate(metrics_to_plot):
    if metric not in combined_rewards.columns and metric == 'action_diversity':
        # Skip action_diversity if it doesn't exist in the dataset
        continue
    elif metric not in combined_rewards.columns:
        print(f"Warning: Metric '{metric}' not found in data. Skipping.")
        continue
    
    # Create pivot table for this metric
    pivot_data = combined_rewards.pivot_table(
        index='Category', 
        columns='Dataset', 
        values=metric
    )
    
    # Reorder categories for better visualization
    pivot_data = pivot_data.reindex(category_order)
    
    # Determine colormap based on metric (matching the original)
    if metric == 'mean_reward':
        cmap = 'RdYlGn'  # Red-Yellow-Green: Red for negative, Green for positive rewards
    elif 'steps' in metric:
        cmap = 'RdYlGn_r'  # Reversed: Green for fewer steps, Red for more steps
    elif metric == 'action_diversity':
        cmap = 'viridis'  # Neutral colormap for diversity
    elif metric == 'std_reward':
        cmap = 'coolwarm'  # Default colormap for std
    else:
        cmap = 'coolwarm'  # Default colormap
    
    # Plot heatmap
    ax = axes[i]
    sns.heatmap(pivot_data, annot=True, fmt=".2f", cmap=cmap, ax=ax, linewidths=0.5)
    
    # Format the heatmap
    ax.set_title(f'{metric}', fontsize=14)
    ax.set_ylabel('')
    if i < len(metrics_to_plot) - 1:
        ax.set_xlabel('')
    else:
        ax.set_xlabel('Dataset')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig('naive_agent_metrics_heatmaps.png', dpi=300)
plt.show()

# Let's also create a bar chart for mean_reward across all categories and datasets
plt.figure(figsize=(14, 8))
sns.barplot(x='Category', y='mean_reward', hue='Dataset', data=combined_rewards)
plt.title('Naive Agent: Mean Reward Comparison Across Categories and Datasets', fontsize=15)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('naive_agent_mean_rewards_barplot.png', dpi=300)
plt.show()

# If mean_steps_to_weaning exists, create a bar chart for it
if 'mean_steps_to_weaning' in combined_rewards.columns:
    plt.figure(figsize=(14, 8))
    sns.barplot(x='Category', y='mean_steps_to_weaning', hue='Dataset', data=combined_rewards)
    plt.title('Naive Agent: Mean Steps to Weaning Comparison Across Categories and Datasets', fontsize=15)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('naive_agent_mean_steps_barplot.png', dpi=300)
    plt.show()

# If your dataset has the necessary columns for a radar chart, include it
# This requires mean_reward, mean_steps_to_weaning, and action_diversity
radar_metrics = ['mean_reward']
if 'mean_steps_to_weaning' in combined_rewards.columns:
    radar_metrics.append('mean_steps_to_weaning')
if 'action_diversity' in combined_rewards.columns:
    radar_metrics.append('action_diversity')

if len(radar_metrics) >= 2:  # Need at least 2 metrics for a radar chart
    # Function to create radar chart (from original code)
    def create_radar_chart(categories, metrics, data, title):
        # Prepare data for radar chart
        radar_data = []
        for category in categories:
            for dataset in ['Train', 'Test', 'eICU']:
                values = {}
                for metric in metrics:
                    subset = data[(data['Dataset'] == dataset) & (data['Category'] == category)]
                    if not subset.empty and metric in subset.columns:
                        values[metric] = subset[metric].values[0]
                
                if len(values) == len(metrics):  # Only add if all metrics are present
                    entry = {'Category': category, 'Dataset': dataset}
                    entry.update(values)
                    radar_data.append(entry)
        
        # Convert to DataFrame
        radar_df = pd.DataFrame(radar_data)
        
        # Normalize data for radar chart (different scales)
        for metric in metrics:
            min_val = radar_df[metric].min()
            max_val = radar_df[metric].max()
            if max_val > min_val:  # Avoid division by zero
                radar_df[metric] = (radar_df[metric] - min_val) / (max_val - min_val)
        
        # Create the radar chart
        num_metrics = len(metrics)
        angles = np.linspace(0, 2*np.pi, num_metrics, endpoint=False).tolist()
        angles += angles[:1]  # Close the polygon
        
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
        
        # Add metrics labels
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(metrics)
        
        # Add data for each category and dataset
        for dataset in ['Train', 'Test', 'eICU']:
            for category in categories:
                subset = radar_df[(radar_df['Dataset'] == dataset) & (radar_df['Category'] == category)]
                if not subset.empty:
                    values = [subset[metric].values[0] for metric in metrics]
                    values += values[:1]  # Close the polygon
                    
                    ax.plot(angles, values, linewidth=2, label=f"{dataset} - {category}")
                    ax.fill(angles, values, alpha=0.1)
        
        ax.set_title(title, fontsize=15)
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        
        return fig

    # Define categories for radar chart
    radar_categories = combined_rewards['Category'].unique().tolist()[:3]  # Limit to 3 categories for clarity
    
    # Create radar chart
    radar_fig = create_radar_chart(
        radar_categories, 
        radar_metrics,
        combined_rewards,
        'Naive Agent: Normalized Comparison of Categories Across Datasets'
    )

    plt.tight_layout()
    plt.savefig('naive_agent_radar_chart.png', dpi=300)
    plt.show()

# Display the summary statistics for each dataset
print("\nNaive Agent Summary Statistics:")
for dataset in ['Train', 'Test', 'eICU']:
    subset = combined_rewards[combined_rewards['Dataset'] == dataset]
    print(f"\n{dataset} Dataset:")
    print(f"Average Mean Reward: {subset['mean_reward'].mean():.2f}")
    print(f"Average Std Reward: {subset['std_reward'].mean():.2f}")
    if 'mean_steps_to_weaning' in subset.columns:
        print(f"Average Steps to Weaning: {subset['mean_steps_to_weaning'].mean():.2f}")
    print(f"Best Category: {subset.loc[subset['mean_reward'].idxmax(), 'Category']} ({subset['mean_reward'].max():.2f})")
    print(f"Worst Category: {subset.loc[subset['mean_reward'].idxmin(), 'Category']} ({subset['mean_reward'].min():.2f})")

print("Visualizations created successfully!")

## Visualize the Training Progress of the Reward and action diversity

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
file_path = "../models/training_log/rewards_summary_DiscreteBCQ_eICU.csv"
data = pd.read_csv(file_path)

# Filter data for specific algorithm and data source
algo = "DiscreteBCQ"
data_source = "eICU"
filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

# Extract unique categories
unique_categories = filtered_data['category'].unique()

# Plot the mean reward and standard deviation for each category
plt.figure(figsize=(12, 8))

for category in unique_categories:
    category_data = filtered_data[filtered_data['category'] == category]
    epochs = category_data['epoch']
    mean_rewards = category_data['mean_reward']
    std_rewards = category_data['std_reward']

    # Plot mean reward curve
    plt.plot(epochs, mean_rewards, label=f"{category} (mean)", marker='o')

    # Plot standard deviation as shaded area
    plt.fill_between(
        epochs,
        mean_rewards - std_rewards,
        mean_rewards + std_rewards,
        alpha=0.2,
        label=f"{category} (std)"
    )

# Add title, labels, and legend
plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Reward", fontsize=14)
plt.xticks(np.arange(0, 10, 1))
plt.legend(loc="upper left", fontsize=10)
plt.grid(alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()

Without Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, default parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, Same parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

With Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Filter data for specific algorithm and data source
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        data = pd.read_csv(file_path)
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Plot the mean reward and standard deviation for each category
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_data[filtered_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category}", marker='o')
            # plt.plot(epochs, mean_rewards, label=f"{category} (mean+std)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
                # label=f"{category} (std)"
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Plot the mean reward, standard deviation, and action diversity
        plt.figure(figsize=(12, 8))

        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Plot action diversity curve
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']
        plt.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Add title, labels, and legend
        plt.title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Value", fontsize=14)
        plt.xticks(np.arange(0, 10, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

### Training progress with action diversity across different RL algo

Without Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, Same parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, With Same parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

With Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Combine legends from both axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

### Training progress with action diversity and trajectory length across different RL algo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC"]
data_source_list = ["train", "test", "eICU"]

for algo in algo_list:
    for data_source in data_source_list:
        # Load reward data
        reward_file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            reward_data = pd.read_csv(reward_file_path)
        except FileNotFoundError:
            print(f"Reward file not found for {algo} on {data_source}.")
            continue

        # Filter reward data for the current algorithm and data source
        filtered_reward_data = reward_data[
            (reward_data['algo'] == algo) & (reward_data['data_source'] == data_source)
        ]

        # Check if there is any reward data
        if filtered_reward_data.empty:
            print(f"No reward data found for {algo} on {data_source}.")
            continue

        # Load action diversity data
        diversity_file_path = f"../models/training_log/action_diversity_summary_{algo}_{data_source}.csv"
        try:
            diversity_data = pd.read_csv(diversity_file_path)
        except FileNotFoundError:
            print(f"Action diversity file not found for {algo} on {data_source}.")
            continue

        # Filter action diversity data for the current algorithm and data source
        filtered_diversity_data = diversity_data[
            (diversity_data['algo'] == algo) & (diversity_data['data_source'] == data_source)
        ]

        # Check if there is any action diversity data
        if filtered_diversity_data.empty:
            print(f"No action diversity data found for {algo} on {data_source}.")
            continue

        # Extract unique categories for rewards
        unique_categories = filtered_reward_data['category'].unique()

        # Create a figure and axis for the plot
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot the mean reward and standard deviation
        for category in unique_categories:
            category_data = filtered_reward_data[filtered_reward_data['category'] == category]
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            ax1.plot(epochs, mean_rewards, label=f"{category} (Reward)", marker='o')

            # Plot standard deviation as shaded area
            ax1.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2,
            )

        # Set labels and title for the left y-axis (rewards)
        ax1.set_xlabel("Epoch", fontsize=14)
        ax1.set_ylabel("Reward", fontsize=14, color="blue")
        ax1.tick_params(axis='y', labelcolor="blue")
        ax1.set_title(f"Training Progress for {algo} on {data_source}", fontsize=16)
        ax1.grid(alpha=0.3)

        # Create a secondary y-axis for action diversity
        ax2 = ax1.twinx()
        diversity_epochs = filtered_diversity_data['epoch']
        action_diversity = filtered_diversity_data['action_diversity']

        # Plot action diversity curve
        ax2.plot(diversity_epochs, action_diversity, label="Action Diversity", color="black", linestyle="--", marker='x')

        # Set labels for the right y-axis (action diversity)
        ax2.set_ylabel("Action Diversity", fontsize=14, color="black")
        ax2.tick_params(axis='y', labelcolor="black")

        # Plot mean trajectory length and standard deviation
        traj_epochs = filtered_reward_data['epoch']
        mean_traj_len = filtered_reward_data['mean_traj_len']
        std_traj_len = filtered_reward_data['std_traj_len']

        # Use the same axis as ax1 but with a different color/style
        ax3 = ax1.twinx()
        ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
        ax3.plot(traj_epochs, mean_traj_len, label="Mean Trajectory Length", color="green", linestyle="-.", marker='s')
        ax3.fill_between(
            traj_epochs,
            mean_traj_len - std_traj_len,
            mean_traj_len + std_traj_len,
            color="green",
            alpha=0.2,
        )
        ax3.set_ylabel("Trajectory Length", fontsize=14, color="green")
        ax3.tick_params(axis='y', labelcolor="green")

        # Combine legends from all axes
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        lines_3, labels_3 = ax3.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2 + lines_3, labels_1 + labels_2 + labels_3, loc="upper left", fontsize=10)

        # Adjust layout and show the plot
        plt.tight_layout()
        plt.show()

### Training Progress Curve: Reward

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2
            )

        # Add title, labels, and legend
        plt.title(f"Training Progress for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

### Training Progress Curve: Reward with Naive Agent

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Load naive_agent data
naive_agent_train_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_train.csv')
naive_agent_test_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_test.csv')
naive_agent_eICU_rewards = pd.read_csv('../models/training_log/rewards_summary_naive_agent_eICU.csv')

# Map data sources to naive_agent data
naive_agent_data_map = {
    "train": naive_agent_train_rewards,
    "test": naive_agent_test_rewards,
    "eICU": naive_agent_eICU_rewards
}

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Get naive_agent data for the current data source
    naive_agent_data = naive_agent_data_map[data_source]

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        # Plot data for each algorithm
        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_rewards = category_data['mean_reward']
            std_rewards = category_data['std_reward']

            # Plot mean reward curve
            plt.plot(epochs, mean_rewards, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_rewards - std_rewards,
                mean_rewards + std_rewards,
                alpha=0.2
            )

        # Add naive_agent constant curve
        if category in naive_agent_data['category'].values:
            naive_mean_reward = naive_agent_data[naive_agent_data['category'] == category]['mean_reward'].values[0]
            plt.axhline(y=naive_mean_reward, color='r', linestyle='--', label='Physician Policy')

        # Add title, labels, and legend
        plt.title(f"Training Progress for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Reward", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

### Training Progress Curve: Length of trajectory

Without Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_traj_len = category_data['mean_traj_len_without_nonsense_actions']
            std_traj_len = category_data['std_traj_len_without_nonsense_actions']

            # Plot mean trajectory length curve
            plt.plot(epochs, mean_traj_len, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_traj_len - std_traj_len,
                mean_traj_len + std_traj_len,
                alpha=0.2
            )

        # Add title, labels, and legend
        plt.title(f"Trajectory Length Progress for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Trajectory Length (without nonsense actions)", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, With Same parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_traj_len = category_data['mean_traj_len_without_nonsense_actions']
            std_traj_len = category_data['std_traj_len_without_nonsense_actions']

            # Plot mean trajectory length curve
            plt.plot(epochs, mean_traj_len, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_traj_len - std_traj_len,
                mean_traj_len + std_traj_len,
                alpha=0.2
            )

        # Add title, labels, and legend
        plt.title(f"Trajectory Length Progress for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Trajectory Length (without nonsense actions)", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

With Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_traj_len = category_data['mean_traj_len_without_nonsense_actions']
            std_traj_len = category_data['std_traj_len_without_nonsense_actions']

            # Plot mean trajectory length curve
            plt.plot(epochs, mean_traj_len, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            plt.fill_between(
                epochs,
                mean_traj_len - std_traj_len,
                mean_traj_len + std_traj_len,
                alpha=0.2
            )

        # Add title, labels, and legend
        plt.title(f"Trajectory Length Progress for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Trajectory Length (without nonsense actions)", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

### Training Progress Curve: Nonsense Action

Without Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/{EXP_FOLDER_PREFIX}/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_nonsense_actions = category_data['mean_nonsense_actions']
            # std_nonsense_actions = category_data['std_nonsense_actions']

            # Plot mean nonsense actions curve
            plt.plot(epochs, mean_nonsense_actions, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            # plt.fill_between(
            #     epochs,
            #     mean_nonsense_actions - std_nonsense_actions,
            #     mean_nonsense_actions + std_nonsense_actions,
            #     alpha=0.2
            # )

        # Add title, labels, and legend
        plt.title(f"Mean Anomaly Actions for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Anomaly Actions", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

Without Action penalty, With Weaning Reward on Agent, With Same parameter settings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_nonsense_actions = category_data['mean_nonsense_actions']
            # std_nonsense_actions = category_data['std_nonsense_actions']

            # Plot mean nonsense actions curve
            plt.plot(epochs, mean_nonsense_actions, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            # plt.fill_between(
            #     epochs,
            #     mean_nonsense_actions - std_nonsense_actions,
            #     mean_nonsense_actions + std_nonsense_actions,
            #     alpha=0.2
            # )

        # Add title, labels, and legend
        plt.title(f"Mean Anomaly Actions for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Anomaly Actions", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

With Action penalty

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the list of algorithms and data sources
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]

# Iterate over each data source
for data_source in data_source_list:
    # Initialize a dictionary to store data for each category
    category_data_dict = {}

    # Iterate over each algorithm
    for algo in algo_list:
        file_path = f"../models/training_log/rewards_summary_{algo}_{data_source}.csv"
        try:
            data = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        # Filter data for the current algorithm and data source
        filtered_data = data[(data['algo'] == algo) & (data['data_source'] == data_source)]

        # Check if there is any data for the current algorithm and data source
        if filtered_data.empty:
            print(f"No data found for {algo} on {data_source}.")
            continue

        # Extract unique categories
        unique_categories = filtered_data['category'].unique()

        # Organize data by category
        for category in unique_categories:
            if category not in category_data_dict:
                category_data_dict[category] = []
            category_data_dict[category].append((algo, filtered_data[filtered_data['category'] == category]))

    # Plot data for each category
    for category, algo_data_list in category_data_dict.items():
        plt.figure(figsize=(12, 8))

        for algo, category_data in algo_data_list:
            epochs = category_data['epoch']
            mean_nonsense_actions = category_data['mean_nonsense_actions']
            # std_nonsense_actions = category_data['std_nonsense_actions']

            # Plot mean nonsense actions curve
            plt.plot(epochs, mean_nonsense_actions, label=f"{algo}", marker='o')

            # Plot standard deviation as shaded area
            # plt.fill_between(
            #     epochs,
            #     mean_nonsense_actions - std_nonsense_actions,
            #     mean_nonsense_actions + std_nonsense_actions,
            #     alpha=0.2
            # )

        # Add title, labels, and legend
        plt.title(f"Mean Anomaly Actions for {category} on {data_source}", fontsize=16)
        plt.xlabel("Epoch", fontsize=14)
        plt.ylabel("Mean Anomaly Actions", fontsize=14)
        plt.xticks(np.arange(0, 20, 1))
        plt.legend(loc="upper left", fontsize=10)
        plt.grid(alpha=0.3)

        # Show the plot
        plt.tight_layout()
        plt.show()

## Benchmarking

### Table

Without Action penalty

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Define the lists
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]
categories = ["small_reward", "median_reward", "large_reward"]

# Load all data into a single DataFrame
data_frames = []
for algo in algo_list:
    for data_source in data_source_list:
        file_path = f'../models/training_log/rewards_summary_{algo}_{data_source}.csv'
        try:
            df = pd.read_csv(file_path)
            df['algo'] = algo
            df['data_source'] = data_source
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

# Combine all data into one DataFrame
data = pd.concat(data_frames, ignore_index=True)

# Step 1: Find the best epoch for "small_reward" category
best_epochs = {}
for algo in algo_list:
    for data_source in data_source_list:
        subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == 'small_reward')]
        if not subset.empty:
            best_epoch = subset.loc[subset['mean_reward'].idxmax()]
            best_epochs[(algo, data_source)] = best_epoch

# Step 2: Build tables for each data_source and category
tables = {}
for data_source in data_source_list:
    for category in categories:
        table_rows = []
        for algo in algo_list:
            subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == category)]
            if not subset.empty:
                # Calculate averages and standard deviations
                avg_reward = subset['mean_reward'].mean()
                std_reward = subset['std_reward'].mean()
                avg_traj_len = subset['mean_traj_len_to_meet_weaning'].mean()
                std_traj_len = subset['std_traj_len_without_nonsense_actions'].mean()
                avg_meet_weaning = subset['meet_weaning_percentage'].mean()

                # Append row to the table
                table_rows.append({
                    "RL Algo": algo,
                    "Avg Total Reward (Std)": f"{avg_reward:.2f} ({std_reward:.2f})",
                    "Avg Length Meet Extubation": f"{avg_traj_len:.2f} ({std_traj_len:.2f})",
                    "Percentage of Meet Extubation": f"{avg_meet_weaning:.2f}"
                })

        # Create a DataFrame for the table
        table = pd.DataFrame(table_rows)
        tables[(data_source, category)] = table

# Step 3: Display the tables
# for (data_source, category), table in tables.items():
#     print(f"Table for Data Source: {data_source}, Category: {category}")
#     print(table)
#     print("\n")
for (data_source, category), table in tables.items():
    print(f"Table for Data Source: {data_source}, Category: {category}")
    print(tabulate(table, headers='keys', tablefmt='grid'))
    print("\n")

Without Action penalty, With Weaning Reward on Agent

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Define the lists
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]
categories = ["small_reward", "median_reward", "large_reward"]

# Load all data into a single DataFrame
data_frames = []
for algo in algo_list:
    for data_source in data_source_list:
        file_path = f'../models/training_log/rewards_summary_{algo}_{data_source}.csv'
        try:
            df = pd.read_csv(file_path)
            df['algo'] = algo
            df['data_source'] = data_source
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

# Combine all data into one DataFrame
data = pd.concat(data_frames, ignore_index=True)

# Step 1: Find the best epoch for "small_reward" category
best_epochs = {}
for algo in algo_list:
    for data_source in data_source_list:
        subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == 'small_reward')]
        if not subset.empty:
            best_epoch = subset.loc[subset['mean_reward'].idxmax()]
            best_epochs[(algo, data_source)] = best_epoch

# Step 2: Build tables for each data_source and category
tables = {}
for data_source in data_source_list:
    for category in categories:
        table_rows = []
        for algo in algo_list:
            subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == category)]
            if not subset.empty:
                # Calculate averages and standard deviations
                avg_reward = subset['mean_reward'].mean()
                std_reward = subset['std_reward'].mean()
                avg_traj_len = subset['mean_traj_len_to_meet_weaning'].mean()
                std_traj_len = subset['std_traj_len_without_nonsense_actions'].mean()
                avg_meet_weaning = subset['meet_weaning_percentage'].mean()

                # Append row to the table
                table_rows.append({
                    "RL Algo": algo,
                    "Avg Total Reward (Std)": f"{avg_reward:.2f} ({std_reward:.2f})",
                    "Avg Length Meet Extubation": f"{avg_traj_len:.2f} ({std_traj_len:.2f})",
                    "Percentage of Meet Extubation": f"{avg_meet_weaning:.2f}"
                })

        # Create a DataFrame for the table
        table = pd.DataFrame(table_rows)
        tables[(data_source, category)] = table

# Step 3: Display the tables
# for (data_source, category), table in tables.items():
#     print(f"Table for Data Source: {data_source}, Category: {category}")
#     print(table)
#     print("\n")
for (data_source, category), table in tables.items():
    print(f"Table for Data Source: {data_source}, Category: {category}")
    print(tabulate(table, headers='keys', tablefmt='grid'))
    print("\n")

Without Action penalty, With Weaning Reward on Agent, With Same parameter settings

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Define the lists
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]
categories = ["small_reward", "median_reward", "large_reward"]

# Load all data into a single DataFrame
data_frames = []
for algo in algo_list:
    for data_source in data_source_list:
        file_path = f'../models/training_log/rewards_summary_{algo}_{data_source}.csv'
        try:
            df = pd.read_csv(file_path)
            df['algo'] = algo
            df['data_source'] = data_source
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

# Combine all data into one DataFrame
data = pd.concat(data_frames, ignore_index=True)

# Step 1: Find the best epoch for "small_reward" category
best_epochs = {}
for algo in algo_list:
    for data_source in data_source_list:
        subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == 'small_reward')]
        if not subset.empty:
            best_epoch = subset.loc[subset['mean_reward'].idxmax()]
            best_epochs[(algo, data_source)] = best_epoch

# Step 2: Build tables for each data_source and category
tables = {}
for data_source in data_source_list:
    for category in categories:
        table_rows = []
        for algo in algo_list:
            subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == category)]
            if not subset.empty:
                # Calculate averages and standard deviations
                avg_reward = subset['mean_reward'].mean()
                std_reward = subset['std_reward'].mean()
                avg_traj_len = subset['mean_traj_len_to_meet_weaning'].mean()
                std_traj_len = subset['std_traj_len_without_nonsense_actions'].mean()
                avg_meet_weaning = subset['meet_weaning_percentage'].mean()

                # Append row to the table
                table_rows.append({
                    "RL Algo": algo,
                    "Avg Total Reward (Std)": f"{avg_reward:.2f} ({std_reward:.2f})",
                    "Avg Length Meet Extubation": f"{avg_traj_len:.2f} ({std_traj_len:.2f})",
                    "Percentage of Meet Extubation": f"{avg_meet_weaning:.2f}"
                })

        # Create a DataFrame for the table
        table = pd.DataFrame(table_rows)
        tables[(data_source, category)] = table

# Step 3: Display the tables
# for (data_source, category), table in tables.items():
#     print(f"Table for Data Source: {data_source}, Category: {category}")
#     print(table)
#     print("\n")
for (data_source, category), table in tables.items():
    print(f"Table for Data Source: {data_source}, Category: {category}")
    print(tabulate(table, headers='keys', tablefmt='grid'))
    print("\n")

With Action penalty

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Define the lists
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]
categories = ["small_reward", "median_reward", "large_reward"]

# Load all data into a single DataFrame
data_frames = []
for algo in algo_list:
    for data_source in data_source_list:
        file_path = f'../models/training_log/rewards_summary_{algo}_{data_source}.csv'
        try:
            df = pd.read_csv(file_path)
            df['algo'] = algo
            df['data_source'] = data_source
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

# Combine all data into one DataFrame
data = pd.concat(data_frames, ignore_index=True)

# Step 1: Find the best epoch for "small_reward" category
best_epochs = {}
for algo in algo_list:
    for data_source in data_source_list:
        subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == 'small_reward')]
        if not subset.empty:
            best_epoch = subset.loc[subset['mean_reward'].idxmax()]
            best_epochs[(algo, data_source)] = best_epoch

# Step 2: Build tables for each data_source and category
tables = {}
for data_source in data_source_list:
    for category in categories:
        table_rows = []
        for algo in algo_list:
            subset = data[(data['algo'] == algo) & (data['data_source'] == data_source) & (data['category'] == category)]
            if not subset.empty:
                # Calculate averages and standard deviations
                avg_reward = subset['mean_reward'].mean()
                std_reward = subset['std_reward'].mean()
                avg_traj_len = subset['mean_traj_len_without_nonsense_actions'].mean()
                std_traj_len = subset['std_traj_len_without_nonsense_actions'].mean()
                avg_meet_weaning = subset['meet_weaning_percentage'].mean()

                # Append row to the table
                table_rows.append({
                    "RL Algo": algo,
                    "Avg Total Reward (Std)": f"{avg_reward:.2f} ({std_reward:.2f})",
                    "Avg Length Meet Extubation": f"{avg_traj_len:.2f} ({std_traj_len:.2f})",
                    "Percentage of Meet Extubation": f"{avg_meet_weaning:.2f}"
                })

        # Create a DataFrame for the table
        table = pd.DataFrame(table_rows)
        tables[(data_source, category)] = table

# Step 3: Display the tables
for (data_source, category), table in tables.items():
    print(f"Table for Data Source: {data_source}, Category: {category}")
    print(table)
    print("\n")

In [ ]:
from tabulate import tabulate
import pandas as pd

# Assuming `tables` is a dictionary with keys as (data_source, category) and values as DataFrames
for (data_source, category), table in tables.items():
    print(f"Table for Data Source: {data_source}, Category: {category}")
    print(tabulate(table, headers='keys', tablefmt='grid'))
    print("\n")

### Plot of Action Diversity

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define the lists
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
data_source_list = ["train", "test", "eICU"]
categories = ["small_reward", "median_reward", "large_reward"]

# Initialize an empty DataFrame to store all data
all_data = []

# Load data for each algorithm and data source
for algo in algo_list:
    for data_source in data_source_list:
        file_path = f'../models/training_log/rewards_summary_{algo}_{data_source}.csv'
        if os.path.exists(file_path):
            # Read the CSV file
            df = pd.read_csv(file_path)
            df['algo'] = algo
            df['data_source'] = data_source
            all_data.append(df)

# Concatenate all data into a single DataFrame
if all_data:
    data = pd.concat(all_data, ignore_index=True)
else:
    raise FileNotFoundError("No data files found in the specified paths.")

# Plotting
sns.set(style="whitegrid")
fig, axes = plt.subplots(len(data_source_list), len(categories), figsize=(18, 12), sharex=True, sharey=True)

for i, data_source in enumerate(data_source_list):
    for j, category in enumerate(categories):
        ax = axes[i, j]
        
        # Filter data for the current data_source and category
        subset = data[(data['data_source'] == data_source) & (data['category'] == category)]
        
        # Plot action diversity for each algorithm
        for algo in algo_list:
            algo_data = subset[subset['algo'] == algo]
            ax.plot(algo_data['epoch'], algo_data['action_diversity'], label=algo, marker='o')
        
        # Set plot title and labels
        ax.set_title(f"{data_source} - {category}", fontsize=12)
        ax.set_xlabel("Epoch", fontsize=10)
        ax.set_ylabel("Action Diversity", fontsize=10)
        ax.legend(fontsize=8, loc='upper right')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Benchmark by input trajectory

In [ ]:
def compute_trajectory_metrics_for_benchmark(df, state_cols, action_cols, baseline_cols, 
                               min_max_values_guidelines_bin_dict,
                               obs_hrs=3, stable_hr=2, stable_threshold=2, 
                               timestamp_reward_step=1, max_timestamp=48):
    """
    Compute trajectory metrics per stay:
      - Uses the same penalty logic as the environment.
      - Clips trajectory when weaning (6 consecutive zero-penalty steps) occurs.
      - Calculates total reward, steps to weaning for each stay.
      - Prints summary: reward distribution, average steps, and proportion not weaned.
    """
    import numpy as np
    import pandas as pd

    metrics_list = []
    total_rewards = []
    steps_list = []
    steps_meet_weaning_list = []
    num_stays = 0
    num_not_weaned = 0
    single_agent_action_diversity_set = set()
    
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in')
        if len(group) < obs_hrs + 1:
            continue
        
        rows = group.to_dict('records')
        # Seed history
        history = []
        for i in range(obs_hrs - 1):
            history.append((tuple(rows[i][col] for col in state_cols),
                            tuple(rows[i][col] for col in action_cols)))
        
        current_state = tuple(rows[obs_hrs-1][col] for col in state_cols)
        previous_action = tuple(rows[obs_hrs-2][col] for col in action_cols) # TODO: [update reward]
        current_action = tuple(rows[obs_hrs-2][col] for col in action_cols) # TODO: [update reward]
        total_reward = 0
        weaning_count = 0
        weaning_met = False
        steps_to_weaning = None
        action_diversity_set = set()
        anomaly_action = False
        
        # TODO: [update reward Part 2]
        # Iterate through trajectory
        for t in range(obs_hrs - 1, len(rows) - 1):
            if weaning_met:
                break
            step = t - (obs_hrs - 1)  # zero-index step
            previous_action = current_action
            current_action = tuple(rows[t][col] for col in action_cols)
            next_state = tuple(rows[t+1][col] for col in state_cols)

            action_diversity_set.add(current_action)
            single_agent_action_diversity_set.add(current_action)
            
            # State penalty, This is used to check, not to calculate reward
            state_penalty = -1 if any(
                not (min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= 
                     min_max_values_guidelines_bin_dict[col]['max'])
                for i, col in enumerate(state_cols)
            ) else 0

            state_reward = sum(
                1 if min_max_values_guidelines_bin_dict[col]['min'] <= next_state[i] <= min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)
            
            # Action penalty
            # action_penalty = -1 if any(
            #     not (min_max_values_guidelines_bin_dict[col]['min'] <= a <= 
            #          min_max_values_guidelines_bin_dict[col]['max'])
            #     for a, col in zip(current_action, action_cols)
            # ) else 0
            action_penalty = sum(
                -1 if current_action[i] < min_max_values_guidelines_bin_dict[col]['min'] or current_action[i] > min_max_values_guidelines_bin_dict[col]['max'] else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)
            
            # Stable penalty
            # stable_penalty = 0
            # if len(history) >= stable_hr:
            #     stable = True
            #     for i in range(-stable_hr + 1, 0):
            #         for j in range(len(state_cols)):
            #             diff = abs(history[i][0][j] - history[i-1][0][j])
            #             if diff >= stable_threshold:
            #                 stable = False
            #                 break
            #         if not stable:
            #             break
            #     if stable:
            #         last_state = history[-1][0]
            #         for j in range(len(state_cols)):
            #             diff = abs(current_state[j] - last_state[j])
            #             if diff >= stable_threshold:
            #                 stable = False
            #                 break
            #     stable_penalty = -1 if not stable else 0

            state_stable_penalty = sum(
                -1 if abs(next_state[i] - current_state[i]) >= stable_threshold else 0
                for i, col in enumerate(state_cols)
            ) / len(state_cols)
            
            action_stable_penalty = sum(
                -1 if abs(previous_action[i] - current_action[i]) >= stable_threshold * 2 else 0
                for i, col in enumerate(action_cols)
            ) / len(action_cols)

            # Timestamp penalty
            timestamp_penalty = -1 if (step + 1) % timestamp_reward_step == 0 else 0
            
            # Accumulate reward
            if ACTION_PENALTY_FLAG:
                reward = state_reward + action_penalty + state_stable_penalty + action_stable_penalty + timestamp_penalty
            else:
                reward = state_reward + state_stable_penalty + action_stable_penalty + timestamp_penalty
            total_reward += reward
            
            # Check weaning condition
            # if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0:
            if state_penalty == 0 and action_penalty == 0 and state_stable_penalty == 0 and action_stable_penalty == 0: # TODO: try add action_stable_penalty
                weaning_count += 1
            else:
                weaning_count = 0
            
            # if weaning_count >= 6: # TODO: 6 is set by my self
            # if weaning_count >= 6 and step >= 23: # TODO: 6 is set by my self # TODO: [update reward]
            if weaning_count >= 6: # TODO: 6 is set by my self
                # total_reward += 10 # TODO: [update reward]
                if WEANING_REWARD_FLAG:
                    total_reward += 10
                weaning_met = True
                steps_to_weaning = step + 1
                break
            # [update reward] TODO: elif last step and weaning_cout >= 2
            # elif step == len(rows) - obs_hrs - 1 and weaning_count >= 2:
            #     total_reward += 100
            #     weaning_met = True
            #     steps_to_weaning = step + 1

            # Update history and state for next iteration
            history = history[1:] + [(current_state, current_action)]
            current_state = next_state
        
        # If not weaned, mark steps_to_weaning as total steps
        if not weaning_met:
            steps_to_weaning = len(rows) - obs_hrs
            num_not_weaned += 1
            if steps_to_weaning < max_timestamp:
                anomaly_action = True
            total_reward -= 10  # TODO: [update reward] penalty for not weaning
        
        metrics_list.append({
            'stay_id': stay_id,
            'total_reward': total_reward,
            'meet_weaning': weaning_met,
            'steps_to_weaning': steps_to_weaning,
            'anomaly_action': anomaly_action,
            'action_diversity': len(action_diversity_set),
            'action': action_diversity_set
        })
        total_rewards.append(total_reward)
        steps_list.append(steps_to_weaning)
        if weaning_met:
            steps_meet_weaning_list.append(steps_to_weaning)
        num_stays += 1
    
    metrics_df = pd.DataFrame(metrics_list) # every trajectory is a row
    
    # Summary statistics
    avg_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)
    prop_weaned = metrics_df['meet_weaning'].mean() if num_stays > 0 else None
    avg_steps = np.mean(steps_list)
    std_steps = np.std(steps_list)
    avg_steps_meet_weaning = np.mean(steps_meet_weaning_list) if steps_meet_weaning_list else None
    std_steps_meet_weaning = np.std(steps_meet_weaning_list) if steps_meet_weaning_list else None
    action_diversity = len(single_agent_action_diversity_set)
    anomaly_action = len(metrics_df[metrics_df['anomaly_action']]) / num_stays if num_stays > 0 else None

    prop_not_weaned = num_not_weaned / num_stays if num_stays > 0 else None
    
    # print(f"Total stays evaluated: {num_stays}")
    # print(f"Reward (mean ± std): {avg_reward:.2f} ± {std_reward:.2f}")
    # print(f"Proportion weaned: {prop_weaned:.2%}")
    # print(f"Average steps (mean ± std): {avg_steps:.2f} ± {std_steps:.2f}")
    # print(f"Average steps to weaning (mean ± std): {avg_steps_meet_weaning:.2f} ± {std_steps_meet_weaning:.2f}")
    # print(f"Action diversity: {action_diversity}")
    # print(f"Proportion of anomaly actions: {anomaly_action:.2%}")

    benchmark_summary = {
        'total_stays': num_stays,
        'avg_reward': avg_reward,
        'std_reward': std_reward,
        'prop_weaned': prop_weaned,
        'avg_steps': avg_steps,
        'std_steps': std_steps,
        'avg_steps_meet_weaning': avg_steps_meet_weaning,
        'std_steps_meet_weaning': std_steps_meet_weaning,
        'action_diversity': action_diversity,
        'anomaly_action_proportion': anomaly_action,
    }
    
    return benchmark_summary

In [ ]:
# metrics_df = compute_trajectory_metrics(train_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
# metrics_train_df = compute_trajectory_metrics(train_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
# metrics_test_df = compute_trajectory_metrics(test_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)
# metrics_eICU_df = compute_trajectory_metrics(eICU_disc, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)

In [ ]:
gen_traj_path_folder_list = [
    "2025_06_08_with_action_penalty_training_log",      # tune hyperparameters, with action penalty, without weaning reward
    "2025_06_09_without_action_penalty_training_log",   # tune hyperparameters, without action penalty, without weaning reward
    "2025_06_09_without_action_penalty_with_weaning_reward_training_log", # tune hyperparameters, without action penalty, with weaning reward
    "2025_06_09_without_action_penalty_with_weaning_reward_same_parameter_settings_v1_training_log",    # same hyperparameters, without action penalty, with weaning reward
    "2025_06_10_without_action_penalty_with_weaning_reward_default_parameter_settings_v1_training_log"  # default hyperparameters, without action penalty, with weaning reward
    ]
# algo_list = ["DQN", "DoubleDQN"]
algo_list = ["DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL", "DiscreteBC", "NFQ"]
# data_source_list = ["train", "test"]
data_source_list = ["train", "test", "eICU"]
# train_results = {}
# test_results = {}
# eICU_results = {}
# small_reward_benchmark_results = {}
benchmark_results_df = pd.DataFrame()
for gen_traj_path_folder in gen_traj_path_folder_list:
    for data_source in data_source_list:
        for algo in algo_list:
            file_path = f"../models/{gen_traj_path_folder}/eval_trajectories_{algo}_{data_source}_epoch19.csv"
            try:
                df = pd.read_csv(file_path)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            # get three sub df, as 19_small_reward_{id}, 19_median_reward_{id}, 19_large_reward_{id}
            # if stay_id include "small_reward", "median_reward", "large_reward"
            small_reward_df = df[df['stay_id'].str.contains("small_reward")]
            median_reward_df = df[df['stay_id'].str.contains("median_reward")]
            large_reward_df = df[df['stay_id'].str.contains("large_reward")]
            # Compute metrics for each category
            # print(f"Metrics for {algo} on {data_source} in {gen_traj_path_folder} - Small Reward Category:")
            metrics_small_reward_df = compute_trajectory_metrics_for_benchmark(
                small_reward_df, state_cols, action_cols, baseline_cols, 
                min_max_values_guidelines_bin_dict
            )
            # print(f"Metrics for {algo} on {data_source} in {gen_traj_path_folder} - Median Reward Category:")
            metrics_median_reward_df = compute_trajectory_metrics_for_benchmark(
                median_reward_df, state_cols, action_cols, baseline_cols, 
                min_max_values_guidelines_bin_dict
            )
            # print(f"Metrics for {algo} on {data_source} in {gen_traj_path_folder} - Large Reward Category:")
            metrics_large_reward_df = compute_trajectory_metrics_for_benchmark(
                large_reward_df, state_cols, action_cols, baseline_cols, 
                min_max_values_guidelines_bin_dict
            )
            metrics_small_reward_df['algo'] = algo
            metrics_small_reward_df['data_source'] = data_source
            metrics_small_reward_df['category'] = 'small_reward'
            metrics_small_reward_df['train_settings'] = gen_traj_path_folder
            metrics_median_reward_df['algo'] = algo
            metrics_median_reward_df['data_source'] = data_source
            metrics_median_reward_df['category'] = 'median_reward'
            metrics_median_reward_df['train_settings'] = gen_traj_path_folder
            metrics_large_reward_df['algo'] = algo
            metrics_large_reward_df['data_source'] = data_source
            metrics_large_reward_df['category'] = 'large_reward'
            metrics_large_reward_df['train_settings'] = gen_traj_path_folder
            benchmark_results_df = pd.concat([
                benchmark_results_df, 
                pd.DataFrame([metrics_small_reward_df]),
                pd.DataFrame([metrics_median_reward_df]),
                pd.DataFrame([metrics_large_reward_df])
            ], ignore_index=True)
benchmark_results_df

In [ ]:
benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test")]


In [ ]:
benchmark_results_df.to_csv("../models/benchmark_results.csv", index=False)

In [ ]:
# benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "train")]
# benchmark_results_df[(benchmark_results_df["category"] == "large_reward") & (benchmark_results_df["data_source"] == "train")]
# benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test")]
# benchmark_results_df[(benchmark_results_df["category"] == "median_reward") & (benchmark_results_df["data_source"] == "test")]
# benchmark_results_df[(benchmark_results_df["category"] == "large_reward") & (benchmark_results_df["data_source"] == "test")]
# benchmark_results_df[(benchmark_results_df["category"] == "median_reward") & (benchmark_results_df["data_source"] == "eICU")]
# benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "train") & (benchmark_results_df["algo"] == "DiscreteCQL")]
# benchmark_results_df[(benchmark_results_df["category"] == "median_reward") & (benchmark_results_df["data_source"] == "test") & (benchmark_results_df["algo"] == "DiscreteCQL")]
benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test") & (benchmark_results_df["algo"] == "DiscreteBC")]

In [ ]:
# benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test") & (benchmark_results_df["algo"] == "DQN")]
benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test") & (benchmark_results_df["algo"] == "DQN")]
# benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "eICU") & (benchmark_results_df["algo"] == "DiscreteBC")]

In [ ]:
benchmark_results_df[(benchmark_results_df["category"] == "small_reward") & (benchmark_results_df["data_source"] == "test") & (benchmark_results_df["algo"] == "DiscreteSAC")]


In [ ]:
benchmark_results_df[(benchmark_results_df["category"] == "large_reward") & (benchmark_results_df["data_source"] == "eICU") & (benchmark_results_df["train_settings"] == "2025_06_09_without_action_penalty_with_weaning_reward_same_parameter_settings_v1_training_log")]


In [ ]:
metrics_large_reward_df

In [ ]:
metrics_train_df_v1 = compute_trajectory_metrics_for_benchmark(train_df, state_cols, action_cols, baseline_cols, min_max_values_guidelines_bin_dict)

In [ ]:
def create_formatted_results_table(benchmark_results_df, category, data_source):
    # Filter the dataframe
    filtered_df = benchmark_results_df[
        (benchmark_results_df["category"] == category) & 
        (benchmark_results_df["data_source"] == data_source)
    ]
    
    # Define the desired order of algorithms
    algo_order = ['DiscreteBC', 'NFQ', 'DQN', 'DoubleDQN', 'DiscreteSAC', 'DiscreteBCQ', 'DiscreteCQL']
    algo_mapping = {
        'DiscreteBC': 'BC',
        'NFQ': 'NFQ', 
        'DQN': 'DQN',
        'DoubleDQN': 'DDQN',
        'DiscreteSAC': 'SAC',
        'DiscreteBCQ': 'BCQ',
        'DiscreteCQL': 'CQL'
    }
    
    # Create the formatted dataframe
    formatted_data = []
    
    for algo in algo_order:
        if algo in filtered_df['algo'].values:
            row_data = filtered_df[filtered_df['algo'] == algo].iloc[0]
            
            # Format each column according to your specifications
            total_reward = f"{row_data['avg_reward']:.2f} ({row_data['std_reward']:.2f})"
            
            meet_extubation = f"{row_data['prop_weaned']*100:.0f}%"
            
            avg_hours = f"{row_data['avg_steps']:.2f} ({row_data['std_steps']:.2f})"
            
            # Handle NaN values for avg_steps_meet_weaning
            if pd.isna(row_data['avg_steps_meet_weaning']) or pd.isna(row_data['std_steps_meet_weaning']):
                avg_hours_to_meet = "NaN (NaN)"
            else:
                avg_hours_to_meet = f"{row_data['avg_steps_meet_weaning']:.2f} ({row_data['std_steps_meet_weaning']:.2f})"
            
            action_diversity = f"{row_data['action_diversity']}"
            
            anomaly_action = f"{row_data['anomaly_action_proportion']*100:.0f}%"
            
            formatted_data.append({
                f"{data_source} set / {category}": algo_mapping[algo],
                "Total Reward": total_reward,
                "Meet Extubation": meet_extubation,
                "Avg hours": avg_hours,
                "Avg hours to meet": avg_hours_to_meet,
                "Action diversity": action_diversity,
                "Anomaly action": anomaly_action
            })
    
    return pd.DataFrame(formatted_data)

In [ ]:
# Usage example:
category = "small_reward" # "large_reward" # "median_reward" # "small_reward"
data_source = "eICU" # "test" # "train"
formatted_table = create_formatted_results_table(benchmark_results_df, category, data_source)
formatted_table

In [ ]:
# Usage example:
category = "small_reward" # "large_reward" # "median_reward" # "small_reward"
data_source = "test" # "test" # "train"
formatted_table = create_formatted_results_table(benchmark_results_df, category, data_source)
formatted_table

In [ ]:
# Usage example:
category = "small_reward" # "large_reward" # "median_reward" # "small_reward"
data_source = "eICU" # "test" # "train"
formatted_table = create_formatted_results_table(benchmark_results_df, category, data_source)
formatted_table

### Integrating with PatientEnvironment and D3RLPY

In [ ]:
# algo_train_dict = {
#     'DQN_train': agent_dqn_train,
#     'DoubleDQN_train': agent_ddqn_train,
#     'DiscreteSAC_train': agent_sac_train,
#     'DiscreteBCQ_train': agent_bcq_train,
#     'DiscreteCQL_train': agent_cql_train
# }
# algo_test_dict = {
#     'DQN_test': agent_dqn_test,
#     'DoubleDQN_test': agent_ddqn_test,
#     'DiscreteSAC_test': agent_sac_test,
#     'DiscreteBCQ_test': agent_bcq_test,
#     'DiscreteCQL_test': agent_cql_test
# }
# algo_eICU_dict = {
#     'DQN_eICU': agent_dqn_eICU,
#     'DoubleDQN_eICU': agent_ddqn_eICU,
#     'DiscreteSAC_eICU': agent_sac_eICU,
#     'DiscreteBCQ_eICU': agent_bcq_eICU,
#     'DiscreteCQL_eICU': agent_cql_eICU
# }
algo_train_dict = {
    'DQN_train': agent_dqn_train,
    'DDQN_train': agent_ddqn_train,
    'SAC_train': agent_sac_train,
    'BCQ_train': agent_bcq_train,
    'CQL_train': agent_cql_train,
    'BC_train': agent_bc_train
}
algo_test_dict = {
    'DQN_test': agent_dqn_test,
    'DDQN_test': agent_ddqn_test,
    'SAC_test': agent_sac_test,
    'BCQ_test': agent_bcq_test,
    'CQL_test': agent_cql_test,
    'BC_test': agent_bc_test
}
algo_eICU_dict = {
    'DQN_eICU': agent_dqn_eICU,
    'DDQN_eICU': agent_ddqn_eICU,
    'SAC_eICU': agent_sac_eICU,
    'BCQ_eICU': agent_bcq_eICU,
    'CQL_eICU': agent_cql_eICU,
    'BC_eICU': agent_bc_eICU
}

In [ ]:
action_counts = train_df[['fio2', 'respiratory_rate_set']].value_counts()
print(action_counts)

In [ ]:
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

In [ ]:
obs = env.reset()
print(obs)

In [ ]:
def obs_format(obs):
    """Reset the environment to an initial state."""
    # Helper function to convert np.int32 to plain integers
    def to_plain_int(value):
        return int(value) if isinstance(value, np.integer) else value

    # Convert all elements in obs to plain integers
    obs = [to_plain_int(x) for x in obs]

    state_len = len(state_cols)
    action_len = len(action_cols)
    baseline_len = len(baseline_cols)

    # Split the data into states, actions, and baselines
    num_states = (len(obs) - baseline_len) // (state_len + action_len)
    states = []
    actions = []
    for i in range(num_states):
        start_idx = i * (state_len + action_len)
        states.append(tuple(obs[start_idx:start_idx + state_len]))
        actions.append(tuple(obs[start_idx + state_len:start_idx + state_len + action_len]))
    # last state is the current state
    current_state = tuple(obs[-(state_len + baseline_len):-baseline_len])
    states.append(current_state)
    # Extract baselines
    baselines = tuple(obs[-baseline_len:])

    # Create the final format
    result = []
    for i in range(num_states + 1):
        # Use the last action if the current action is missing
        action = actions[i] if i < len(actions) else None
        result.append((states[i], action))
    
    # Add the baselines
    result.append(baselines)

    obs = tuple(result)
    return obs

In [ ]:
obs = env.reset()
print(obs)
obs_format(obs)

#### Sample Actions with different Agent algo

In [ ]:
for _ in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    for _ in range(1):
        print(f"obs: {obs_format(obs)}")
        for algo_name, agent in algo_train_dict.items():
            action = agent.predict(np.expand_dims(obs, axis=0))[0]  # Add batch dimension
            print(f"{algo_name}: \t{env.id_to_action[action]}")
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        print()

In [ ]:
for _ in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    for _ in range(1):
        print(f"obs: {obs_format(obs)}")
        for algo_name, agent in algo_test_dict.items():
            action = agent.predict(np.expand_dims(obs, axis=0))[0]  # Add batch dimension
            print(f"{algo_name}: \t{env.id_to_action[action]}")
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        print()

In [ ]:
for _ in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    for _ in range(1):
        print(f"obs: {obs_format(obs)}")
        for algo_name, agent in algo_eICU_dict.items():
            action = agent.predict(np.expand_dims(obs, axis=0))[0]  # Add batch dimension
            print(f"{algo_name}: \t{env.id_to_action[action]}")
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        print()

In [ ]:
# common_init_obs[0]

### Evaluate agents on special init categories

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def evaluate_agents_and_visualize(categories, algo_dict, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs):
    """
    Evaluate all agents across different categories and visualize the results.

    Args:
        categories (dict): Dictionary of categories with initial observations.
        algo_dict (dict): Dictionary of agents to evaluate.
        train_transitions: Training transitions for the environment.
        state_cols: State columns for the environment.
        action_cols: Action columns for the environment.
        baseline_cols: Baseline columns for the environment.
        num_bins_dict: Dictionary of binning information for the environment.
        min_max_values_guidelines_bin_dict: Min-max values for binning.
        obs_hrs: Observation hours for the environment.

    Returns:
        results (dict): Dictionary of results with average rewards for each agent and category.
    """
    results = {agent_name: {} for agent_name in algo_dict.keys()}

    for agent_name, agent in algo_dict.items():
        print(f"Evaluating agent: {agent_name}")
        for category, init_obs_list in categories.items():
            if not init_obs_list:
                print(f"Skipping {category}: No valid initial observations")
                results[agent_name][category] = None
                continue

            env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

            # Evaluate the agent
            episode_returns = []
            for idx in range(min(1000, len(init_obs_list))):
                obs = env.reset(init_obs_list[idx])
                done = False
                total_reward = 0
                while not done:
                    action = agent.predict(np.expand_dims(obs, axis=0))[0]
                    obs, reward, done, _ = env.step(action)
                    total_reward += reward
                episode_returns.append(total_reward)

            if len(episode_returns) == 0:
                print(f"Skipping {category}: No valid episode returns")
                results[agent_name][category] = None
                continue

            avg_reward = np.mean(episode_returns)
            results[agent_name][category] = avg_reward
            print(f"{agent_name} - {category}: Average Total Reward = {avg_reward}")

    # Visualization
    visualize_agents_performance_results(results)

    return results


def visualize_agents_performance_results(results):
    """
    Visualize the results using a heatmap.

    Args:
        results (dict): Dictionary of results with average rewards for each agent and category.
    """
    # Convert results to a DataFrame for easier visualization
    import pandas as pd
    results_df = pd.DataFrame(results).T  # Transpose to have agents as rows and categories as columns

    # Plot heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(results_df, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Average Reward'})
    plt.title("Agent Performance Across Categories")
    plt.xlabel("Categories")
    plt.ylabel("Agents")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


# Example usage
# categories = {
#     'small_reward': small_reward_init_obs,
#     'median_reward': median_reward_init_obs,
#     'large_reward': large_reward_init_obs,
#     'short_to_wean': short_to_wean_init_obs,
#     'median_to_wean': median_to_wean_init_obs,
#     'long_to_wean': long_to_wean_init_obs,
#     'common': common_init_obs
# }

# Filter out None values from init_obs lists
# categories = {k: [obs for obs in v if obs is not None] for k, v in categories.items()}

# Combine all agent dictionaries into one for evaluation
all_agents = {**algo_train_dict, **algo_test_dict, **algo_eICU_dict}

# Evaluate and visualize
results = evaluate_agents_and_visualize(eval_init_obs_categories, all_agents, train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

In [ ]:
# # Load the trained agent (example)
# def get_newest_json(model_dir):
#     json_files = [f for f in os.listdir(model_dir) if f.endswith('.json')]
#     return max(json_files, key=lambda x: os.path.getmtime(os.path.join(model_dir, x)))


# model_dir = "../models"
# os.makedirs(model_dir, exist_ok=True)
# model_path = os.path.join(model_dir, f"{algo}_{data_source}.pkl")


# model_dir = "path_to_your_model"
# params_file = get_newest_json(model_dir)
# algo = d3rlpy.algos.DQN.from_json(os.path.join(model_dir, params_file))
# algo.load_model(os.path.join(model_dir, "model.pt"))


# TODO: Load the model by function, given the algo name and data source
# algo = agent_dqn
# algo = agent_bcq_test
# algo = agent_dqn_train
algo = agent_bcq_train
# algo = naive_agent


# # Evaluate per category
# categories = {
#     'small_reward': small_reward_init_obs,
#     'median_reward': median_reward_init_obs,
#     'large_reward': large_reward_init_obs,
#     'short_to_wean': short_to_wean_init_obs,
#     'median_to_wean': median_to_wean_init_obs,
#     'long_to_wean': long_to_wean_init_obs,
#     'common': common_init_obs
# }

# results = {}
# for category, init_obs_list in categories.items():
#     env = FixedInitialObsEnv(initial_obs_list=init_obs_list, base_env_class=PatientEnvironment)
#     # Run 1000 episodes or as many as init_obs_list allows
#     avg_reward = d3rlpy.metrics.evaluate_on_environment(env, n_trials=min(1000, len(init_obs_list)))(algo)
#     results[category] = avg_reward
#     print(f"{category}: Average Total Reward = {avg_reward}")

# # Alternative: Per patient evaluation
# for category, init_obs_list in categories.items():
#     stay_ids = [metrics_df[metrics_df['stay_id'].isin(patients)]['stay_id'] for patients in [small_reward_patients, median_reward_patients, large_reward_patients, short_to_wean_patients, median_to_wean_patients, long_to_wean_patients]][list(categories.keys()).index(category)]
#     rewards = [evaluate_on_patient(PatientEnvironment, sid, algo) for sid in stay_ids]
#     avg_reward = np.mean(rewards)
#     results[category] = avg_reward
#     print(f"{category}: Average Total Reward = {avg_reward}")

import d3rlpy
import numpy as np

# Define categories (from previous response)
# categories = {
#     'small_reward': small_reward_init_obs,
#     'median_reward': median_reward_init_obs,
#     'large_reward': large_reward_init_obs,
#     'short_to_wean': short_to_wean_init_obs,
#     'median_to_wean': median_to_wean_init_obs,
#     'long_to_wean': long_to_wean_init_obs,
#     'common': common_init_obs
# }

# # Filter out None values from init_obs lists
# categories = {k: [obs for obs in v if obs is not None] for k, v in categories.items()}

# Evaluation with FixedInitialObsEnv
results = {}
for category, init_obs_list in eval_init_obs_categories.items():
    if not init_obs_list:
        print(f"Skipping {category}: No valid initial observations")
        results[category] = None
        continue

    env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict, obs_hrs)

    # Run up to 1000 episodes or as many as init_obs_list allows
    # avg_reward = d3rlpy.metrics.scorer.evaluate_on_environment(env, n_trials=min(1000, len(init_obs_list)))(algo)
    # implementation of evaluate_on_environment
    episode_returns = []
    for idx in range(min(1000, len(init_obs_list))):
        # print(f"init state: {init_obs_list[idx]}")
        obs = env.reset(init_obs_list[idx])
        done = False
        total_reward = 0
        while not done:
            action = algo.predict(np.expand_dims(obs, axis=0))[0]
            # print(f"action: {action}")
            # print(f"action: {env.id_to_action[action]}")
            obs, reward, done, _ = env.step(action)
            total_reward += reward
        episode_returns.append(total_reward)
    avg_reward = np.mean(episode_returns)
    if len(episode_returns) == 0:
        print(f"Skipping {category}: No valid episode returns")
        results[category] = None
        continue
    results[category] = avg_reward
    print(f"{category}: Average Total Reward = {avg_reward}")

In [ ]:
for category, init_obs_list in eval_init_obs_categories.items():
    print(len(init_obs_list))

In [ ]:
results


In [ ]:
results

In [ ]:
# naive_agent performance
results

In [ ]:
# bcq agent performance
results

In [ ]:
# bcq agent test performance
results

In [ ]:
results

In [ ]:
results

In [ ]:
results

In [ ]:
action_counts = eICU_disc[['fio2', 'respiratory_rate_set']].value_counts()
print(action_counts)

In [ ]:
action_counts = train_df[['fio2', 'respiratory_rate_set']].value_counts()
print(action_counts)

## Generates trajectories by interacting with different offline policy doctor agents

### naive agent interact with patient environment

In [ ]:
import pandas as pd
import random

def interact_with_environment(env, agent, obs_hrs, num_steps=10, stay_id=None):
    """
    Interact with the PatientEnvironment using the NaiveAgent.

    Args:
        env (PatientEnvironment): The environment to interact with.
        agent (NaiveAgent): The naive agent.
        obs_hrs (int): Number of observation hours (historical + current).
        num_steps (int): Number of steps to interact with the environment.
        stay_id (int, optional): The stay_id for the trajectory. If None, a random stay_id will be generated.

    Returns:
        list: A list of dictionaries representing the trajectory.
    """
    trajectory = []
    obs = env.reset()
    stay_id = stay_id if stay_id is not None else random.randint(10000000, 99999999)  # Generate random stay_id
    
    # Observation structure: [s_1(3), a_1(2), s_2(3), a_2(2), ..., s_current(3), baseline(2)]
    state_size = len(env.state_cols)  # 3
    action_size = len(env.action_cols)  # 2
    baseline_size = len(env.baseline_cols)  # 2
    
    # Extract last action (a_(obs_hrs-1)) from obs
    last_action_start = (obs_hrs - 1) * (state_size + action_size) - action_size  # Index of last action
    last_action = tuple(obs[last_action_start:last_action_start + action_size])  # e.g., obs[8:10] for obs_hrs=3
    
    # At t=0, include historical states and current state
    for h in range(obs_hrs):  # h=0 for s_1, h=1 for s_2, h=2 for s_current
        state_start = h * (state_size + action_size)  # 0 for s_1, 5 for s_2, 10 for s_current
        state = obs[state_start:state_start + state_size]  # Extract state
        # Action: use a_1 for s_1, a_2 for s_2, last_action for s_current
        action = tuple(obs[state_start + state_size:state_start + state_size + action_size]) if h < obs_hrs - 1 else last_action
        # Baseline variables
        baseline_start = obs.size - baseline_size  # Last 2 indices
        baseline = obs[baseline_start:baseline_start + baseline_size]  # e.g., obs[13:15]
        
        trajectory.append({
            'stay_id': stay_id,
            'hours_in': h,  # Start from 0, e.g., 0, 1, 2 for obs_hrs=3
            **{col: state[i] for i, col in enumerate(env.state_cols)},  # State variables
            **{col: action[i] for i, col in enumerate(env.action_cols)},  # Action variables
            **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}  # Baseline variables
        })
    
    # Continue for remaining steps
    for t in range(obs_hrs, num_steps):
        # Convert observation to state_with_pre_state_action_baseline key
        state_with_pre_state_action_baseline = tuple(obs.tolist())
        
        # Sample action from the naive agent
        action_tuple = tuple(agent.sample_action(state_with_pre_state_action_baseline, last_action))
        action = env.action_to_id.get(action_tuple, 0)  # Map action tuple to action ID
        
        # Step in the environment
        next_obs, reward, done, _ = env.step(action)
        
        # Extract current state and baseline
        state_start = (obs_hrs - 1) * (state_size + action_size)  # Index of s_current, e.g., 10 for obs_hrs=3
        state = next_obs[state_start:state_start + state_size]  # s_current
        baseline_start = next_obs.size - baseline_size  # Last 2 indices
        baseline = next_obs[baseline_start:baseline_start + baseline_size]  # baseline
        
        # Store trajectory
        trajectory.append({
            'stay_id': stay_id,
            'hours_in': t,  # Continue from obs_hrs, e.g., 3, 4, ... for obs_hrs=3
            **{col: state[i] for i, col in enumerate(env.state_cols)},  # State variables
            **{col: action_tuple[i] for i, col in enumerate(env.action_cols)},  # Action variables
            **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}  # Baseline variables
        })
        
        # Update observation and last action
        obs = next_obs
        last_action = action_tuple
        
        if done:
            break
    
    return trajectory

def generate_trajectories_as_dataframe(env, agent, num_trajectories=10, num_steps=10):
    """
    Generate a target number of trajectories and format them as a DataFrame.

    Args:
        env (PatientEnvironment): The environment to interact with.
        agent (NaiveAgent): The naive agent.
        num_trajectories (int): The number of trajectories to generate.
        num_steps (int): The number of steps per trajectory.

    Returns:
        pd.DataFrame: A DataFrame containing all trajectories.
    """
    all_trajectories = []
    
    for num in range(num_trajectories):
        trajectory = interact_with_environment(env, agent, obs_hrs, num_steps, stay_id=num)
        all_trajectories.extend(trajectory)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_trajectories)
    return df

In [ ]:
# Example initialization
naive_agent = NaiveAgent(train_df, obs_hrs, state_cols, action_cols, baseline_cols)

In [ ]:
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict)

In [ ]:
# Interact with the environment
trajectory = interact_with_environment(env, naive_agent, obs_hrs, num_steps=120)
# for step in trajectory:
#     print(step)

In [ ]:
env.reset()

In [ ]:
obs = env.reset()
print(obs)
print(obs[-7:-5])

In [ ]:
naive_agent.naive_agent_dict

In [ ]:
# Generate 10 trajectories, each with 20 steps
df_trajectories = generate_trajectories_as_dataframe(env, naive_agent, num_trajectories=10000, num_steps=120)

# Display the resulting DataFrame
df_trajectories.head()

In [ ]:
df_trajectories

In [ ]:
df_trajectories.describe()

In [ ]:
df_trajectories['respiratory_rate_set'].unique()

In [ ]:
bin_edges_dict

#### Evaluate naive agent total reward

In [ ]:
def run_multiple_episodes(env, agent, obs_hrs, num_episodes=100, num_steps=10):
    """
    Run multiple episodes using the NaiveAgent in the PatientEnvironment and collect total rewards.

    Args:
        env (PatientEnvironment): The environment to interact with.
        agent (NaiveAgent): The naive agent.
        obs_hrs (int): Number of observation hours (historical + current).
        num_episodes (int): Number of episodes to run.
        num_steps (int): Number of steps per episode.

    Returns:
        list: A list of total rewards for each episode.
        list: A list of trajectories for each episode.
    """
    episode_rewards = []
    all_trajectories = []
    
    for episode in range(num_episodes):
        # Generate a random stay_id for this episode
        stay_id = random.randint(10000000, 99999999)
        
        # Reset the environment for a new episode
        obs = env.reset()
        trajectory = []
        episode_reward = 0
        
        # Get initial state and baseline information
        state_size = len(env.state_cols)  # 3
        action_size = len(env.action_cols)  # 2
        baseline_size = len(env.baseline_cols)  # 2
        
        # Extract last action from obs
        last_action_start = (obs_hrs - 1) * (state_size + action_size) - action_size
        last_action = tuple(obs[last_action_start:last_action_start + action_size])
        
        # Record initial observations (historical + current states)
        for h in range(obs_hrs):
            state_start = h * (state_size + action_size)
            state = obs[state_start:state_start + state_size]
            action = tuple(obs[state_start + state_size:state_start + state_size + action_size]) if h < obs_hrs - 1 else last_action
            baseline_start = obs.size - baseline_size
            baseline = obs[baseline_start:baseline_start + baseline_size]
            
            trajectory.append({
                'stay_id': stay_id,
                'hours_in': h,
                **{col: state[i] for i, col in enumerate(env.state_cols)},
                **{col: action[i] for i, col in enumerate(env.action_cols)},
                **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
            })
        
        # Continue for remaining steps
        for t in range(obs_hrs, num_steps):
            # Convert observation to state_with_pre_state_action_baseline
            state_with_pre_state_action_baseline = tuple(obs.tolist())
            
            # Sample action from the naive agent
            action_tuple = tuple(agent.sample_action(state_with_pre_state_action_baseline, last_action))
            action = env.action_to_id.get(action_tuple, 0)
            
            # Step in the environment
            next_obs, reward, done, _ = env.step(action)
            episode_reward += reward  # Accumulate reward
            
            # Extract current state and baseline
            state_start = (obs_hrs - 1) * (state_size + action_size)
            state = next_obs[state_start:state_start + state_size]
            baseline_start = next_obs.size - baseline_size
            baseline = next_obs[baseline_start:baseline_start + baseline_size]
            
            # Store trajectory
            trajectory.append({
                'stay_id': stay_id,
                'hours_in': t,
                **{col: state[i] for i, col in enumerate(env.state_cols)},
                **{col: action_tuple[i] for i, col in enumerate(env.action_cols)},
                **{col: baseline[i] for i, col in enumerate(env.baseline_cols)}
            })
            
            # Update observation and last action
            obs = next_obs
            last_action = action_tuple
            
            if done:
                break
        
        # Store the total reward and trajectory for this episode
        episode_rewards.append(episode_reward)
        all_trajectories.append(trajectory)
        
        # Optional: Print progress
        if (episode + 1) % 10 == 0:
            print(f"Completed {episode + 1}/{num_episodes} episodes")
    
    return episode_rewards, all_trajectories

In [ ]:
# Example usage:
# Initialize your agent and environment
naive_agent = NaiveAgent(train_df, obs_hrs, state_cols, action_cols, baseline_cols)
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict)

# Run multiple episodes
rewards, trajectories = run_multiple_episodes(env, naive_agent, obs_hrs=3, num_episodes=100, num_steps=120)

# Print summary statistics
print(f"Average reward across episodes: {sum(rewards) / len(rewards):.4f}")
print(f"Min reward: {min(rewards):.4f}, Max reward: {max(rewards):.4f}")

In [ ]:
# Example usage:
# Initialize your agent and environment
naive_agent = NaiveAgent(train_df, obs_hrs, state_cols, action_cols, baseline_cols)
env = PatientEnvironment(train_transitions, state_cols, action_cols, baseline_cols, num_bins_dict, min_max_values_guidelines_bin_dict)

# Run multiple episodes
rewards, trajectories = run_multiple_episodes(env, naive_agent, obs_hrs=3, num_episodes=100, num_steps=120)

# Print summary statistics
print(f"Average reward across episodes: {sum(rewards) / len(rewards):.4f}")
print(f"Min reward: {min(rewards):.4f}, Max reward: {max(rewards):.4f}")

In [ ]:
# average len of trajectories
avg_len = sum(len(traj) for traj in trajectories) / len(trajectories)
print(f"Average length of trajectories: {avg_len:.2f}")

#### Convert generated trajectories from discrete to orignal

In [ ]:
def convert_discrete_to_original(df, bin_edges_dict, columns):
    """
    Convert discrete values in the DataFrame back to their original ranges.

    Args:
        df (pd.DataFrame): The DataFrame containing discrete values.
        bin_edges_dict (dict): Dictionary containing bin edges for each feature.
        columns (list): List of columns to convert.

    Returns:
        pd.DataFrame: A new DataFrame with the converted values.
    """
    def map_bin_to_range(bin_index, feature, bin_edges_dict):
        """
        Map a discrete bin index to its original range and return the mean of the range.
        Args:
            bin_index (int): The bin index.
            feature (str): The feature name.
            bin_edges_dict (dict): Dictionary containing bin edges for each feature.
        Returns:
            int: The mean of the range, rounded for better visualization.
        """
        if feature not in bin_edges_dict:
            return "N/A"
        edges = bin_edges_dict[feature]
        if bin_index < 0 or bin_index >= len(edges) - 1:
            return "Invalid"
        lower = edges[bin_index]
        upper = edges[bin_index + 1]
        if feature == "fio2":
            return int(round(lower))  # For fio2, return the lower bound
        else:
            return int(round((lower + upper) / 2))  # Return the mean of the range

    # Create a copy of the DataFrame to avoid modifying the original
    df_converted = df.copy()

    # Apply the mapping to each column
    for col in columns:
        if col in bin_edges_dict:
            df_converted[col] = df[col].apply(lambda x: map_bin_to_range(x, col, bin_edges_dict))

    return df_converted

In [ ]:
# Define the columns to convert
columns_to_convert = state_cols + action_cols + ["age"]
# Convert the discrete values back to their original ranges
df_converted = convert_discrete_to_original(df_trajectories, bin_edges_dict, columns_to_convert)

# Display the converted DataFrame
df_converted.head()

In [ ]:
df_converted

In [ ]:
df_trajectories["respiratory_rate_set"].unique()

In [ ]:
df_converted["respiratory_rate_set"].unique()

In [ ]:
bin_edges_dict

In [ ]:
# save the DataFrame to a CSV file
df_trajectories.to_csv(f'{mimic_iv_prefix_path}/naive_agent_generated_traj_disc_larger_24_hr_10000_update.csv', index=False)
df_converted.to_csv(f'{mimic_iv_prefix_path}/naive_agent_generated_traj_cont_larger_24_hr_10000_update.csv', index=False)

In [ ]:
len(df_converted["stay_id"].unique())

## Plotly: Visualizes trajectories with transition action options

### Create transitin history for Plotly

In [ ]:
# ================================
# Interactive Trajectory Visualization with Dash & Plotly
# Updated to use 2 history states + 1 current state
# ================================

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Function to create transition history from the discretized DataFrame
# Modified to use 2 history states + 1 current state
def create_transition_history(df, n_history, state_cols, action_cols, baseline_cols):
    """
    Create a transition history DataFrame with state, action, and baseline variables.
    Now includes current state in the history window.

    Args:
        df (pd.DataFrame): Input DataFrame.
        n_history (int): Number of historical observations to include (excluding current).
        state_cols (list): List of state variable columns.
        action_cols (list): List of action variable columns.
        baseline_cols (list): List of baseline variable columns.

    Returns:
        pd.DataFrame: Transition history DataFrame.
    """
    # Ensure the DataFrame is sorted by stay_id and hours_in
    df = df.sort_values(['stay_id', 'hours_in']).reset_index(drop=True)
    
    # Create history columns for state and action variables (only for n_history previous periods)
    for i in range(1, n_history + 1):
        for col in state_cols + action_cols:
            df[f"{col}_-{i}"] = df.groupby('stay_id')[col].shift(i)
    
    # Drop rows without full history
    df = df.dropna(subset=[f"{state_cols[0]}_-{i}" for i in range(1, n_history + 1)])
    
    # Create history tuples for previous state and action
    df['S_hist'] = df[[f"{col}_-{i}" for col in state_cols for i in range(1, n_history + 1)]].apply(lambda row: tuple(row), axis=1)
    df['A_hist'] = df[[f"{col}_-{i}" for col in action_cols for i in range(1, n_history + 1)]].apply(lambda row: tuple(row), axis=1)
    
    # Current state, action, and next state
    df['S_t'] = df[state_cols].apply(lambda row: tuple(row), axis=1)
    df['A_t'] = df[action_cols].apply(lambda row: tuple(row), axis=1)
    df['S_t+1'] = df.groupby('stay_id')['S_t'].shift(-1)
    
    # Add baseline variables
    df['baseline'] = df[baseline_cols].apply(lambda row: tuple(row), axis=1)
    
    return df.dropna(subset=['S_t+1'])

In [ ]:
# Build the transition history DataFrame using the discretized data
# Now using n_history = 2 for 2 previous hours + current hour
transition_history_df = create_transition_history(mimic_iv_disc.copy(), obs_hrs-1, state_cols, action_cols, baseline_cols)

In [ ]:
transition_history_mimic_iv_disc_df = transition_history_df.copy()

In [ ]:
algo = "DiscreteCQL" # "naive_agent" # "DiscreteCQL"

In [ ]:
pd.read_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_train_epoch19.csv')

In [ ]:
# read models/training_log/eval_trajectories_DiscreteSAC_train_epoch0.csv models/training_log/eval_trajectories_DoubleDQN_train_epoch0.csv
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_train_epoch19.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
if algo == "naive_agent":
    transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log/naive_agent/eval_trajectories_{algo}_train.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols) # Naive Agent
else:
    transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_train_epoch19.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)

transition_history_df

In [ ]:
mimic_iv_disc

In [ ]:
# 10,5,4,8,5,1,2
# 10,6,3,4,6,1,2
# mimic_iv_disc[(mimic_iv_disc["heart_rate"] == 10) & (mimic_iv_disc["resp_rate"] == 5) & (mimic_iv_disc["spo2"] == 4) & (mimic_iv_disc["fio2"] == 8) & (mimic_iv_disc["respiratory_rate_set"] == 5) & (mimic_iv_disc["gender_M"] == 1) & (mimic_iv_disc["age"] == 2)]
# 0_small_reward_88
# 7,5,1,8,4,1,2
# 9,3,4,6,5,1,2
# mimic_iv_disc[(mimic_iv_disc["heart_rate"] == 7) & (mimic_iv_disc["resp_rate"] == 5) & (mimic_iv_disc["spo2"] == 1) & (mimic_iv_disc["fio2"] == 8) & (mimic_iv_disc["respiratory_rate_set"] == 4) & (mimic_iv_disc["gender_M"] == 1) & (mimic_iv_disc["age"] == 2)]
mimic_iv_disc[(mimic_iv_disc["heart_rate"] == 6) & (mimic_iv_disc["resp_rate"] == 5) & (mimic_iv_disc["spo2"] == 1) & (mimic_iv_disc["fio2"] == 8) & (mimic_iv_disc["respiratory_rate_set"] == 3)]

In [ ]:
# mimic_iv_disc[mimic_iv_disc["stay_id"] == 38297623] # this is small_reward_0
# mimic_iv_disc[mimic_iv_disc["stay_id"] == 31726457] # this is small_reward_88
mimic_iv_disc[mimic_iv_disc["stay_id"] == 37768645] # this is small_reward_66

In [ ]:
mimic_iv_disc[mimic_iv_disc["stay_id"] == 30139485]


### Add background for guidelines

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Helper functions for guideline checking
def is_within_guidelines(row, state_cols, action_cols, guideline_dict):
    state_dict = dict(zip(state_cols, row['S_t']))
    action_dict = dict(zip(action_cols, row['A_t']))
    all_vars = state_cols + action_cols
    for var in all_vars:
        if var in guideline_dict:
            limits = guideline_dict[var]
            if limits['min'] == 0 and limits['max'] == 0:
                continue
            if limits['max'] <= 15:  # Assuming bin indices for max <= 15
                value = state_dict[var] if var in state_cols else action_dict[var]
                if not (limits['min'] <= value <= limits['max']):
                    return False
    return True

def count_not_within_guidelines(tup, cols, guideline_dict):
    tup_dict = dict(zip(cols, tup))
    count = 0
    for var in cols:
        if var in guideline_dict:
            limits = guideline_dict[var]
            if limits['min'] == 0 and limits['max'] == 0:
                continue
            if limits['max'] <= 15:
                if not (limits['min'] <= tup_dict[var] <= limits['max']):
                    count += 1
    return count

# Functions for actions and outcomes
def get_possible_actions(history_state, history_action, current_state):
    similar_states = transition_history_df[(transition_history_df['S_hist'] == history_state) &
                                           (transition_history_df['A_hist'] == history_action) &
                                           (transition_history_df['S_t'] == current_state)]
    if similar_states.empty:
        return []
    action_counts = similar_states.groupby('A_t').size().reset_index(name='count')
    action_patients = similar_states.groupby(['A_t', 'stay_id']).size().reset_index()
    distinct_patients = action_patients.groupby('A_t').size().reset_index(name='patient_count')
    action_stats = pd.merge(action_counts, distinct_patients, on='A_t')
    return [{'action': row['A_t'], 'count': row['count'], 'patient_count': row['patient_count']}
            for _, row in action_stats.iterrows()]

def get_action_outcomes(history_state, history_action, current_state, selected_action, top_n=5):
    similar_states = transition_history_df[(transition_history_df['S_hist'] == history_state) &
                                           (transition_history_df['A_hist'] == history_action) &
                                           (transition_history_df['S_t'] == current_state) &
                                           (transition_history_df['A_t'] == selected_action)]
    if similar_states.empty:
        return []
    total_occurrences = similar_states.groupby('S_t+1').size().reset_index(name='total_count')
    distinct_patients = similar_states.groupby('S_t+1')['stay_id'].nunique().reset_index(name='distinct_patients')
    outcomes = pd.merge(total_occurrences, distinct_patients, on='S_t+1')
    outcomes = outcomes.sort_values('total_count', ascending=False).head(top_n)
    return [{'S_t+1': row['S_t+1'], 'total_count': row['total_count'], 'distinct_patients': row['distinct_patients']}
            for _, row in outcomes.iterrows()]

# Formatting functions
def format_tuple(t):
    return str(tuple(int(x) for x in t))

def format_tuple_grouped(data, group_size):
    if len(data) % group_size != 0:
        raise ValueError("Data length must be divisible by group size.")
    num_groups = len(data) // group_size
    grouped = [tuple(data[i::num_groups]) for i in range(num_groups)]
    grouped.reverse()
    return ", ".join(str(group) for group in grouped)

# Set up Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Patient Trajectory Visualization"),
    dcc.Dropdown(
        id='stay-id-dropdown',
        options=[{'label': str(sid), 'value': sid} for sid in sorted(transition_history_df['stay_id'].unique())],
        value=sorted(transition_history_df['stay_id'].unique())[0],
        clearable=False
    ),
    dcc.Graph(id='trajectory-plot'),
    html.Div(
        id='action-details',
        style={
            'margin-top': '20px',
            'backgroundColor': 'white',
            'padding': '15px',
            'border': '1px solid #ccc',
            'borderRadius': '5px',
            'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
            'fontFamily': 'Arial, sans-serif',
            'color': '#333'
        }
    )
], style={'backgroundColor': 'white', 'padding': '20px'})

# Callback to update trajectory plot
@app.callback(
    Output('trajectory-plot', 'figure'),
    Input('stay-id-dropdown', 'value')
)
def update_plot(selected_stay_id):
    patient_df = transition_history_df[transition_history_df['stay_id'] == selected_stay_id].copy()
    patient_df['within_guidelines'] = patient_df.apply(
        lambda row: is_within_guidelines(row, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        axis=1
    )
    
    fig = px.line(
        patient_df,
        x='hours_in',
        y=state_cols,
        title=f'Trajectory for stay_id {selected_stay_id}',
        labels={'value': 'Vital Sign Value', 'variable': 'Vital Sign', 'hours_in': 'Time (hours)'}
    )
    fig.update_traces(mode='lines+markers')
    
    hover_texts = []
    for _, row in patient_df.iterrows():
        hover_text = (
            f"<b>Hour:</b> {row['hours_in']}<br>"
            f"<b>Current State:</b> {format_tuple(row['S_t'])}<br>"
            f"<b>Current Action:</b> {format_tuple(row['A_t'])}<br>"
        )
        hover_texts.append(hover_text)
    
    for trace in fig.data:
        trace.customdata = patient_df[['stay_id', 'S_hist', 'A_hist', 'S_t', 'A_t', 'hours_in']].to_dict('records')
        trace.text = hover_texts
        trace.hovertemplate = '%{text}<extra></extra>'
    
    shapes = []
    for idx, row in patient_df.iterrows():
        t = row['hours_in']
        color = 'rgba(144, 238, 144, 0.3)' if row['within_guidelines'] else 'rgba(255, 99, 71, 0.3)'
        shapes.append(dict(
            type="rect",
            x0=t - 0.5,
            y0=0,
            x1=t + 0.5,
            y1=1,
            xref="x",
            yref="paper",
            fillcolor=color,
            line_width=0,
            layer="below"
        ))
    
    fig.update_layout(
        shapes=shapes,
        clickmode='event+select',
        template='plotly_white'
    )
    return fig

# Callback to update action details
@app.callback(
    Output('action-details', 'children'),
    Input('trajectory-plot', 'clickData')
)
def display_action_details(clickData):
    if not clickData:
        return html.P("Click a point on the plot to see detailed transition outcomes.")
    
    point = clickData['points'][0]
    data = point['customdata']
    history_state = tuple(data['S_hist'])
    history_action = tuple(data['A_hist'])
    current_state = tuple(data['S_t'])
    
    state_group_size = len(current_state)
    action_group_size = len(data['A_t'])
    
    details = [
        html.H3(f"Details for stay_id {data['stay_id']} at Hour {data['hours_in']}"),
        html.P(f"Current State: {format_tuple(current_state)}"),
        html.P(f"Current Action: {format_tuple(data['A_t'])}"),
        html.P(f"History States: {format_tuple_grouped(history_state, state_group_size)}"),
        html.P(f"History Actions: {format_tuple_grouped(history_action, action_group_size)}")
    ]
    
    possible_actions = get_possible_actions(history_state, history_action, current_state)
    if possible_actions:
        action_panels = []
        for act in possible_actions:
            action_tuple = act['action']
            count_not_satisfy = count_not_within_guidelines(action_tuple, action_cols, min_max_values_guidelines_bin_dict)
            bg_color = 'lightgreen' if count_not_satisfy == 0 else 'lightsalmon'
            action_text = html.Span(
                f"Action {format_tuple(action_tuple)}",
                style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
            )
            
            outcomes = get_action_outcomes(history_state, history_action, current_state, action_tuple)
            outcome_lis = []
            for outcome in outcomes:
                state_tuple = outcome['S_t+1']
                count_not_satisfy = count_not_within_guidelines(state_tuple, state_cols, min_max_values_guidelines_bin_dict)
                bg_color = 'lightgreen' if count_not_satisfy == 0 else 'orange'
                state_text = html.Span(
                    f"State {format_tuple(state_tuple)}",
                    style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
                )
                outcome_lis.append(
                    html.Li([state_text, f" ({outcome['total_count']} occurrences across {outcome['distinct_patients']} patients)"])
                )
            
            action_panels.append(
                html.Div([
                    html.H4([action_text, f" ({act['count']} occurrences across {act['patient_count']} patients)"]),
                    html.Ul(outcome_lis)
                ], style={'border': '1px solid #ddd', 'padding': '10px', 'margin': '5px'})
            )
        details.append(html.Div(action_panels))
    else:
        details.append(html.P("No available action transitions for this state combination."))
    
    return html.Div(details)

# Run the app
# app.run(debug=True)

### With baseline bottom

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Patient Trajectory Visualization"),
    dcc.Dropdown(
        id='stay-id-dropdown',
        options=[{'label': str(sid), 'value': sid} for sid in sorted(transition_history_df['stay_id'].unique())],
        value=sorted(transition_history_df['stay_id'].unique())[0],
        clearable=False
    ),
    # Added RadioItems for baseline toggle
    dcc.RadioItems(
        id='consider-baseline',
        options=[
            {'label': 'Consider Baseline (Gender, Age)', 'value': 'yes'},
            {'label': 'Ignore Baseline', 'value': 'no'}
        ],
        value='no',  # Default to ignoring baseline
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),
    dcc.Graph(id='trajectory-plot'),
    html.Div(
        id='action-details',
        style={
            'margin-top': '20px',
            'backgroundColor': 'white',
            'padding': '15px',
            'border': '1px solid #ccc',
            'borderRadius': '5px',
            'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
            'fontFamily': 'Arial, sans-serif',
            'color': '#333'
        }
    )
], style={'backgroundColor': 'white', 'padding': '20px'})

def get_range_str(feat, bin_index):
    if feat not in bin_edges_dict: # TODO: set as corresponding bin edges strategy
        return f"{feat}: N/A"
    edges = bin_edges_dict[feat]
    bin_index = int(bin_index)
    if bin_index < 0 or bin_index >= len(edges) - 1:
        return f"{feat}: Invalid bin"
    lower = edges[bin_index]
    upper = edges[bin_index + 1]
    return f"{feat}: ({lower:.1f} ~ {upper:.1f})"

# Updated update_plot function (only showing the modified part)
@app.callback(
    Output('trajectory-plot', 'figure'),
    Input('stay-id-dropdown', 'value')
)
def update_plot(selected_stay_id):
    patient_df = transition_history_df[transition_history_df['stay_id'] == selected_stay_id].copy()
    patient_df['within_guidelines'] = patient_df.apply(
        lambda row: is_within_guidelines(row, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        axis=1
    )
    
    fig = px.line(
        patient_df,
        x='hours_in',
        y=state_cols,
        title=f'Trajectory for stay_id {selected_stay_id}',
        labels={'value': 'Vital Sign Value', 'variable': 'Vital Sign', 'hours_in': 'Time (hours)'}
    )
    fig.update_traces(mode='lines+markers')
    
    # Updated hover text with ranges
    hover_texts = []
    for _, row in patient_df.iterrows():
        state_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(state_cols, row['S_t'])]
        action_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(action_cols, row['A_t'])]
        hover_text = (
            f"<b>Hour:</b> {row['hours_in']}<br>"
            f"<b>Current State:</b> {format_tuple(row['S_t'])}<br>"
            + "<br>".join(state_ranges) + "<br>"
            f"<b>Current Action:</b> {format_tuple(row['A_t'])}<br>"
            + "<br>".join(action_ranges)
        )
        hover_texts.append(hover_text)
    
    for trace in fig.data:
        trace.customdata = patient_df[['stay_id', 'S_hist', 'A_hist', 'S_t', 'A_t', 'hours_in', 'baseline']].to_dict('records')
        trace.text = hover_texts
        trace.hovertemplate = '%{text}<extra></extra>'
    
    # [Rest of the function remains unchanged: shapes, layout, etc.]
    shapes = []
    for idx, row in patient_df.iterrows():
        t = row['hours_in']
        color = 'rgba(144, 238, 144, 0.3)' if row['within_guidelines'] else 'rgba(255, 99, 71, 0.3)'
        shapes.append(dict(
            type="rect",
            x0=t - 0.5,
            y0=0,
            x1=t + 0.5,
            y1=1,
            xref="x",
            yref="paper",
            fillcolor=color,
            line_width=0,
            layer="below"
        ))
    
    fig.update_layout(
        shapes=shapes,
        clickmode='event+select',
        template='plotly_white'
    )
    return fig

def get_possible_actions(history_state, history_action, current_state, baseline=None):
    # Base filters
    filters = (
        (transition_history_df['S_hist'] == history_state) &
        (transition_history_df['A_hist'] == history_action) &
        (transition_history_df['S_t'] == current_state)
    )
    # Add baseline filter if provided
    if baseline is not None:
        filters &= (transition_history_df['baseline'] == baseline)
    
    similar_states = transition_history_df[filters]
    if similar_states.empty:
        return []
    
    action_counts = similar_states.groupby('A_t').size().reset_index(name='count')
    action_patients = similar_states.groupby(['A_t', 'stay_id']).size().reset_index()
    distinct_patients = action_patients.groupby('A_t').size().reset_index(name='patient_count')
    action_stats = pd.merge(action_counts, distinct_patients, on='A_t')
    return [{'action': row['A_t'], 'count': row['count'], 'patient_count': row['patient_count']}
            for _, row in action_stats.iterrows()]

def get_action_outcomes(history_state, history_action, current_state, selected_action, baseline=None, top_n=5):
    # Base filters
    filters = (
        (transition_history_df['S_hist'] == history_state) &
        (transition_history_df['A_hist'] == history_action) &
        (transition_history_df['S_t'] == current_state) &
        (transition_history_df['A_t'] == selected_action)
    )
    # Add baseline filter if provided
    if baseline is not None:
        filters &= (transition_history_df['baseline'] == baseline)
    
    similar_states = transition_history_df[filters]
    if similar_states.empty:
        return []
    
    total_occurrences = similar_states.groupby('S_t+1').size().reset_index(name='total_count')
    distinct_patients = similar_states.groupby('S_t+1')['stay_id'].nunique().reset_index(name='distinct_patients')
    outcomes = pd.merge(total_occurrences, distinct_patients, on='S_t+1')
    outcomes = outcomes.sort_values('total_count', ascending=False).head(top_n)
    return [{'S_t+1': row['S_t+1'], 'total_count': row['total_count'], 'distinct_patients': row['distinct_patients']}
            for _, row in outcomes.iterrows()]

@app.callback(
    Output('action-details', 'children'),
    [Input('trajectory-plot', 'clickData'),
     Input('consider-baseline', 'value')]
)


# order by action occurrences
def display_action_details(clickData, consider_baseline_value):
    if not clickData:
        return html.P("Click a point on the plot to see detailed transition outcomes.")
    
    point = clickData['points'][0]
    data = point['customdata']
    history_state = tuple(data['S_hist'])
    history_action = tuple(data['A_hist'])
    current_state = tuple(data['S_t'])
    
    state_group_size = len(current_state)
    action_group_size = len(data['A_t'])
    
    # Determine whether to consider baseline
    consider_baseline = (consider_baseline_value == 'yes')
    baseline = tuple(data['baseline']) if consider_baseline else None
    
    # Build details with baseline consideration status
    details = [
        html.H3(f"Details for stay_id {data['stay_id']} at Hour {data['hours_in']}"),
        html.P(f"Considering baseline: {'Yes' if consider_baseline else 'No'}"),
        html.P(f"Current State: {format_tuple(current_state)}"),
        html.P(f"Current Action: {format_tuple(data['A_t'])}"),
        html.P(f"History States: {format_tuple_grouped(history_state, state_group_size)}"),
        html.P(f"History Actions: {format_tuple_grouped(history_action, action_group_size)}")
    ]
    
    # Get possible actions and outcomes with baseline parameter
    possible_actions = get_possible_actions(history_state, history_action, current_state, baseline)
    
    # Sort possible_actions by 'count' in descending order
    possible_actions = sorted(possible_actions, key=lambda x: x['count'], reverse=True)
    
    if possible_actions:
        action_panels = []
        for act in possible_actions:
            action_tuple = act['action']
            count_not_satisfy = count_not_within_guidelines(action_tuple, action_cols, min_max_values_guidelines_bin_dict)
            bg_color = 'lightgreen' if count_not_satisfy == 0 else 'lightsalmon'
            action_text = html.Span(
                f"Action {format_tuple(action_tuple)}",
                style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
            )
            
            outcomes = get_action_outcomes(history_state, history_action, current_state, action_tuple, baseline)
            outcome_lis = []
            for outcome in outcomes:
                state_tuple = outcome['S_t+1']
                count_not_satisfy = count_not_within_guidelines(state_tuple, state_cols, min_max_values_guidelines_bin_dict)
                bg_color = 'lightgreen' if count_not_satisfy == 0 else 'orange'
                state_text = html.Span(
                    f"State {format_tuple(state_tuple)}",
                    style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
                )
                outcome_lis.append(
                    html.Li([state_text, f" ({outcome['total_count']} occurrences across {outcome['distinct_patients']} patients)"])
                )
            
            action_panels.append(
                html.Div([
                    html.H4([action_text, f" ({act['count']} occurrences across {act['patient_count']} patients)"]),
                    html.Ul(outcome_lis)
                ], style={'border': '1px solid #ddd', 'padding': '10px', 'margin': '5px'})
            )
        details.append(html.Div(action_panels))
    else:
        details.append(html.P("No available action transitions for this state combination."))
    
    return html.Div(details)

# Run the app
app.run(debug=True)

In [ ]:
# read models/training_log/eval_trajectories_DiscreteSAC_train_epoch0.csv models/training_log/eval_trajectories_DoubleDQN_train_epoch0.csv
transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteSAC_train_epoch9.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
transition_history_df

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Patient Trajectory Visualization"),
    dcc.Dropdown(
        id='stay-id-dropdown',
        options=[{'label': str(sid), 'value': sid} for sid in sorted(transition_history_df['stay_id'].unique())],
        value=sorted(transition_history_df['stay_id'].unique())[0],
        clearable=False
    ),
    # Added RadioItems for baseline toggle
    dcc.RadioItems(
        id='consider-baseline',
        options=[
            {'label': 'Consider Baseline (Gender, Age)', 'value': 'yes'},
            {'label': 'Ignore Baseline', 'value': 'no'}
        ],
        value='no',  # Default to ignoring baseline
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),
    dcc.Graph(id='trajectory-plot'),
    html.Div(
        id='action-details',
        style={
            'margin-top': '20px',
            'backgroundColor': 'white',
            'padding': '15px',
            'border': '1px solid #ccc',
            'borderRadius': '5px',
            'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
            'fontFamily': 'Arial, sans-serif',
            'color': '#333'
        }
    )
], style={'backgroundColor': 'white', 'padding': '20px'})

# Helper function to map discrete bin values to their original ranges
def map_bin_to_range(bin_index, feature, bin_edges_dict):
    """
    Map a discrete bin index to its original range and return the mean of the range.
    Args:
        bin_index (int): The bin index.
        feature (str): The feature name.
        bin_edges_dict (dict): Dictionary containing bin edges for each feature.
    Returns:
        int: The mean of the range, rounded for better visualization.
    """
    if feature not in bin_edges_dict:
        return "N/A"
    edges = bin_edges_dict[feature]
    if bin_index < 0 or bin_index >= len(edges) - 1:
        return "Invalid"
    lower = edges[bin_index]
    upper = edges[bin_index + 1]
    if feature == "fio2":
        # round it to the nearest 5 or 10
        # return int(round(round((lower + upper) / 2) / 5) * 5)
        return int(round(lower))
    else:
        return int(round((lower + upper) / 2))

# Updated update_plot function (only showing the modified part)
@app.callback(
    Output('trajectory-plot', 'figure'),
    Input('stay-id-dropdown', 'value')
)
def update_plot(selected_stay_id):
    patient_df = transition_history_df[transition_history_df['stay_id'] == selected_stay_id].copy()
    patient_df['within_guidelines'] = patient_df.apply(
        lambda row: is_within_guidelines(row, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        axis=1
    )
    
    fig = px.line(
        patient_df,
        x='hours_in',
        y=state_cols,
        title=f'Trajectory for stay_id {selected_stay_id}',
        labels={'value': 'Vital Sign Value', 'variable': 'Vital Sign', 'hours_in': 'Time (hours)'}
    )
    fig.update_traces(mode='lines+markers')
    
    # Updated hover text with ranges
    hover_texts = []
    for _, row in patient_df.iterrows():
        state_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(state_cols, row['S_t'])]
        action_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(action_cols, row['A_t'])]
        hover_text = (
            f"<b>Hour:</b> {row['hours_in']}<br>"
            f"<b>Current State:</b> {format_tuple(row['S_t'])}<br>"
            + "<br>".join(state_ranges) + "<br>"
            f"<b>Current Action:</b> {format_tuple(row['A_t'])}<br>"
            + "<br>".join(action_ranges)
        )
        hover_texts.append(hover_text)
    
    for trace in fig.data:
        trace.customdata = patient_df[['stay_id', 'S_hist', 'A_hist', 'S_t', 'A_t', 'hours_in', 'baseline']].to_dict('records')
        trace.text = hover_texts
        trace.hovertemplate = '%{text}<extra></extra>'
    
    # [Rest of the function remains unchanged: shapes, layout, etc.]
    shapes = []
    for idx, row in patient_df.iterrows():
        t = row['hours_in']
        color = 'rgba(144, 238, 144, 0.3)' if row['within_guidelines'] else 'rgba(255, 99, 71, 0.3)'
        shapes.append(dict(
            type="rect",
            x0=t - 0.5,
            y0=0,
            x1=t + 0.5,
            y1=1,
            xref="x",
            yref="paper",
            fillcolor=color,
            line_width=0,
            layer="below"
        ))
    
    fig.update_layout(
        shapes=shapes,
        clickmode='event+select',
        template='plotly_white'
    )
    return fig

@app.callback(
    Output('action-details', 'children'),
    [Input('trajectory-plot', 'clickData'),
     Input('consider-baseline', 'value')]
)
def display_action_details(clickData, consider_baseline_value):
    if not clickData:
        return html.P("Click a point on the plot to see detailed transition outcomes.")
    
    point = clickData['points'][0]
    data = point['customdata']
    history_state = tuple(data['S_hist'])
    history_action = tuple(data['A_hist'])
    current_state = tuple(data['S_t'])
    current_action = tuple(data['A_t'])
    
    # Map state and action bins to their original ranges and add units
    mapped_current_state = [
        f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
        for feat, bin_idx, unit in zip(state_cols, current_state, state_units)
    ]
    mapped_current_action = [
        f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
        for feat, bin_idx, unit in zip(action_cols, current_action, action_units)
    ]
    
    # Determine whether to consider baseline
    consider_baseline = (consider_baseline_value == 'yes')
    baseline = tuple(data['baseline']) if consider_baseline else None
    
    # Build details with baseline consideration status
    details = [
        html.H3(f"Details for stay_id {data['stay_id']} at Hour {data['hours_in']}"),
        html.P(f"Considering baseline: {'Yes' if consider_baseline else 'No'}"),
        html.P(f"Current State: State ({', '.join(mapped_current_state)})"),
        html.P(f"Current Action: Action ({', '.join(mapped_current_action)})")
    ]
    
    # Get possible actions and outcomes with baseline parameter
    possible_actions = get_possible_actions(history_state, history_action, current_state, baseline)
    
    # Sort possible_actions by 'count' in descending order
    possible_actions = sorted(possible_actions, key=lambda x: x['count'], reverse=True)
    
    if possible_actions:
        action_panels = []
        for act in possible_actions:
            action_tuple = act['action']
            mapped_action = [
                f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
                for feat, bin_idx, unit in zip(action_cols, action_tuple, action_units)
            ]
            count_not_satisfy = count_not_within_guidelines(action_tuple, action_cols, min_max_values_guidelines_bin_dict)
            bg_color = 'lightgreen' if count_not_satisfy == 0 else 'lightsalmon'
            action_text = html.Span(
                f"Action ({', '.join(mapped_action)})",
                style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
            )
            
            outcomes = get_action_outcomes(history_state, history_action, current_state, action_tuple, baseline)
            outcome_lis = []
            for outcome in outcomes:
                state_tuple = outcome['S_t+1']
                mapped_state = [
                    f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
                    for feat, bin_idx, unit in zip(state_cols, state_tuple, state_units)
                ]
                count_not_satisfy = count_not_within_guidelines(state_tuple, state_cols, min_max_values_guidelines_bin_dict)
                bg_color = 'lightgreen' if count_not_satisfy == 0 else 'orange'
                state_text = html.Span(
                    f"State ({', '.join(mapped_state)})",
                    style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
                )
                outcome_lis.append(
                    html.Li([state_text, f" ({outcome['total_count']} occurrences across {outcome['distinct_patients']} patients)"])
                )
            
            action_panels.append(
                html.Div([
                    html.H4([action_text, f" ({act['count']} occurrences across {act['patient_count']} patients)"]),
                    html.Ul(outcome_lis)
                ], style={'border': '1px solid #ddd', 'padding': '10px', 'margin': '5px'})
            )
        details.append(html.Div(action_panels))
    else:
        details.append(html.P("No available action transitions for this state combination."))
    
    return html.Div(details)

app.run(debug=True)


#### Plotly with Action (change color align as trajectory demo cases)

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Patient Trajectory Visualization"),
    dcc.Dropdown(
        id='stay-id-dropdown',
        options=[{'label': str(sid), 'value': sid} for sid in sorted(transition_history_df['stay_id'].unique())],
        value=sorted(transition_history_df['stay_id'].unique())[0],
        clearable=False
    ),
    # Added RadioItems for baseline toggle
    dcc.RadioItems(
        id='consider-baseline',
        options=[
            {'label': 'Consider Baseline (Gender, Age)', 'value': 'yes'},
            {'label': 'Ignore Baseline', 'value': 'no'}
        ],
        value='no',  # Default to ignoring baseline
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),
    dcc.Graph(id='trajectory-plot'),
    html.Div(
        id='action-details',
        style={
            'margin-top': '20px',
            'backgroundColor': 'white',
            'padding': '15px',
            'border': '1px solid #ccc',
            'borderRadius': '5px',
            'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
            'fontFamily': 'Arial, sans-serif',
            'color': '#333'
        }
    )
], style={'backgroundColor': 'white', 'padding': '20px'})

# Helper function to map discrete bin values to their original ranges
def map_bin_to_range(bin_index, feature, bin_edges_dict):
    """
    Map a discrete bin index to its original range and return the mean of the range.
    Args:
        bin_index (int): The bin index.
        feature (str): The feature name.
        bin_edges_dict (dict): Dictionary containing bin edges for each feature.
    Returns:
        int: The mean of the range, rounded for better visualization.
    """
    if feature not in bin_edges_dict:
        return "N/A"
    edges = bin_edges_dict[feature]
    if bin_index < 0 or bin_index >= len(edges) - 1:
        return "Invalid"
    lower = edges[bin_index]
    upper = edges[bin_index + 1]
    if feature == "fio2":
        # round it to the nearest 5 or 10
        # return int(round(round((lower + upper) / 2) / 5) * 5)
        return int(round(lower))
    else:
        return int(round((lower + upper) / 2))

# Updated update_plot function (only showing the modified part)
@app.callback(
    Output('trajectory-plot', 'figure'),
    Input('stay-id-dropdown', 'value')
)
def update_plot(selected_stay_id):
    patient_df = transition_history_df[transition_history_df['stay_id'] == selected_stay_id].copy()
    patient_df['within_guidelines'] = patient_df.apply(
        lambda row: is_within_guidelines(row, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        axis=1
    )
    
    # Combine state and action columns for plotting
    combined_cols = state_cols + action_cols
    patient_df_melted = patient_df.melt(
        id_vars=['hours_in', 'within_guidelines'],
        value_vars=combined_cols,
        var_name='variable',
        value_name='value'
    )
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'] # change colors as trajectory demo cases

    # Create the line plot
    fig = px.line(
        patient_df_melted,
        x='hours_in',
        y='value',
        color='variable',
        title=f'Trajectory for stay_id {selected_stay_id}',
        labels={'value': 'Value', 'variable': 'Feature', 'hours_in': 'Time (hours)'},
        color_discrete_sequence=colors # change colors as trajectory demo cases
    )
    fig.update_traces(mode='lines+markers')
    
    # Updated hover text with ranges
    hover_texts = []
    for _, row in patient_df.iterrows():
        state_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(state_cols, row['S_t'])]
        action_ranges = [get_range_str(feat, bin_idx) for feat, bin_idx in zip(action_cols, row['A_t'])]
        hover_text = (
            f"<b>Hour:</b> {row['hours_in']}<br>"
            f"<b>Current State:</b> {format_tuple(row['S_t'])}<br>"
            + "<br>".join(state_ranges) + "<br>"
            f"<b>Current Action:</b> {format_tuple(row['A_t'])}<br>"
            + "<br>".join(action_ranges)
        )
        hover_texts.append(hover_text)
    
    for trace in fig.data:
        trace.customdata = patient_df[['stay_id', 'S_hist', 'A_hist', 'S_t', 'A_t', 'hours_in', 'baseline']].to_dict('records')
        trace.text = hover_texts
        trace.hovertemplate = '%{text}<extra></extra>'
    
    # Add background shapes to indicate guideline adherence
    shapes = []
    for idx, row in patient_df.iterrows():
        t = row['hours_in']
        color = 'rgba(144, 238, 144, 0.3)' if row['within_guidelines'] else 'rgba(255, 99, 71, 0.3)'
        shapes.append(dict(
            type="rect",
            x0=t - 0.5,
            y0=0,
            x1=t + 0.5,
            y1=1,
            xref="x",
            yref="paper",
            fillcolor=color,
            line_width=0,
            layer="below"
        ))
    
    fig.update_layout(
        shapes=shapes,
        clickmode='event+select',
        template='plotly_white'
    )
    return fig

@app.callback(
    Output('action-details', 'children'),
    [Input('trajectory-plot', 'clickData'),
     Input('consider-baseline', 'value')]
)
def display_action_details(clickData, consider_baseline_value):
    if not clickData:
        return html.P("Click a point on the plot to see detailed transition outcomes.")
    
    point = clickData['points'][0]
    data = point['customdata']
    history_state = tuple(data['S_hist'])
    history_action = tuple(data['A_hist'])
    current_state = tuple(data['S_t'])
    current_action = tuple(data['A_t'])
    
    # Map state and action bins to their original ranges and add units
    mapped_current_state = [
        f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
        for feat, bin_idx, unit in zip(state_cols, current_state, state_units)
    ]
    mapped_current_action = [
        f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
        for feat, bin_idx, unit in zip(action_cols, current_action, action_units)
    ]
    
    # Determine whether to consider baseline
    consider_baseline = (consider_baseline_value == 'yes')
    baseline = tuple(data['baseline']) if consider_baseline else None
    
    # Build details with baseline consideration status
    details = [
        html.H3(f"Details for stay_id {data['stay_id']} at Hour {data['hours_in']}"),
        html.P(f"Considering baseline: {'Yes' if consider_baseline else 'No'}"),
        html.P(f"Current State: State ({', '.join(mapped_current_state)})"),
        html.P(f"Current Action: Action ({', '.join(mapped_current_action)})")
    ]
    
    # Get possible actions and outcomes with baseline parameter
    possible_actions = get_possible_actions(history_state, history_action, current_state, baseline)
    
    # Sort possible_actions by 'count' in descending order
    possible_actions = sorted(possible_actions, key=lambda x: x['count'], reverse=True)
    
    if possible_actions:
        action_panels = []
        for act in possible_actions:
            action_tuple = act['action']
            mapped_action = [
                f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
                for feat, bin_idx, unit in zip(action_cols, action_tuple, action_units)
            ]
            count_not_satisfy = count_not_within_guidelines(action_tuple, action_cols, min_max_values_guidelines_bin_dict)
            bg_color = 'lightgreen' if count_not_satisfy == 0 else 'lightsalmon'
            action_text = html.Span(
                f"Action ({', '.join(mapped_action)})",
                style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
            )
            
            outcomes = get_action_outcomes(history_state, history_action, current_state, action_tuple, baseline)
            outcome_lis = []
            for outcome in outcomes:
                state_tuple = outcome['S_t+1']
                mapped_state = [
                    f"{map_bin_to_range(bin_idx, feat, bin_edges_dict)} {unit}"
                    for feat, bin_idx, unit in zip(state_cols, state_tuple, state_units)
                ]
                count_not_satisfy = count_not_within_guidelines(state_tuple, state_cols, min_max_values_guidelines_bin_dict)
                bg_color = 'lightgreen' if count_not_satisfy == 0 else 'orange'
                state_text = html.Span(
                    f"State ({', '.join(mapped_state)})",
                    style={'backgroundColor': bg_color, 'padding': '2px 5px', 'borderRadius': '3px'}
                )
                outcome_lis.append(
                    html.Li([state_text, f" ({outcome['total_count']} occurrences across {outcome['distinct_patients']} patients)"])
                )
            
            action_panels.append(
                html.Div([
                    html.H4([action_text, f" ({act['count']} occurrences across {act['patient_count']} patients)"]),
                    html.Ul(outcome_lis)
                ], style={'border': '1px solid #ddd', 'padding': '10px', 'margin': '5px'})
            )
        details.append(html.Div(action_panels))
    else:
        details.append(html.P("No available action transitions for this state combination."))
    
    return html.Div(details)

app.run(debug=True)


In [ ]:
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteSAC_train_epoch0.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DQN_train_epoch0.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DoubleDQN_train_epoch0.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteSAC_train_epoch0.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteBCQ_train_epoch9.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteCQL_train_epoch9.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df = create_transition_history(pd.read_csv(f'../models/training_log_backup/eval_trajectories_DiscreteBC_train_epoch0.csv'), obs_hrs-1, state_cols, action_cols, baseline_cols)
# transition_history_df

In [ ]:
bin_edges_dict_s2

In [ ]:
min_max_values_guidelines

### Visualize from hours 0, without click for possible action

In [ ]:
algo_list = ["DiscreteBC", "NFQ", "DQN", "DoubleDQN", "DiscreteSAC", "DiscreteBCQ", "DiscreteCQL"]
data_source_list = ["train", "test", "eICU"]
data_source = "test"
epoch = 1
gen_stay_id_num = 66

In [ ]:
# concate different algorithms' transition history dataframes, and also update stay_id by concate f"_{algo}" at the end of original stay_id
merged_rl_algo_df = pd.DataFrame()
for algo in algo_list:
    rl_algo_df = pd.read_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_{data_source}_epoch{epoch}.csv')
    rl_algo_df['stay_id'] = rl_algo_df['stay_id'].astype(str) + f'_{algo}'
    merged_rl_algo_df = pd.concat([
        merged_rl_algo_df,
        rl_algo_df
    ])
merged_rl_algo_df

In [ ]:
# concate different algorithms' transition history dataframes, and also update stay_id by concate f"_{algo}" at the end of original stay_id
# for all epoch merge into one dataframe, with given same init e.g., f"{epoch}_small_reward_66"
merged_rl_algo_df = pd.DataFrame()
for epoch in range(20):
    for algo in algo_list:
        rl_algo_df = pd.read_csv(f'../models/training_log/{EXP_FOLDER_PREFIX}/eval_trajectories_{algo}_{data_source}_epoch{epoch}.csv')
        # "stay_id" == f"{epoch}_small_reward_66"
        rl_algo_df = rl_algo_df[rl_algo_df["stay_id"] == f"{epoch}_small_reward_{gen_stay_id_num}"]
        rl_algo_df['stay_id'] = rl_algo_df['stay_id'].astype(str) + f'_{algo}'
        merged_rl_algo_df = pd.concat([
            merged_rl_algo_df,
            rl_algo_df
        ])
merged_rl_algo_df

In [ ]:
# merged_rl_algo_with_physician_policy_df
# algo = "naive_agent"
# physician_policy_df = pd.read_csv(f'../models/training_log/naive_agent/eval_trajectories_{algo}_{data_source}.csv')
# physician_policy_df["stay_id"] = f"{epoch}_" + physician_policy_df["stay_id"].astype(str) + "_physician_policy"
# physician_policy_df

# merged_rl_algo_with_physician_policy_df
algo = "naive_agent"
physician_policy_df = pd.read_csv(f'../models/training_log/naive_agent/eval_trajectories_{algo}_{data_source}.csv')
physician_policy_df = physician_policy_df[physician_policy_df["stay_id"] == f"small_reward_{gen_stay_id_num}"]
physician_policy_df["stay_id"] = f"{epoch}_" + physician_policy_df["stay_id"].astype(str) + "_physician_policy"
physician_policy_df

In [ ]:
merged_rl_algo_with_physician_policy_df = pd.concat([
    merged_rl_algo_df,
    physician_policy_df
])
merged_rl_algo_with_physician_policy_df

### Visualize Trajectory compared train / test / eICU, with merged_rl_algo_with_physician_policy_df

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def is_within_guidelines_updated(current_state, current_action, previous_state, previous_action, 
                                state_cols, action_cols, min_max_values_guidelines_bin_dict, stable_threshold=2):
    """
    Updated guideline check based on weaning criteria from Part 2.
    Returns True if all conditions for weaning_count += 1 are met:
    - state_penalty == 0 (all states within guidelines)
    - action_penalty == 0 (all actions within guidelines) 
    - state_stable_penalty == 0 (state changes below threshold)
    - action_stable_penalty == 0 (action changes below threshold)
    """
    
    # Check state guidelines (state_penalty == 0)
    state_within_guidelines = all(
        min_max_values_guidelines_bin_dict[col]['min'] <= current_state[i] <= min_max_values_guidelines_bin_dict[col]['max']
        for i, col in enumerate(state_cols)
        if col in min_max_values_guidelines_bin_dict
    )
    
    # Check action guidelines (action_penalty == 0)
    action_within_guidelines = all(
        min_max_values_guidelines_bin_dict[col]['min'] <= current_action[i] <= min_max_values_guidelines_bin_dict[col]['max']
        for i, col in enumerate(action_cols)
        if col in min_max_values_guidelines_bin_dict
    )
    
    # For the first timestep, we don't have previous state/action to compare
    # so we only check state and action guidelines
    if previous_action is None or previous_state is None:
        return state_within_guidelines and action_within_guidelines
    
    # Check state stability (state_stable_penalty == 0)
    state_stable = all(
        abs(previous_state[i] - current_state[i]) < stable_threshold
        for i in range(len(state_cols))
    )
    
    # Check action stability (action_stable_penalty == 0)
    action_stable = all(
        abs(previous_action[i] - current_action[i]) < stable_threshold * 2
        for i in range(len(action_cols))
    )
    
    return state_within_guidelines and action_within_guidelines and state_stable and action_stable

def prepare_trajectory_data(df, state_cols, action_cols, min_max_values_guidelines_bin_dict, 
                           obs_hrs=3, stable_threshold=2):
    """
    Prepare trajectory data with guideline compliance from hour 0.
    """
    trajectory_data = []
    
    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('hours_in').reset_index(drop=True)
        
        # Start from hour 0 instead of obs_hrs-1
        for i in range(len(group)):
            row = group.iloc[i]
            
            # Get current state and action
            current_state = tuple(row[col] for col in state_cols)
            current_action = tuple(row[col] for col in action_cols)
            
            # Get previous state and action if available
            previous_state = None
            previous_action = None
            if i > 0:
                prev_row = group.iloc[i-1]
                previous_state = tuple(prev_row[col] for col in state_cols)
                previous_action = tuple(prev_row[col] for col in action_cols)
            
            # Check if within guidelines using updated criteria
            within_guidelines = is_within_guidelines_updated(
                current_state, current_action, previous_state, previous_action,
                state_cols, action_cols, min_max_values_guidelines_bin_dict, stable_threshold
            )
            
            # Add to trajectory data
            traj_point = {
                'stay_id': stay_id,
                'hours_in': row['hours_in'],
                'within_guidelines': within_guidelines
            }
            
            # Add state and action values
            for j, col in enumerate(state_cols):
                traj_point[col] = current_state[j]
            for j, col in enumerate(action_cols):
                traj_point[col] = current_action[j]
                
            trajectory_data.append(traj_point)
    
    return pd.DataFrame(trajectory_data)

def map_bin_to_range(bin_index, feature, bin_edges_dict):
    """
    Map a discrete bin index to its original range and return the mean of the range.
    """
    if feature not in bin_edges_dict:
        return "N/A"
    edges = bin_edges_dict[feature]
    if bin_index < 0 or bin_index >= len(edges) - 1:
        return "Invalid"
    lower = edges[bin_index]
    upper = edges[bin_index + 1]
    if feature == "fio2":
        return int(round(lower))
    else:
        return int(round((lower + upper) / 2))

def create_trajectory_app(train_df, test_df, eICU_disc, rl_algo_df, 
                         state_cols, action_cols, state_units, action_units,
                         min_max_values_guidelines_bin_dict, bin_edges_dict):
    """
    Create Dash app for trajectory visualization.
    """
    
    # Prepare data for all datasets
    datasets = {
        'Train': prepare_trajectory_data(train_df, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        'Test': prepare_trajectory_data(test_df, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        'eICU': prepare_trajectory_data(eICU_disc, state_cols, action_cols, min_max_values_guidelines_bin_dict),
        'RL_Algo': prepare_trajectory_data(rl_algo_df, state_cols, action_cols, min_max_values_guidelines_bin_dict)
    }
    
    # Combine all stay_ids for dropdown
    all_stay_ids = []
    for dataset_name, df in datasets.items():
        for stay_id in df['stay_id'].unique():
            all_stay_ids.append(f"{dataset_name}_{stay_id}")
    
    app = dash.Dash(__name__)
    
    app.layout = html.Div([
        html.H1("Patient Trajectory Visualization - Multi-Dataset Comparison"),
        
        html.Div([
            html.Label("Select Dataset:"),
            dcc.Dropdown(
                id='dataset-dropdown',
                options=[
                    {'label': 'Train Dataset', 'value': 'Train'},
                    {'label': 'Test Dataset', 'value': 'Test'},
                    {'label': 'eICU Dataset', 'value': 'eICU'},
                    {'label': 'RL Algorithm Results', 'value': 'RL_Algo'}
                ],
                value='Train',
                clearable=False
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Stay ID:"),
            dcc.Dropdown(
                id='stay-id-dropdown',
                clearable=False
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
        
        dcc.Graph(id='trajectory-plot'),
        
        html.Div(id='trajectory-summary', style={
            'margin-top': '20px',
            'backgroundColor': 'white',
            'padding': '15px',
            'border': '1px solid #ccc',
            'borderRadius': '5px',
            'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
            'fontFamily': 'Arial, sans-serif',
            'color': '#333'
        })
        
    ], style={'backgroundColor': 'white', 'padding': '20px'})
    
    @app.callback(
        Output('stay-id-dropdown', 'options'),
        Output('stay-id-dropdown', 'value'),
        Input('dataset-dropdown', 'value')
    )
    def update_stay_id_options(selected_dataset):
        if selected_dataset in datasets:
            stay_ids = sorted(datasets[selected_dataset]['stay_id'].unique())
            options = [{'label': str(sid), 'value': sid} for sid in stay_ids]
            value = stay_ids[0] if stay_ids else None
            return options, value
        return [], None
    
    @app.callback(
        Output('trajectory-plot', 'figure'),
        [Input('dataset-dropdown', 'value'),
         Input('stay-id-dropdown', 'value')]
    )
    def update_plot(selected_dataset, selected_stay_id):
        if not selected_dataset or not selected_stay_id or selected_dataset not in datasets:
            return go.Figure()
        
        patient_df = datasets[selected_dataset][
            datasets[selected_dataset]['stay_id'] == selected_stay_id
        ].copy()
        
        if patient_df.empty:
            return go.Figure()
        
        # Prepare data for plotting
        combined_cols = state_cols + action_cols
        
        # Melt the dataframe for line plotting
        patient_df_melted = patient_df.melt(
            id_vars=['hours_in', 'within_guidelines', 'stay_id'],
            value_vars=combined_cols,
            var_name='variable',
            value_name='value'
        )
        
        # Create color mapping
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
        
        # Create the line plot
        fig = px.line(
            patient_df_melted,
            x='hours_in',
            y='value',
            color='variable',
            title=f'Trajectory for {selected_dataset} - Stay ID: {selected_stay_id}',
            labels={'value': 'Bin Index', 'variable': 'Feature', 'hours_in': 'Time (hours)'},
            color_discrete_sequence=colors
        )
        
        fig.update_traces(mode='lines+markers')
        
        # Add background shapes to indicate guideline adherence
        shapes = []
        for _, row in patient_df.iterrows():
            t = row['hours_in']
            color = 'rgba(144, 238, 144, 0.3)' if row['within_guidelines'] else 'rgba(255, 99, 71, 0.3)'
            shapes.append(dict(
                type="rect",
                x0=t - 0.5,
                y0=0,
                x1=t + 0.5,
                y1=1,
                xref="x",
                yref="paper",
                fillcolor=color,
                line_width=0,
                layer="below"
            ))
        
        fig.update_layout(
            shapes=shapes,
            template='plotly_white',
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        return fig
    
    @app.callback(
        Output('trajectory-summary', 'children'),
        [Input('dataset-dropdown', 'value'),
         Input('stay-id-dropdown', 'value')]
    )
    def update_summary(selected_dataset, selected_stay_id):
        if not selected_dataset or not selected_stay_id or selected_dataset not in datasets:
            return html.P("Select a dataset and stay ID to see trajectory summary.")
        
        patient_df = datasets[selected_dataset][
            datasets[selected_dataset]['stay_id'] == selected_stay_id
        ].copy()
        
        if patient_df.empty:
            return html.P("No data available for selected stay ID.")
        
        # Calculate summary statistics
        total_hours = len(patient_df)
        hours_within_guidelines = patient_df['within_guidelines'].sum()
        percentage_compliant = (hours_within_guidelines / total_hours) * 100
        
        # Count consecutive weaning periods
        weaning_sequences = []
        current_sequence = 0
        for compliant in patient_df['within_guidelines']:
            if compliant:
                current_sequence += 1
            else:
                if current_sequence > 0:
                    weaning_sequences.append(current_sequence)
                current_sequence = 0
        if current_sequence > 0:
            weaning_sequences.append(current_sequence)
        
        max_consecutive_weaning = max(weaning_sequences) if weaning_sequences else 0
        weaning_achieved = max_consecutive_weaning >= 6
        
        # Get final state and action values with units
        final_row = patient_df.iloc[-1]
        final_state_display = []
        final_action_display = []
        
        for i, col in enumerate(state_cols):
            mapped_value = map_bin_to_range(final_row[col], col, bin_edges_dict)
            unit = state_units[i] if i < len(state_units) else ""
            final_state_display.append(f"{col}: {mapped_value} {unit}")
        
        for i, col in enumerate(action_cols):
            mapped_value = map_bin_to_range(final_row[col], col, bin_edges_dict)
            unit = action_units[i] if i < len(action_units) else ""
            final_action_display.append(f"{col}: {mapped_value} {unit}")
        
        summary = [
            html.H3(f"Trajectory Summary - {selected_dataset} Dataset"),
            html.P(f"Stay ID: {selected_stay_id}"),
            html.P(f"Total Duration: {total_hours} hours"),
            html.P(f"Hours Within Guidelines: {hours_within_guidelines}/{total_hours} ({percentage_compliant:.1f}%)"),
            html.P(f"Maximum Consecutive Weaning Hours: {max_consecutive_weaning}"),
            html.P(f"Weaning Achieved (≥6 consecutive hours): {'Yes' if weaning_achieved else 'No'}",
                  style={'color': 'green' if weaning_achieved else 'red', 'fontWeight': 'bold'}),
            html.Hr(),
            html.H4("Final State:"),
            html.Ul([html.Li(item) for item in final_state_display]),
            html.H4("Final Action:"),
            html.Ul([html.Li(item) for item in final_action_display])
        ]
        
        return html.Div(summary)
    
    return app

# Example usage:
def run_trajectory_visualization(train_df, test_df, eICU_disc, rl_algo_df,
                               state_cols, action_cols, state_units, action_units,
                               min_max_values_guidelines_bin_dict, bin_edges_dict):
    """
    Run the trajectory visualization app.
    """
    app = create_trajectory_app(
        train_df, test_df, eICU_disc, rl_algo_df,
        state_cols, action_cols, state_units, action_units,
        min_max_values_guidelines_bin_dict, bin_edges_dict
    )
    
    app.run(debug=True)

# To use this, you would call:
run_trajectory_visualization(train_df[train_df["stay_id"] == 37768645], test_df[:1000], eICU_disc[:1000], merged_rl_algo_with_physician_policy_df,
                           state_cols, action_cols, state_units, action_units,
                           min_max_values_guidelines_bin_dict, bin_edges_dict) # 37768645 (small_reward_66), 31726457 (small_reward_88)